In [3]:
# --- 1. Import Libraries ---
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from IPython.display import display, clear_output
import io
import numpy as np # For best-fit line

# --- 2. Database Helper Functions ---
DB_NAME = 'Dataset.db'

def get_table_names():
    """Fetches all table names."""
    try:
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = [table[0].strip() for table in cursor.fetchall() if table[0] != 'sqlite_sequence']
        conn.close()
        return tables
    except Exception as e:
        print(f"Database error: {e}")
        return []

def get_column_names(table_name):
    """Fetches all column names for a table."""
    if not table_name:
        return []
    try:
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        cursor.execute(f'PRAGMA table_info("{table_name}")')
        columns = [col[1].strip() for col in cursor.fetchall()]
        conn.close()
        return columns
    except Exception as e:
        print(f"Database error: {e}")
        return []

# --- 3. Create GUI Widgets ---
table_dropdown = widgets.Dropdown(description='Select Table:')
x_axis_dropdown = widgets.Dropdown(description='X-Axis:')
y_axis_dropdown = widgets.Dropdown(description='Y-Axis:')
plot_type_dropdown = widgets.Dropdown(options=['Scatter', 'Bar'], description='Plot Type:')
best_fit_checkbox = widgets.Checkbox(value=True, description='Add Best Fit Curve')
generate_button = widgets.Button(description='Generate Plot')
plot_output = widgets.Output()

# --- 4. Define Event Handlers ---
def on_table_change(change):
    """Called when the table dropdown changes."""
    table = change['new']
    columns = get_column_names(table)
    x_axis_dropdown.options = columns
    y_axis_dropdown.options = columns
    if columns:
        x_axis_dropdown.value = columns[0]
        y_axis_dropdown.value = columns[1] if len(columns) > 1 else columns[0]

def on_plot_type_change(change):
    """Called when the plot type dropdown changes."""
    if change['new'] == 'Scatter':
        best_fit_checkbox.layout.display = 'flex'
        y_axis_dropdown.description = 'Y-Axis:'
    else:
        best_fit_checkbox.layout.display = 'none'
        y_axis_dropdown.description = 'Y-Axis (for Count):'

def on_generate_click(b):
    """Called when the 'Generate Plot' button is clicked."""
    with plot_output:
        clear_output(wait=True)  # Clear the old plot
        
        try:
            # --- Get all user selections ---
            table = table_dropdown.value
            x_col = x_axis_dropdown.value
            y_col = y_axis_dropdown.value 
            plot_type = plot_type_dropdown.value
            show_best_fit = best_fit_checkbox.value

            # --- Fetch data (including 'age' for filtering) ---
            all_columns = get_column_names(table)
            cols_to_fetch = set([x_col, y_col])
            if 'age' in all_columns:
                cols_to_fetch.add('age')
            
            col_str = '", "'.join(cols_to_fetch)
            query = f'SELECT "{col_str}" FROM "{table}"'
            
            conn = sqlite3.connect(DB_NAME)
            df = pd.read_sql_query(query, conn)
            conn.close()

            # --- Apply Age Filter (18-22) ---
            if 'age' in df.columns:
                df['age'] = pd.to_numeric(df['age'], errors='coerce')
                df = df.dropna(subset=['age'])
                df['age'] = df['age'].astype(float)
                
                df_filtered = df[(df['age'] >= 18) & (df['age'] <= 22)]
                
                if not df_filtered.empty:
                    df = df_filtered
                    print(f"Filter applied: Showing {len(df)} students aged 18-22.")
                else:
                    print("Note: No data found for ages 18-22. Showing all ages.")

            # --- Clean data based on plot type ---
            df_cleaned = df.copy()
            if plot_type == 'Scatter':
                df_cleaned[x_col] = pd.to_numeric(df_cleaned[x_col], errors='coerce')
                df_cleaned[y_col] = pd.to_numeric(df_cleaned[y_col], errors='coerce')
                df_cleaned = df_cleaned.dropna(subset=[x_col, y_col])
            elif plot_type == 'Bar':
                df_cleaned = df_cleaned.dropna(subset=[x_col])
            
            if df_cleaned.empty:
                print("Error: No valid data to plot after cleaning/filtering.")
                return

            # --- Plotting ---
            fig, ax = plt.subplots()
            
            y_label = y_col if plot_type == 'Scatter' else 'Count'
            title = f"{y_col} vs. {x_col}" if plot_type == 'Scatter' else f"Count of Students vs. {x_col}"
            
            if plot_type == 'Scatter':
                ax.scatter(df_cleaned[x_col], df_cleaned[y_col], alpha=0.7)
                if show_best_fit and len(df_cleaned[x_col]) > 1:
                    m, b = np.polyfit(df_cleaned[x_col], df_cleaned[y_col], 1)
                    ax.plot(df_cleaned[x_col], m * df_cleaned[x_col] + b, color='red')
            
            elif plot_type == 'Bar':
                grouped_data = df_cleaned.groupby(df_cleaned[x_col].astype(str))[y_col].count()
                ax.bar(grouped_data.index, grouped_data.values)
                if len(grouped_data) > 20:
                    ax.set_xticks([])
                else:
                    plt.xticks(rotation=45, ha='right')

            # --- Set labels and save the plot ---
            ax.set_xlabel(x_col)
            ax.set_ylabel(y_label) 
            ax.set_title(f"{title} (Table: {table})")
            ax.grid(True)
            plt.tight_layout()
            
            buf = io.BytesIO()
            fig.savefig(buf, format='png')
            buf.seek(0)
            
            image_widget = widgets.Image(value=buf.read(), format='png')
            display(image_widget)
            
            buf.close()
            plt.close(fig)

        except Exception as e:
            print(f"An unexpected error occurred: {e}")

# --- 5. Wire Up Widgets ---
table_dropdown.observe(on_table_change, names='value')
plot_type_dropdown.observe(on_plot_type_change, names='value')
generate_button.on_click(on_generate_click)

# --- 6. Create Layout and Display GUI ---
grid = GridspecLayout(3, 2)
grid[0, 0] = table_dropdown
grid[0, 1] = plot_type_dropdown
grid[1, 0] = x_axis_dropdown
grid[1, 1] = y_axis_dropdown
grid[2, 0] = generate_button
grid[2, 1] = best_fit_checkbox

gui_layout = widgets.VBox([grid, plot_output])

# --- 7. Initial Population ---
table_names = get_table_names()
if table_names:
    table_dropdown.options = table_names
    table_dropdown.value = table_names[0]
else:
    with plot_output:
        print("Warning: 'Dataset.db' not found or is empty.")

on_plot_type_change({'new': plot_type_dropdown.value})

# --- 8. Display the GUI ---
display(gui_layout)

In [2]:
import sqlite3
import csv
import sys

DB_NAME = 'Dataset.db'
CSV_NAME = 'StressLevelDataset.csv'

try:
    # --- 1. Read all data from the CSV ---
    data_rows = []
    with open(CSV_NAME, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader) # Skip the header row
        
        for row in reader:
            data_rows.append(row)
    
    print(f"Read {len(data_rows)} rows from {CSV_NAME}.")
    
    # Check if data matches the 843 student_ids in the Students table
    if len(data_rows) != 843:
        print(f"Warning: CSV has {len(data_rows)} rows, but 'Students' table has 843. This may cause issues.")

    # --- 2. Connect to the database ---
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # --- 3. Clear old data from the tables ---
    print("Clearing old data from tables...")
    cursor.execute("DELETE FROM Academic;")
    cursor.execute("DELETE FROM Psychological;")
    cursor.execute("DELETE FROM Physiological;")

    # --- 4. Define column mappings (CSV index -> DB column) ---
    # These indices are based on the 'StressLevelDataset.csv' headers
    psychological_cols = {
        'anxiety_level': 0,
        'self_esteem': 1,
        'mental_health_history': 2,
        'depression': 3
    }
    
    physiological_cols = {
        'headache': 4,
        'blood_pressure': 5,
        'sleep_quality': 6,
        'breathing_problem': 7
    }
    
    academic_cols = {
        'academic_performance': 12,
        'study_load': 13,
        'teacher_student_relationship': 14,
        'future_career_concerns': 15
    }

    # --- 5. Loop through each row and insert data ---
    student_id = 1
    for row in data_rows:
        try:
            # Insert into Psychological
            p_data = (
                student_id, 
                row[psychological_cols['anxiety_level']],
                row[psychological_cols['self_esteem']],
                row[psychological_cols['mental_health_history']],
                row[psychological_cols['depression']]
            )
            cursor.execute("INSERT INTO Psychological (student_id, anxiety_level, self_esteem, mental_health_history, depression) VALUES (?, ?, ?, ?, ?);", p_data)

            # Insert into Physiological
            phys_data = (
                student_id,
                row[physiological_cols['blood_pressure']],
                row[physiological_cols['breathing_problem']],
                row[physiological_cols['sleep_quality']],
                row[physiological_cols['headache']]
            )
            cursor.execute("INSERT INTO Physiological (student_id, blood_pressure, breathing_problem, sleep_quality, headache) VALUES (?, ?, ?, ?, ?);", phys_data)
            
            # Insert into Academic
            a_data = (
                student_id,
                row[academic_cols['academic_performance']],
                row[academic_cols['study_load']],
                row[academic_cols['teacher_student_relationship']],
                row[academic_cols['future_career_concerns']]
            )
            cursor.execute("INSERT INTO Academic (student_id, academic_performance, study_load, teacher_student_relationship, future_career_concerns) VALUES (?, ?, ?, ?, ?);", a_data)
            
            student_id += 1
            
        except IndexError:
            print(f"Skipped row {student_id}. Data was incomplete.")
        except Exception as e:
            print(f"Error on row {student_id}: {e}")
            
    # --- 6. Commit changes and close ---
    conn.commit()
    conn.close()
    
    print("---")
    print(f"Successfully populated all tables for {student_id - 1} students.")
    print("Your database is now correct. Please re-run the GUI code cell.")

except FileNotFoundError:
    print(f"ERROR: '{CSV_NAME}' not found. Make sure it is in the same folder.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Read 1100 rows from StressLevelDataset.csv.
Clearing old data from tables...
---
Successfully populated all tables for 1100 students.
Your database is now correct. Please re-run the GUI code cell.


In [4]:
# --- 1. Import Libraries ---
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from IPython.display import display, clear_output
import io
import numpy as np # For best-fit line

# --- 2. Database Helper Functions ---
DB_NAME = 'Dataset.db'

def get_table_names():
    """Fetches all table names."""
    try:
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = [table[0].strip() for table in cursor.fetchall() if table[0] != 'sqlite_sequence']
        conn.close()
        return tables
    except Exception as e:
        print(f"Database error: {e}")
        return []

def get_column_names(table_name):
    """Fetches all column names for a table."""
    if not table_name:
        return []
    try:
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        cursor.execute(f'PRAGMA table_info("{table_name}")')
        columns = [col[1].strip() for col in cursor.fetchall()]
        conn.close()
        return columns
    except Exception as e:
        print(f"Database error: {e}")
        return []

# --- 3. Create GUI Widgets ---
table_dropdown = widgets.Dropdown(description='Select Table:')
x_axis_dropdown = widgets.Dropdown(description='X-Axis:')
y_axis_dropdown = widgets.Dropdown(description='Y-Axis:')
plot_type_dropdown = widgets.Dropdown(options=['Scatter', 'Bar'], description='Plot Type:')
best_fit_checkbox = widgets.Checkbox(value=True, description='Add Best Fit Curve')
generate_button = widgets.Button(description='Generate Plot')
plot_output = widgets.Output()

# --- 4. Define Event Handlers ---
def on_table_change(change):
    """Called when the table dropdown changes."""
    table = change['new']
    columns = get_column_names(table)
    x_axis_dropdown.options = columns
    y_axis_dropdown.options = columns
    if columns:
        x_axis_dropdown.value = columns[0]
        y_axis_dropdown.value = columns[1] if len(columns) > 1 else columns[0]

def on_plot_type_change(change):
    """Called when the plot type dropdown changes."""
    if change['new'] == 'Scatter':
        best_fit_checkbox.layout.display = 'flex'
        y_axis_dropdown.description = 'Y-Axis:'
    else:
        best_fit_checkbox.layout.display = 'none'
        y_axis_dropdown.description = 'Y-Axis (for Count):'

def on_generate_click(b):
    """Called when the 'Generate Plot' button is clicked."""
    with plot_output:
        clear_output(wait=True)  # Clear the old plot
        
        try:
            # --- Get all user selections ---
            table = table_dropdown.value
            x_col = x_axis_dropdown.value
            y_col = y_axis_dropdown.value 
            plot_type = plot_type_dropdown.value
            show_best_fit = best_fit_checkbox.value

            # --- Fetch data (including 'age' for filtering) ---
            all_columns = get_column_names(table)
            cols_to_fetch = set([x_col, y_col])
            if 'age' in all_columns:
                cols_to_fetch.add('age')
            
            col_str = '", "'.join(cols_to_fetch)
            query = f'SELECT "{col_str}" FROM "{table}"'
            
            conn = sqlite3.connect(DB_NAME)
            df = pd.read_sql_query(query, conn)
            conn.close()

            # --- Apply Strict Age Filter (18-22) ---
            if 'age' in df.columns:
                df['age'] = pd.to_numeric(df['age'], errors='coerce')
                df = df.dropna(subset=['age'])
                df['age'] = df['age'].astype(float)
                
                df_filtered = df[(df['age'] >= 18) & (df['age'] <= 22)]
                
                if not df_filtered.empty:
                    df = df_filtered
                    print(f"Filter applied: Showing {len(df)} students aged 18-22.")
                else:
                    # Set df to the empty filtered list
                    df = df_filtered 
                    print("Filter applied: No students found between ages 18-22.")

            # --- Check if data is empty AFTER filtering ---
            if df.empty:
                print("Error: No data to plot after applying filters.")
                return

            # --- Clean data based on plot type ---
            df_cleaned = df.copy()
            if plot_type == 'Scatter':
                df_cleaned[x_col] = pd.to_numeric(df_cleaned[x_col], errors='coerce')
                df_cleaned[y_col] = pd.to_numeric(df_cleaned[y_col], errors='coerce')
                df_cleaned = df_cleaned.dropna(subset=[x_col, y_col])
            elif plot_type == 'Bar':
                df_cleaned = df_cleaned.dropna(subset=[x_col])
            
            if df_cleaned.empty:
                print("Error: No valid data to plot after cleaning.")
                return

            # --- Plotting ---
            fig, ax = plt.subplots()
            
            y_label = y_col if plot_type == 'Scatter' else 'Count'
            title = f"{y_col} vs. {x_col}" if plot_type == 'Scatter' else f"Count of Students vs. {x_col}"
            
            if plot_type == 'Scatter':
                ax.scatter(df_cleaned[x_col], df_cleaned[y_col], alpha=0.7)
                if show_best_fit and len(df_cleaned[x_col]) > 1:
                    m, b = np.polyfit(df_cleaned[x_col], df_cleaned[y_col], 1)
                    ax.plot(df_cleaned[x_col], m * df_cleaned[x_col] + b, color='red')
            
            elif plot_type == 'Bar':
                grouped_data = df_cleaned.groupby(df_cleaned[x_col].astype(str))[y_col].count()
                ax.bar(grouped_data.index, grouped_data.values)
                if len(grouped_data) > 20:
                    ax.set_xticks([])
                else:
                    plt.xticks(rotation=45, ha='right')

            # --- Set labels and save the plot ---
            ax.set_xlabel(x_col)
            ax.set_ylabel(y_label) 
            ax.set_title(f"{title} (Table: {table})")
            ax.grid(True)
            plt.tight_layout()
            
            buf = io.BytesIO()
            fig.savefig(buf, format='png')
            buf.seek(0)
            
            image_widget = widgets.Image(value=buf.read(), format='png')
            display(image_widget)
            
            buf.close()
            plt.close(fig)

        except Exception as e:
            print(f"An unexpected error occurred: {e}")

# --- 5. Wire Up Widgets ---
table_dropdown.observe(on_table_change, names='value')
plot_type_dropdown.observe(on_plot_type_change, names='value')
generate_button.on_click(on_generate_click)

# --- 6. Create Layout and Display GUI ---
grid = GridspecLayout(3, 2)
grid[0, 0] = table_dropdown
grid[0, 1] = plot_type_dropdown
grid[1, 0] = x_axis_dropdown
grid[1, 1] = y_axis_dropdown
grid[2, 0] = generate_button
grid[2, 1] = best_fit_checkbox

gui_layout = widgets.VBox([grid, plot_output])

# --- 7. Initial Population ---
table_names = get_table_names()
if table_names:
    table_dropdown.options = table_names
    table_dropdown.value = table_names[0]
else:
    with plot_output:
        print("Warning: 'Dataset.db' not found or is empty.")

on_plot_type_change({'new': plot_type_dropdown.value})

# --- 8. Display the GUI ---
display(gui_layout)

In [5]:
import sqlite3
conn = sqlite3.connect('Dataset.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())
conn.close()


[('sqlite_sequence',), ('Students',), ('Academic',), ('Psychological',), ('Physiological',)]


In [6]:
print(get_column_names('Students'))


['student_id', 'age', 'gender']


In [7]:
# --- 1. Import Libraries ---
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

# --- 2. Database Setup ---
# Make sure your 'Dataset.db' file is in the same folder as this notebook
DB_NAME = 'Dataset.db'

# --- 3. Get All Tables from Database ---
try:
    conn = sqlite3.connect(DB_NAME)
    tables = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table' AND name != 'sqlite_sequence';",
        conn
    )
    table_list = tables['name'].tolist()
    conn.close()
except Exception as e:
    print("Error loading database:", e)
    table_list = []

# --- 4. Create Widgets ---
table_dropdown = widgets.Dropdown(options=table_list, description="Table:")
plot_type = widgets.Dropdown(options=["Scatter", "Bar"], description="Plot Type:")
x_dropdown = widgets.Dropdown(description="X-Axis:")
y_dropdown = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style='success')
output_area = widgets.Output()

# --- 5. When user picks a table ---
def update_columns(change):
    clear_output(wait=True)
    display(ui)

    table = table_dropdown.value
    ptype = plot_type.value

    # Load sample data
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f'SELECT * FROM "{table}" LIMIT 100', conn)
    conn.close()

    # Choose columns based on plot type
    if ptype == "Scatter":
        numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
        x_dropdown.options = numeric_cols
        y_dropdown.options = numeric_cols
    else:
        all_cols = df.columns.tolist()
        x_dropdown.options = all_cols
        y_dropdown.options = all_cols

# --- 6. When user clicks "Generate Plot" ---
def generate_plot(b):
    with output_area:
        clear_output(wait=True)

        table = table_dropdown.value
        x_col = x_dropdown.value
        y_col = y_dropdown.value
        ptype = plot_type.value

        if not table or not x_col or not y_col:
            print("⚠️ Please select a table and valid columns.")
            return

        # Load full table data
        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f'SELECT * FROM "{table}"', conn)
        conn.close()

        # Filter by age if available
        if 'age' in df.columns:
            df['age'] = pd.to_numeric(df['age'], errors='coerce')
            df = df[(df['age'] >= 18) & (df['age'] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        if df.empty:
            print("No data available after filtering.")
            return

        # --- Create Plot ---
        fig, ax = plt.subplots()

        if ptype == "Scatter":
            # Convert to numbers and drop missing values
            df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
            df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
            df = df.dropna(subset=[x_col, y_col])

            ax.scatter(df[x_col], df[y_col], color='blue', alpha=0.7)

            # Add best-fit line if checked
            if best_fit.value and len(df) > 1:
                m, b = np.polyfit(df[x_col], df[y_col], 1)
                ax.plot(df[x_col], m * df[x_col] + b, color='red')
            ax.set_ylabel(y_col)

        else:  # Bar plot
            grouped = df.groupby(x_col)[y_col].count()
            grouped.plot(kind='bar', ax=ax, color='skyblue')
            plt.xticks(rotation=45, ha='right')
            ax.set_ylabel("Count")

        ax.set_xlabel(x_col)
        ax.set_title(f"{y_col} vs. {x_col} ({ptype})")
        ax.grid(True)
        plt.tight_layout()

        # Show image inside widget
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        display(widgets.Image(value=buf.read(), format='png'))
        plt.close(fig)

# --- 7. Link Widgets to Events ---
table_dropdown.observe(update_columns, names='value')
plot_type.observe(update_columns, names='value')
generate_button.on_click(generate_plot)

# --- 8. Display Interface ---
ui = widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_dropdown, y_dropdown]),
    widgets.HBox([generate_button, best_fit]),
    output_area
])
display(ui)


In [8]:
# --- 1. Import Libraries ---
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

# --- 2. Database Setup ---
DB_NAME = 'Dataset.db'  # Ensure your database file is in the same directory

# --- 3. Get All Tables from Database ---
try:
    conn = sqlite3.connect(DB_NAME)
    tables = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table' AND name != 'sqlite_sequence';",
        conn
    )
    table_list = tables['name'].tolist()
    conn.close()
except Exception as e:
    print("Error loading database:", e)
    table_list = []

# --- 4. Create Widgets ---
table_dropdown = widgets.Dropdown(options=table_list, description="Table:")
plot_type = widgets.Dropdown(options=["Scatter", "Bar"], description="Plot Type:")
x_dropdown = widgets.Dropdown(description="X-Axis:")
y_dropdown = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style='success')
output_area = widgets.Output()

# --- 5. Update columns when user picks a table or plot type ---
def update_columns(change):
    clear_output(wait=True)
    display(ui)

    table = table_dropdown.value
    ptype = plot_type.value

    if not table:
        return

    # Load sample data
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f'SELECT * FROM "{table}" LIMIT 100', conn)
    conn.close()

    # Choose columns based on plot type
    if ptype == "Scatter":
        numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
        x_dropdown.options = numeric_cols
        y_dropdown.options = numeric_cols
    else:
        all_cols = df.columns.tolist()
        x_dropdown.options = all_cols
        y_dropdown.options = all_cols

# --- 6. When user clicks "Generate Plot" ---
def generate_plot(b):
    with output_area:
        clear_output(wait=True)

        table = table_dropdown.value
        x_col = x_dropdown.value
        y_col = y_dropdown.value
        ptype = plot_type.value

        if not table or not x_col or not y_col:
            print("⚠️ Please select a table and valid columns.")
            return

        # Load full table data
        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f'SELECT * FROM "{table}"', conn)
        conn.close()

        # Filter by age if available
        if 'age' in df.columns:
            df['age'] = pd.to_numeric(df['age'], errors='coerce')
            df = df[(df['age'] >= 18) & (df['age'] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        if df.empty:
            print("No data available after filtering.")
            return

        # --- Create Plot ---
        fig, ax = plt.subplots(figsize=(6, 4))

        if ptype == "Scatter":
            # Convert to numbers and drop missing values
            df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
            df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
            df = df.dropna(subset=[x_col, y_col])

            ax.scatter(df[x_col], df[y_col], color='blue', alpha=0.7)

            # Add best-fit line if checked
            if best_fit.value and len(df) > 1:
                m, b = np.polyfit(df[x_col], df[y_col], 1)
                ax.plot(df[x_col], m * df[x_col] + b, color='red')
            ax.set_ylabel(y_col)

        else:  # --- Bar Plot ---
            # Handle grouped bars when both are categorical (e.g., gender vs. age)
            if df[x_col].dtype == 'object' or df[y_col].dtype == 'object':
                grouped = pd.crosstab(df[x_col], df[y_col])
                grouped.plot(kind='bar', ax=ax, width=0.8, alpha=0.8)
                plt.xticks(rotation=45, ha='right')
                ax.set_ylabel("Count")
                ax.legend(title=y_col)
            else:
                # Default single-variable count bar
                grouped = df.groupby(x_col)[y_col].count()
                grouped.plot(kind='bar', ax=ax, color='skyblue')
                plt.xticks(rotation=45, ha='right')
                ax.set_ylabel("Count")

        # --- Final Plot Formatting ---
        ax.set_xlabel(x_col)
        ax.set_title(f"{y_col} vs. {x_col} ({ptype})")
        ax.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()

        # Show image inside widget
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        display(widgets.Image(value=buf.read(), format='png'))
        plt.close(fig)

# --- 7. Link Widgets to Events ---
table_dropdown.observe(update_columns, names='value')
plot_type.observe(update_columns, names='value')
generate_button.on_click(generate_plot)

# --- 8. Display Interface ---
ui = widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_dropdown, y_dropdown]),
    widgets.HBox([generate_button, best_fit]),
    output_area
])

display(ui)


In [9]:
# --- 1. Import Libraries ---
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

# --- 2. Database Setup ---
DB_NAME = 'Dataset.db'  # Ensure your database file is in the same directory

# --- 3. Get All Tables from Database ---
try:
    conn = sqlite3.connect(DB_NAME)
    tables = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table' AND name != 'sqlite_sequence';",
        conn
    )
    table_list = tables['name'].tolist()
    conn.close()
except Exception as e:
    print("Error loading database:", e)
    table_list = []

# --- 4. Create Widgets ---
table_dropdown = widgets.Dropdown(options=table_list, description="Table:")
plot_type = widgets.Dropdown(options=["Scatter", "Bar"], description="Plot Type:")
x_dropdown = widgets.Dropdown(description="X-Axis:")
y_dropdown = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style='success')
output_area = widgets.Output()

# --- 5. Update columns when user picks a table or plot type ---
def update_columns(change):
    clear_output(wait=True)
    display(ui)

    table = table_dropdown.value
    ptype = plot_type.value

    if not table:
        return

    # Load sample data
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f'SELECT * FROM "{table}" LIMIT 100', conn)
    conn.close()

    # Detect ID-like columns (likely unique identifiers)
    id_like = [col for col in df.columns if 'id' in col.lower() or df[col].nunique() == len(df)]

    # Choose columns for X and Y
    all_cols = df.columns.tolist()
    x_allowed = [col for col in all_cols if col not in id_like]  # exclude IDs only from X
    y_allowed = all_cols  # keep everything, including IDs

    if ptype == "Scatter":
        numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
        x_dropdown.options = [col for col in numeric_cols if col not in id_like]
        y_dropdown.options = numeric_cols
    else:
        x_dropdown.options = x_allowed
        y_dropdown.options = y_allowed

# --- 6. When user clicks "Generate Plot" ---
def generate_plot(b):
    with output_area:
        clear_output(wait=True)

        table = table_dropdown.value
        x_col = x_dropdown.value
        y_col = y_dropdown.value
        ptype = plot_type.value

        if not table or not x_col or not y_col:
            print("⚠️ Please select a table and valid columns.")
            return

        # Load full table data
        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f'SELECT * FROM "{table}"', conn)
        conn.close()

        # Filter by age if available
        if 'age' in df.columns:
            df['age'] = pd.to_numeric(df['age'], errors='coerce')
            df = df[(df['age'] >= 18) & (df['age'] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        if df.empty:
            print("No data available after filtering.")
            return

        # --- Create Plot ---
        fig, ax = plt.subplots(figsize=(6, 4))

        if ptype == "Scatter":
            # Convert to numbers and drop missing values
            df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
            df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
            df = df.dropna(subset=[x_col, y_col])

            ax.scatter(df[x_col], df[y_col], color='blue', alpha=0.7)

            # Add best-fit line if checked
            if best_fit.value and len(df) > 1:
                m, b = np.polyfit(df[x_col], df[y_col], 1)
                ax.plot(df[x_col], m * df[x_col] + b, color='red')
            ax.set_ylabel(y_col)

        else:  # --- Bar Plot ---
            # Handle grouped bars when both are categorical (e.g., gender vs. age)
            if df[x_col].dtype == 'object' or df[y_col].dtype == 'object':
                grouped = pd.crosstab(df[x_col], df[y_col])
                grouped.plot(kind='bar', ax=ax, width=0.8, alpha=0.8)
                plt.xticks(rotation=45, ha='right')
                ax.set_ylabel("Count")
                ax.legend(title=y_col)
            else:
                # Default single-variable count bar
                grouped = df.groupby(x_col)[y_col].count()
                grouped.plot(kind='bar', ax=ax, color='skyblue')
                plt.xticks(rotation=45, ha='right')
                ax.set_ylabel("Count")

        # --- Final Plot Formatting ---
        ax.set_xlabel(x_col)
        ax.set_title(f"{y_col} vs. {x_col} ({ptype})")
        ax.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()

        # Show image inside widget
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        display(widgets.Image(value=buf.read(), format='png'))
        plt.close(fig)

# --- 7. Link Widgets to Events ---
table_dropdown.observe(update_columns, names='value')
plot_type.observe(update_columns, names='value')
generate_button.on_click(generate_plot)

# --- 8. Display Interface ---
ui = widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_dropdown, y_dropdown]),
    widgets.HBox([generate_button, best_fit]),
    output_area
])

display(ui)


In [10]:
# --- 1. Import Libraries ---
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

# --- 2. Database Setup ---
DB_NAME = 'Dataset.db'  # Ensure your database file is in the same directory

# --- 3. Get All Tables from Database ---
try:
    conn = sqlite3.connect(DB_NAME)
    tables = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table' AND name != 'sqlite_sequence';",
        conn
    )
    table_list = tables['name'].tolist()
    conn.close()
except Exception as e:
    print("Error loading database:", e)
    table_list = []

# --- 4. Create Widgets ---
table_dropdown = widgets.Dropdown(options=table_list, description="Table:")
plot_type = widgets.Dropdown(options=["Scatter", "Bar"], description="Plot Type:")
x_dropdown = widgets.Dropdown(description="X-Axis:")
y_dropdown = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style='success')
output_area = widgets.Output()

# --- 5. Update columns when user picks a table or plot type ---
def update_columns(change):
    clear_output(wait=True)
    display(ui)

    table = table_dropdown.value
    ptype = plot_type.value

    if not table:
        return

    # Load sample data
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f'SELECT * FROM "{table}" LIMIT 100', conn)
    conn.close()

    # Choose columns based on plot type
    if ptype == "Scatter":
        numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
        x_dropdown.options = numeric_cols
        y_dropdown.options = numeric_cols
    else:
        all_cols = df.columns.tolist()
        x_dropdown.options = all_cols
        y_dropdown.options = all_cols

# --- 6. When user clicks "Generate Plot" ---
def generate_plot(b):
    with output_area:
        clear_output(wait=True)

        table = table_dropdown.value
        x_col = x_dropdown.value
        y_col = y_dropdown.value
        ptype = plot_type.value

        if not table or not x_col or not y_col:
            print("⚠️ Please select a table and valid columns.")
            return

        # Load full table data
        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f'SELECT * FROM "{table}"', conn)
        conn.close()

        # Filter by age if available
        if 'age' in df.columns:
            df['age'] = pd.to_numeric(df['age'], errors='coerce')
            df = df[(df['age'] >= 18) & (df['age'] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        if df.empty:
            print("No data available after filtering.")
            return

        # --- Create Plot ---
        fig, ax = plt.subplots(figsize=(6, 4))

        if ptype == "Scatter":
            # Convert to numbers and drop missing values
            df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
            df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
            df = df.dropna(subset=[x_col, y_col])

            ax.scatter(df[x_col], df[y_col], color='blue', alpha=0.7)

            # Add best-fit line if checked
            if best_fit.value and len(df) > 1:
                m, b = np.polyfit(df[x_col], df[y_col], 1)
                ax.plot(df[x_col], m * df[x_col] + b, color='red')
            ax.set_ylabel(y_col)

        else:  # --- Bar Plot ---
            # Handle grouped bars when both are categorical (e.g., gender vs. age)
            if df[x_col].dtype == 'object' or df[y_col].dtype == 'object':
                grouped = pd.crosstab(df[x_col], df[y_col])
                grouped.plot(kind='bar', ax=ax, width=0.8, alpha=0.8)
                plt.xticks(rotation=45, ha='right')
                ax.set_ylabel("Count")
                ax.legend(title=y_col)
            else:
                # Default single-variable count bar
                grouped = df.groupby(x_col)[y_col].count()
                grouped.plot(kind='bar', ax=ax, color='skyblue')
                plt.xticks(rotation=45, ha='right')
                ax.set_ylabel("Count")

        # --- Final Plot Formatting ---
        ax.set_xlabel(x_col)
        ax.set_title(f"{y_col} vs. {x_col} ({ptype})")
        ax.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()

        # Show image inside widget
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        display(widgets.Image(value=buf.read(), format='png'))
        plt.close(fig)

# --- 7. Link Widgets to Events ---
table_dropdown.observe(update_columns, names='value')
plot_type.observe(update_columns, names='value')
generate_button.on_click(generate_plot)

# --- 8. Display Interface ---
ui = widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_dropdown, y_dropdown]),
    widgets.HBox([generate_button, best_fit]),
    output_area
])

display(ui)


In [11]:
# --- 1. Import Libraries ---
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

# --- 2. Database Setup ---
DB_NAME = 'Dataset.db'  # Ensure your database file is in the same directory

# --- 3. Get All Tables from Database ---
try:
    conn = sqlite3.connect(DB_NAME)
    tables = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table' AND name != 'sqlite_sequence';",
        conn
    )
    table_list = tables['name'].tolist()
    conn.close()
except Exception as e:
    print("Error loading database:", e)
    table_list = []

# --- 4. Create Widgets ---
table_dropdown = widgets.Dropdown(options=table_list, description="Table:")
plot_type = widgets.Dropdown(options=["Scatter", "Bar"], description="Plot Type:")
x_dropdown = widgets.Dropdown(description="X-Axis:")
y_dropdown = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style='success')
output_area = widgets.Output()

# --- 5. Update columns when user picks a table or plot type ---
def update_columns(change):
    clear_output(wait=True)
    display(ui)

    table = table_dropdown.value
    ptype = plot_type.value

    if not table:
        return

    # Load sample data
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f'SELECT * FROM "{table}" LIMIT 100', conn)
    conn.close()

    # Choose columns based on plot type
    if ptype == "Scatter":
        numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
        x_dropdown.options = numeric_cols
        y_dropdown.options = numeric_cols
    else:
        all_cols = df.columns.tolist()
        x_dropdown.options = all_cols
        y_dropdown.options = all_cols

# --- 6. When user clicks "Generate Plot" ---
def generate_plot(b):
    with output_area:
        clear_output(wait=True)

        table = table_dropdown.value
        x_col = x_dropdown.value
        y_col = y_dropdown.value
        ptype = plot_type.value

        if not table or not x_col or not y_col:
            print("⚠️ Please select a table and valid columns.")
            return

        # Load full table data
        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f'SELECT * FROM "{table}"', conn)
        conn.close()

        # Filter by age if available
        if 'age' in df.columns:
            df['age'] = pd.to_numeric(df['age'], errors='coerce')
            df = df[(df['age'] >= 18) & (df['age'] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        if df.empty:
            print("No data available after filtering.")
            return

        # --- Create Plot ---
        fig, ax = plt.subplots(figsize=(6, 4))

        if ptype == "Scatter":
            # Convert to numbers and drop missing values
            df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
            df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
            df = df.dropna(subset=[x_col, y_col])

            ax.scatter(df[x_col], df[y_col], color='blue', alpha=0.7)

            # Add best-fit line if checked
            if best_fit.value and len(df) > 1:
                m, b = np.polyfit(df[x_col], df[y_col], 1)
                ax.plot(df[x_col], m * df[x_col] + b, color='red')
            ax.set_ylabel(y_col)

        else:  # --- Bar Plot ---
            # Handle grouped bars when both are categorical (e.g., gender vs. age)
            if df[x_col].dtype == 'object' or df[y_col].dtype == 'object':
                grouped = pd.crosstab(df[x_col], df[y_col])
                grouped.plot(kind='bar', ax=ax, width=0.8, alpha=0.8)
                plt.xticks(rotation=45, ha='right')
                ax.set_ylabel("Count")
                ax.legend(title=y_col)
            else:
                # Default single-variable count bar
                grouped = df.groupby(x_col)[y_col].count()
                grouped.plot(kind='bar', ax=ax, color='skyblue')
                plt.xticks(rotation=45, ha='right')
                ax.set_ylabel("Count")

        # --- Final Plot Formatting ---
        ax.set_xlabel(x_col)
        ax.set_title(f"{y_col} vs. {x_col} ({ptype})")
        ax.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()

        # Show image inside widget
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        display(widgets.Image(value=buf.read(), format='png'))
        plt.close(fig)

# --- 7. Link Widgets to Events ---
table_dropdown.observe(update_columns, names='value')
plot_type.observe(update_columns, names='value')
generate_button.on_click(generate_plot)

# --- 8. Display Interface ---
ui = widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_dropdown, y_dropdown]),
    widgets.HBox([generate_button, best_fit]),
    output_area
])

display(ui)


In [1]:
# --- 1. Import Libraries ---
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

# --- 2. Database Setup ---
DB_NAME = 'Dataset.db'  # Ensure your database file is in the same directory

# --- 3. Get All Tables from Database ---
try:
    conn = sqlite3.connect(DB_NAME)
    tables = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table' AND name != 'sqlite_sequence';",
        conn
    )
    table_list = tables['name'].tolist()
    conn.close()
except Exception as e:
    print("Error loading database:", e)
    table_list = []

# --- 4. Create Widgets ---
table_dropdown = widgets.Dropdown(options=table_list, description="Table:")
plot_type = widgets.Dropdown(options=["Scatter", "Bar"], description="Plot Type:")
x_dropdown = widgets.Dropdown(description="X-Axis:")
y_dropdown = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style='success')
output_area = widgets.Output()

# --- 5. Update columns when user picks a table or plot type ---
def update_columns(change):
    clear_output(wait=True)
    display(ui)

    table = table_dropdown.value
    ptype = plot_type.value

    if not table:
        return

    # Load sample data
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f'SELECT * FROM "{table}" LIMIT 100', conn)
    conn.close()

    # Choose columns based on plot type
    if ptype == "Scatter":
        numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
        x_dropdown.options = numeric_cols
        y_dropdown.options = numeric_cols
    else:
        all_cols = df.columns.tolist()
        x_dropdown.options = all_cols
        y_dropdown.options = all_cols

# --- 6. When user clicks "Generate Plot" ---
def generate_plot(b):
    with output_area:
        clear_output(wait=True)

        table = table_dropdown.value
        x_col = x_dropdown.value
        y_col = y_dropdown.value
        ptype = plot_type.value

        if not table or not x_col or not y_col:
            print("⚠️ Please select a table and valid columns.")
            return

        # Load full table data
        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f'SELECT * FROM "{table}"', conn)
        conn.close()

        # Filter by age if available
        if 'age' in df.columns:
            df['age'] = pd.to_numeric(df['age'], errors='coerce')
            df = df[(df['age'] >= 18) & (df['age'] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        if df.empty:
            print("No data available after filtering.")
            return

        # --- Create Plot ---
        fig, ax = plt.subplots(figsize=(6, 4))

        if ptype == "Scatter":
            # Convert to numbers and drop missing values
            df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
            df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
            df = df.dropna(subset=[x_col, y_col])

            ax.scatter(df[x_col], df[y_col], color='blue', alpha=0.7)

            # Add best-fit line if checked
            if best_fit.value and len(df) > 1:
                m, b = np.polyfit(df[x_col], df[y_col], 1)
                ax.plot(df[x_col], m * df[x_col] + b, color='red')
            ax.set_ylabel(y_col)

        else:  # --- Bar Plot ---
            # Handle grouped bars when both are categorical (e.g., gender vs. age)
            if df[x_col].dtype == 'object' or df[y_col].dtype == 'object':
                grouped = pd.crosstab(df[x_col], df[y_col])
                grouped.plot(kind='bar', ax=ax, width=0.8, alpha=0.8)
                plt.xticks(rotation=45, ha='right')
                ax.set_ylabel("Count")
                ax.legend(title=y_col)
            else:
                # Default single-variable count bar
                grouped = df.groupby(x_col)[y_col].count()
                grouped.plot(kind='bar', ax=ax, color='skyblue')
                plt.xticks(rotation=45, ha='right')
                ax.set_ylabel("Count")

        # --- Final Plot Formatting ---
        ax.set_xlabel(x_col)
        ax.set_title(f"{y_col} vs. {x_col} ({ptype})")
        ax.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()

        # Show image inside widget
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        display(widgets.Image(value=buf.read(), format='png'))
        plt.close(fig)

# --- 7. Link Widgets to Events ---
table_dropdown.observe(update_columns, names='value')
plot_type.observe(update_columns, names='value')
generate_button.on_click(generate_plot)

# --- 8. Display Interface ---
ui = widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_dropdown, y_dropdown]),
    widgets.HBox([generate_button, best_fit]),
    output_area
])

display(ui)


In [2]:
# --- 1. Import Libraries ---
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

# --- 2. Database Setup ---
DB_NAME = 'Dataset.db' # Ensure your database file is in the same directory

# --- 3. Get All Tables from Database ---
try:
    conn = sqlite3.connect(DB_NAME)
    tables = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table' AND name != 'sqlite_sequence';",
        conn
    )
    table_list = tables['name'].tolist()
    conn.close()
except Exception as e:
    print("Error loading database:", e)
    table_list = []

# --- 4. Create Widgets ---
table_dropdown = widgets.Dropdown(options=table_list, description="Table:")
plot_type = widgets.Dropdown(options=["Scatter", "Bar"], description="Plot Type:")
x_dropdown = widgets.Dropdown(description="X-Axis:")
y_dropdown = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style='success')
output_area = widgets.Output()

# --- 5. Update columns when user picks a table or plot type ---
def update_columns(change):
    # This keeps the UI from jumping around
    with output_area:
        pass 
    
    table = table_dropdown.value
    ptype = plot_type.value

    if not table:
        return

    # Load sample data to find column types
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f'SELECT * FROM "{table}" LIMIT 50', conn)
    conn.close()

    # Choose columns based on plot type
    if ptype == "Scatter":
        # Scatter plots need numbers
        numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
        x_dropdown.options = numeric_cols
        y_dropdown.options = numeric_cols
        best_fit.layout.display = 'flex' # Show best-fit option
    else:
        # Bar plots can use any column
        all_cols = df.columns.tolist()
        x_dropdown.options = all_cols
        y_dropdown.options = all_cols
        best_fit.layout.display = 'none' # Hide best-fit option

# --- 6. Plotting Helper Functions (Cleaner) ---

def plot_scatter_on_axis(ax, df, x_col, y_col, add_best_fit):
    """Creates a scatter plot on the given axes."""
    # Convert to numbers and drop missing values
    df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
    df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
    df = df.dropna(subset=[x_col, y_col])

    if df.empty:
        ax.text(0.5, 0.5, "No numeric data to plot.", ha='center', va='center')
        return

    ax.scatter(df[x_col], df[y_col], color='blue', alpha=0.7)
    ax.set_ylabel(y_col)

    # Add best-fit line if checked
    if add_best_fit and len(df) > 1:
        m, b = np.polyfit(df[x_col], df[y_col], 1)
        ax.plot(df[x_col], m * df[x_col] + b, color='red', linewidth=2)

def plot_bar_on_axis(ax, df, x_col, y_col):
    """Creates a bar plot, including the grouped 'gender vs age' style."""
    df = df.dropna(subset=[x_col, y_col])
    
    if df.empty:
        ax.text(0.5, 0.5, "No data to plot.", ha='center', va='center')
        return

    # If X or Y are categorical (text), create a grouped bar chart
    is_x_categorical = df[x_col].dtype == 'object'
    is_y_categorical = df[y_col].dtype == 'object'

    if is_x_categorical or is_y_categorical:
        # This is the "color for gender vs age" plot (e.g., crosstab)
        grouped = pd.crosstab(df[x_col], df[y_col])
        grouped.plot(kind='bar', ax=ax, width=0.8, alpha=0.8)
        ax.set_ylabel("Count")
        ax.legend(title=y_col, loc='upper right')
    else:
        # Otherwise, just make a simple bar chart (count of X)
        df.groupby(x_col)[y_col].count().plot(kind='bar', ax=ax, color='skyblue')
        ax.set_ylabel("Count")
    
    plt.xticks(rotation=45, ha='right')


# --- 7. When user clicks "Generate Plot" (Simple Version) ---
def generate_plot(b):
    with output_area:
        clear_output(wait=True)

        # --- 1. Get User Selections ---
        table = table_dropdown.value
        x_col = x_dropdown.value
        y_col = y_dropdown.value
        ptype = plot_type.value

        if not table or not x_col or not y_col:
            print("⚠️ Please select a table and valid X/Y columns.")
            return

        # --- 2. Load Full Data ---
        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f'SELECT * FROM "{table}"', conn)
        conn.close()

        # --- 3. Apply Age Filter (if 'age' column exists) ---
        if 'age' in df.columns:
            df['age'] = pd.to_numeric(df['age'], errors='coerce')
            df = df.dropna(subset=['age'])
            df = df[(df['age'] >= 18) & (df['age'] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        if df.empty:
            print("No data available after filtering.")
            return

        # --- 4. Create Plot ---
        fig, ax = plt.subplots(figsize=(7, 4)) # A bit wider for labels

        if ptype == "Scatter":
            plot_scatter_on_axis(ax, df, x_col, y_col, best_fit.value)
        else:
            plot_bar_on_axis(ax, df, x_col, y_col)

        # --- 5. Final Plot Formatting ---
        ax.set_xlabel(x_col)
        ax.set_title(f"{y_col} vs. {x_col} ({ptype})")
        ax.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()

        # --- 6. Show Image in Output ---
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        display(widgets.Image(value=buf.read(), format='png'))
        plt.close(fig)

# --- 8. Link Widgets to Events ---
table_dropdown.observe(update_columns, names='value')
plot_type.observe(update_columns, names='value')
generate_button.on_click(generate_plot)

# --- 9. Display Interface ---
# A simple layout with two rows for controls
controls = widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_dropdown, y_dropdown]),
    widgets.HBox([generate_button, best_fit])
])

# The final UI is the controls on top of the output area
ui = widgets.VBox([controls, output_area])

# Trigger the update once to populate the column dropdowns
update_columns(None)

display(ui)

In [3]:
# --- 1. Import Libraries ---
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

# --- 2. Database Setup ---
DB_NAME = 'Dataset.db'

# --- 3. Get All Tables from Database ---
try:
    conn = sqlite3.connect(DB_NAME)
    tables = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table' AND name != 'sqlite_sequence';",
        conn
    )
    table_list = tables['name'].tolist()
    conn.close()
except Exception as e:
    print("Error loading database:", e)
    table_list = []

# --- 4. Create Widgets ---
table_dropdown = widgets.Dropdown(options=table_list, description="Table:")
plot_type = widgets.Dropdown(options=["Scatter", "Bar"], description="Plot Type:")
x_dropdown = widgets.Dropdown(description="X-Axis:")
y_dropdown = widgets.Dropdown(description="Y-Axis:")
# --- NEW WIDGET ---
color_dropdown = widgets.Dropdown(description="Color by:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style='success')
output_area = widgets.Output()

# --- 5. Update columns when user picks a table or plot type ---
def update_columns(change):
    with output_area:
        pass # Keep UI stable
    
    table = table_dropdown.value
    ptype = plot_type.value

    if not table:
        return

    # Load sample data to find column types
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f'SELECT * FROM "{table}" LIMIT 50', conn)
    conn.close()

    # Get different types of columns
    all_cols = df.columns.tolist()
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    # Get text/categorical columns for the new 'color' dropdown
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    color_options = ['None'] + categorical_cols # Add 'None' option

    if ptype == "Scatter":
        # X and Y MUST be numeric
        x_dropdown.options = numeric_cols
        y_dropdown.options = numeric_cols
        # Color dropdown gets text columns
        color_dropdown.options = color_options
        color_dropdown.layout.display = 'flex' # Show color dropdown
        best_fit.layout.display = 'flex'
    else:
        # Bar plots can use any column
        x_dropdown.options = all_cols
        y_dropdown.options = all_cols
        color_dropdown.layout.display = 'none' # Hide color dropdown
        best_fit.layout.display = 'none'

    # Set default values to prevent errors
    if x_dropdown.options:
        x_dropdown.value = x_dropdown.options[0]
    if y_dropdown.options and len(y_dropdown.options) > 1:
        y_dropdown.value = y_dropdown.options[1]
    elif y_dropdown.options:
        y_dropdown.value = y_dropdown.options[0]
    if color_dropdown.options:
        color_dropdown.value = 'None'


# --- 6. Plotting Helper Functions (Cleaner) ---

def plot_scatter_on_axis(ax, df, x_col, y_col, color_col, add_best_fit):
    """Creates a scatter plot, now with optional coloring."""
    
    # Convert to numbers and drop missing values for X and Y
    df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
    df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
    df = df.dropna(subset=[x_col, y_col])

    if df.empty:
        ax.text(0.5, 0.5, "No numeric data to plot.", ha='center', va='center')
        return

    # --- Color Logic ---
    if color_col and color_col != 'None' and color_col in df.columns:
        df = df.dropna(subset=[color_col]) # Drop rows where color is missing
        categories = df[color_col].unique()
        
        # Plot each category with a different color
        for category in categories:
            df_group = df[df[color_col] == category]
            ax.scatter(df_group[x_col], df_group[y_col], alpha=0.7, label=str(category))
        if len(categories) > 0:
            ax.legend(title=color_col)
    else:
        # No color selected, plot all as one color
        ax.scatter(df[x_col], df[y_col], color='blue', alpha=0.7)

    ax.set_ylabel(y_col)

    # --- Best-Fit Line Logic ---
    # Plot one single best-fit line for all data, regardless of color
    if add_best_fit and len(df) > 1:
        m, b = np.polyfit(df[x_col], df[y_col], 1)
        ax.plot(df[x_col], m * df[x_col] + b, color='red', linewidth=2)


def plot_bar_on_axis(ax, df, x_col, y_col):
    """Creates a bar plot, including the grouped 'gender vs age' style."""
    df = df.dropna(subset=[x_col, y_col])
    
    if df.empty:
        ax.text(0.5, 0.5, "No data to plot.", ha='center', va='center')
        return

    is_x_categorical = df[x_col].dtype == 'object'
    is_y_categorical = df[y_col].dtype == 'object'

    if is_x_categorical or is_y_categorical:
        # This is the "color for gender vs age" plot
        grouped = pd.crosstab(df[x_col], df[y_col])
        grouped.plot(kind='bar', ax=ax, width=0.8, alpha=0.8)
        ax.set_ylabel("Count")
        ax.legend(title=y_col, loc='upper right')
    else:
        df.groupby(x_col)[y_col].count().plot(kind='bar', ax=ax, color='skyblue')
        ax.set_ylabel("Count")
    
    plt.xticks(rotation=45, ha='right')


# --- 7. When user clicks "Generate Plot" (Simple Version) ---
def generate_plot(b):
    with output_area:
        clear_output(wait=True)

        # --- 1. Get User Selections ---
        table = table_dropdown.value
        x_col = x_dropdown.value
        y_col = y_dropdown.value
        ptype = plot_type.value
        color_col = color_dropdown.value # <-- GET NEW VALUE
        
        if not table or not x_col or not y_col:
            print("⚠️ Please select a table and valid X/Y columns.")
            return

        # --- 2. Load Full Data ---
        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f'SELECT * FROM "{table}"', conn)
        conn.close()

        # --- 3. Apply Age Filter (if 'age' column exists) ---
        if 'age' in df.columns:
            df['age'] = pd.to_numeric(df['age'], errors='coerce')
            df = df.dropna(subset=['age'])
            df = df[(df['age'] >= 18) & (df['age'] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        if df.empty:
            print("No data available after filtering.")
            return

        # --- 4. Create Plot ---
        fig, ax = plt.subplots(figsize=(7, 4))

        if ptype == "Scatter":
            # --- PASS NEW VALUE TO PLOTTER ---
            plot_scatter_on_axis(ax, df, x_col, y_col, color_col, best_fit.value)
        else:
            plot_bar_on_axis(ax, df, x_col, y_col)

        # --- 5. Final Plot Formatting ---
        ax.set_xlabel(x_col)
        ax.set_title(f"{y_col} vs. {x_col} ({ptype})")
        ax.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()

        # --- 6. Show Image in Output ---
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        display(widgets.Image(value=buf.read(), format='png'))
        plt.close(fig)

# --- 8. Link Widgets to Events ---
table_dropdown.observe(update_columns, names='value')
plot_type.observe(update_columns, names='value')
generate_button.on_click(generate_plot)

# --- 9. Display Interface ---
# Add the new 'color_dropdown' to the layout
controls = widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_dropdown, y_dropdown, color_dropdown]), # <-- UPDATED ROW
    widgets.HBox([generate_button, best_fit])
])

# The final UI
ui = widgets.VBox([controls, output_area])

# Trigger the update once to populate the column dropdowns
update_columns(None)

display(ui)

In [4]:
# --- 1. Import Libraries ---
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker # <-- IMPORTED FOR TICK MARKS
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

# --- 2. Database Setup ---
DB_NAME = 'Dataset.db'

# --- 3. Get All Tables from Database ---
try:
    conn = sqlite3.connect(DB_NAME)
    tables = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table' AND name != 'sqlite_sequence';",
        conn
    )
    table_list = tables['name'].tolist()
    conn.close()
except Exception as e:
    print("Error loading database:", e)
    table_list = []

# --- 4. Create Widgets ---
table_dropdown = widgets.Dropdown(options=table_list, description="Table:")
plot_type = widgets.Dropdown(options=["Scatter", "Bar"], description="Plot Type:")
x_dropdown = widgets.Dropdown(description="X-Axis:")
y_dropdown = widgets.Dropdown(description="Y-Axis:")
color_dropdown = widgets.Dropdown(description="Color by:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style='success')
output_area = widgets.Output()

# --- 5. Update columns when user picks a table or plot type ---
def update_columns(change):
    with output_area:
        pass # Keep UI stable
    
    table = table_dropdown.value
    ptype = plot_type.value

    if not table:
        return

    # Load sample data to find column types
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f'SELECT * FROM "{table}" LIMIT 50', conn)
    conn.close()

    # Get different types of columns
    all_cols = df.columns.tolist()
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    color_options = ['None'] + categorical_cols # Add 'None' option

    if ptype == "Scatter":
        x_dropdown.options = numeric_cols
        y_dropdown.options = numeric_cols
        color_dropdown.options = color_options
        color_dropdown.layout.display = 'flex' 
        best_fit.layout.display = 'flex'
    else:
        x_dropdown.options = all_cols
        y_dropdown.options = all_cols
        color_dropdown.layout.display = 'none'
        best_fit.layout.display = 'none'

    # Set default values to prevent errors
    if x_dropdown.options:
        x_dropdown.value = x_dropdown.options[0]
    if y_dropdown.options and len(y_dropdown.options) > 1:
        y_dropdown.value = y_dropdown.options[1]
    elif y_dropdown.options:
        y_dropdown.value = y_dropdown.options[0]
    if color_dropdown.options:
        color_dropdown.value = 'None'


# --- 6. Plotting Helper Functions (Cleaner) ---

def plot_scatter_on_axis(ax, df, x_col, y_col, color_col, add_best_fit):
    """Creates a scatter plot, now with optional coloring and integer axis ticks."""
    
    df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
    df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
    df = df.dropna(subset=[x_col, y_col])

    if df.empty:
        ax.text(0.5, 0.5, "No numeric data to plot.", ha='center', va='center')
        return

    # --- Color Logic ---
    if color_col and color_col != 'None' and color_col in df.columns:
        df = df.dropna(subset=[color_col]) 
        categories = df[color_col].unique()
        
        for category in categories:
            df_group = df[df[color_col] == category]
            ax.scatter(df_group[x_col], df_group[y_col], alpha=0.7, label=str(category))
        if len(categories) > 0:
            ax.legend(title=color_col)
    else:
        ax.scatter(df[x_col], df[y_col], color='blue', alpha=0.7)

    ax.set_ylabel(y_col)

    # --- Best-Fit Line Logic ---
    if add_best_fit and len(df) > 1:
        m, b = np.polyfit(df[x_col], df[y_col], 1)
        ax.plot(df[x_col], m * df[x_col] + b, color='red', linewidth=2)
        
    # --- NEW: Set Integer Ticks for 'age' ---
    if x_col == 'age':
        ax.xaxis.set_major_locator(mticker.MultipleLocator(1))
    if y_col == 'age':
        ax.yaxis.set_major_locator(mticker.MultipleLocator(1))


def plot_bar_on_axis(ax, df, x_col, y_col):
    """Creates a bar plot. 'age' data should be rounded before calling this."""
    df = df.dropna(subset=[x_col, y_col])
    
    if df.empty:
        ax.text(0.5, 0.5, "No data to plot.", ha='center', va='center')
        return

    is_x_categorical = df[x_col].dtype == 'object'
    is_y_categorical = df[y_col].dtype == 'object'
    # Check for integer type as well, since we converted 'age'
    is_x_numeric_like = pd.api.types.is_numeric_dtype(df[x_col])
    is_y_numeric_like = pd.api.types.is_numeric_dtype(df[y_col])

    if (is_x_categorical or is_y_categorical) or (x_col == 'age' and is_y_categorical):
        # This is the "color for gender vs age" plot
        grouped = pd.crosstab(df[x_col], df[y_col])
        grouped.plot(kind='bar', ax=ax, width=0.8, alpha=0.8)
        ax.set_ylabel("Count")
        ax.legend(title=y_col, loc='upper right')
    else:
        df.groupby(x_col)[y_col].count().plot(kind='bar', ax=ax, color='skyblue')
        ax.set_ylabel("Count")
    
    plt.xticks(rotation=45, ha='right')


# --- 7. When user clicks "Generate Plot" (Simple Version) ---
def generate_plot(b):
    with output_area:
        clear_output(wait=True)

        # --- 1. Get User Selections ---
        table = table_dropdown.value
        x_col = x_dropdown.value
        y_col = y_dropdown.value
        ptype = plot_type.value
        color_col = color_dropdown.value
        
        if not table or not x_col or not y_col:
            print("⚠️ Please select a table and valid X/Y columns.")
            return

        # --- 2. Load Full Data ---
        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f'SELECT * FROM "{table}"', conn)
        conn.close()

        # --- 3. Apply Age Filter (if 'age' column exists) ---
        if 'age' in df.columns:
            df['age'] = pd.to_numeric(df['age'], errors='coerce')
            df = df.dropna(subset=['age'])
            df = df[(df['age'] >= 18) & (df['age'] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        if df.empty:
            print("No data available after filtering.")
            return

        # --- 4. Create Plot ---
        fig, ax = plt.subplots(figsize=(7, 4))

        if ptype == "Scatter":
            plot_scatter_on_axis(ax, df, x_col, y_col, color_col, best_fit.value)
        else:
            # --- NEW: Round ages for Bar Charts ---
            if 'age' in df.columns:
                df['age'] = df['age'].round().astype(int)
            plot_bar_on_axis(ax, df, x_col, y_col)

        # --- 5. Final Plot Formatting ---
        ax.set_xlabel(x_col)
        ax.set_title(f"{y_col} vs. {x_col} ({ptype})")
        ax.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()

        # --- 6. Show Image in Output ---
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        display(widgets.Image(value=buf.read(), format='png'))
        plt.close(fig)

# --- 8. Link Widgets to Events ---
table_dropdown.observe(update_columns, names='value')
plot_type.observe(update_columns, names='value')
generate_button.on_click(generate_plot)

# --- 9. Display Interface ---
controls = widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_dropdown, y_dropdown, color_dropdown]),
    widgets.HBox([generate_button, best_fit])
])

ui = widgets.VBox([controls, output_area])

# Trigger the update once to populate the column dropdowns
update_columns(None)

display(ui)

In [5]:
# --- 1. Import Libraries ---
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker # For integer axis ticks
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

# --- 2. Database Setup ---
DB_NAME = 'Dataset.db'

# --- 3. Get All Tables from Database ---
try:
    conn = sqlite3.connect(DB_NAME)
    tables = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table' AND name != 'sqlite_sequence';",
        conn
    )
    table_list = tables['name'].tolist()
    conn.close()
except Exception as e:
    print("Error loading database:", e)
    table_list = []

# --- 4. Create Widgets ---
table_dropdown = widgets.Dropdown(options=table_list, description="Table:")
plot_type = widgets.Dropdown(options=["Scatter", "Bar"], description="Plot Type:")
x_dropdown = widgets.Dropdown(description="X-Axis:")
y_dropdown = widgets.Dropdown(description="Y-Axis:")
color_dropdown = widgets.Dropdown(description="Color by:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style='success')
output_area = widgets.Output()

# --- 5. Update columns when user picks a table or plot type ---
def update_columns(change):
    with output_area:
        pass # Keep UI stable
    
    table = table_dropdown.value
    ptype = plot_type.value

    if not table:
        return

    # Load sample data to find column types
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f'SELECT * FROM "{table}" LIMIT 50', conn)
    conn.close()

    # Get different types of columns
    all_cols = df.columns.tolist()
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    color_options = ['None'] + categorical_cols # Add 'None' option

    if ptype == "Scatter":
        x_dropdown.options = numeric_cols
        y_dropdown.options = numeric_cols
        color_dropdown.options = color_options
        color_dropdown.layout.display = 'flex' 
        best_fit.layout.display = 'flex'
    else:
        x_dropdown.options = all_cols
        y_dropdown.options = all_cols
        color_dropdown.layout.display = 'none'
        best_fit.layout.display = 'none'

    # Set default values to prevent errors
    if x_dropdown.options:
        x_dropdown.value = x_dropdown.options[0]
    if y_dropdown.options and len(y_dropdown.options) > 1:
        y_dropdown.value = y_dropdown.options[1]
    elif y_dropdown.options:
        y_dropdown.value = y_dropdown.options[0]
    if color_dropdown.options:
        color_dropdown.value = 'None'


# --- 6. Plotting Helper Functions (Cleaner) ---

def plot_scatter_on_axis(ax, df, x_col, y_col, color_col, add_best_fit):
    """Creates a scatter plot, now with optional coloring and integer axis ticks."""
    
    df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
    df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
    df = df.dropna(subset=[x_col, y_col])

    if df.empty:
        ax.text(0.5, 0.5, "No numeric data to plot.", ha='center', va='center')
        return

    # --- Color Logic ---
    if color_col and color_col != 'None' and color_col in df.columns:
        df = df.dropna(subset=[color_col]) 
        categories = df[color_col].unique()
        
        for category in categories:
            df_group = df[df[color_col] == category]
            ax.scatter(df_group[x_col], df_group[y_col], alpha=0.7, label=str(category))
        if len(categories) > 0:
            ax.legend(title=color_col)
    else:
        ax.scatter(df[x_col], df[y_col], color='blue', alpha=0.7)

    ax.set_ylabel(y_col)

    # --- Best-Fit Line Logic ---
    if add_best_fit and len(df) > 1:
        m, b = np.polyfit(df[x_col], df[y_col], 1)
        ax.plot(df[x_col], m * df[x_col] + b, color='red', linewidth=2)
        
    # --- Set Integer Ticks for 'age' ---
    if x_col == 'age':
        ax.xaxis.set_major_locator(mticker.MultipleLocator(1))
    if y_col == 'age':
        ax.yaxis.set_major_locator(mticker.MultipleLocator(1))


def plot_bar_on_axis(ax, df, x_col, y_col):
    """Creates a bar plot. 'age' data should be rounded before calling this."""
    df = df.dropna(subset=[x_col, y_col])
    
    if df.empty:
        ax.text(0.5, 0.5, "No data to plot.", ha='center', va='center')
        return

    is_x_categorical = df[x_col].dtype == 'object'
    is_y_categorical = df[y_col].dtype == 'object'
    is_x_numeric_like = pd.api.types.is_numeric_dtype(df[x_col])
    is_y_numeric_like = pd.api.types.is_numeric_dtype(df[y_col])

    if (is_x_categorical or is_y_categorical) or (x_col == 'age' and is_y_categorical):
        # This is the "color for gender vs age" plot
        grouped = pd.crosstab(df[x_col], df[y_col])
        grouped.plot(kind='bar', ax=ax, width=0.8, alpha=0.8)
        ax.set_ylabel("Count")
        ax.legend(title=y_col, loc='upper right')
    else:
        df.groupby(x_col)[y_col].count().plot(kind='bar', ax=ax, color='skyblue')
        ax.set_ylabel("Count")
    
    # Tick rotation is now handled in the main generate_plot function


# --- 7. When user clicks "Generate Plot" (Simple Version) ---
def generate_plot(b):
    with output_area:
        clear_output(wait=True)

        # --- 1. Get User Selections ---
        table = table_dropdown.value
        x_col = x_dropdown.value
        y_col = y_dropdown.value
        ptype = plot_type.value
        color_col = color_dropdown.value
        
        if not table or not x_col or not y_col:
            print("⚠️ Please select a table and valid X/Y columns.")
            return

        # --- 2. Load Full Data ---
        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f'SELECT * FROM "{table}"', conn)
        conn.close()

        # --- 3. Apply Age Filter (if 'age' column exists) ---
        if 'age' in df.columns:
            df['age'] = pd.to_numeric(df['age'], errors='coerce')
            df = df.dropna(subset=['age'])
            df = df[(df['age'] >= 18) & (df['age'] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        if df.empty:
            print("No data available after filtering.")
            return

        # --- 4. Create Plot ---
        fig, ax = plt.subplots(figsize=(7, 4))

        if ptype == "Scatter":
            plot_scatter_on_axis(ax, df, x_col, y_col, color_col, best_fit.value)
        else:
            if 'age' in df.columns:
                df['age'] = df['age'].round().astype(int)
            plot_bar_on_axis(ax, df, x_col, y_col)

        # --- 5. Final Plot Formatting ---
        
        # --- Move X-axis to the top ---
        ax.xaxis.tick_top()
        ax.xaxis.set_label_position('top')
        
        # --- Ensure Y-axis is on the left (it is by default) ---
        ax.yaxis.tick_left()
        ax.yaxis.set_label_position('left')
        
        # Set labels
        ax.set_xlabel(x_col)
        ax.set_title(f"{y_col} vs. {x_col} ({ptype})")
        ax.grid(True, linestyle='--', alpha=0.5)

        # --- Adjust tick rotation for top axis ---
        if ptype == "Bar":
            # ha='left' aligns the start of the label with the tick
            plt.xticks(rotation=45, ha='left') 
        
        plt.tight_layout()

        # --- 6. Show Image in Output ---
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        display(widgets.Image(value=buf.read(), format='png'))
        plt.close(fig)

# --- 8. Link Widgets to Events ---
table_dropdown.observe(update_columns, names='value')
plot_type.observe(update_columns, names='value')
generate_button.on_click(generate_plot)

# --- 9. Display Interface ---
controls = widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    # --- THIS IS THE ONLY LINE THAT CHANGED ---
    widgets.HBox([y_dropdown, x_dropdown, color_dropdown]),
    widgets.HBox([generate_button, best_fit])
])

ui = widgets.VBox([controls, output_area])

# Trigger the update once to populate the column dropdowns
update_columns(None)

display(ui)

In [6]:
# --- 1. Import Libraries ---
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker # For integer axis ticks
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

# --- 2. Database Setup ---
DB_NAME = 'Dataset.db'

# --- 3. Get All Tables from Database ---
try:
    conn = sqlite3.connect(DB_NAME)
    tables = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table' AND name != 'sqlite_sequence';",
        conn
    )
    table_list = tables['name'].tolist()
    conn.close()
except Exception as e:
    print("Error loading database:", e)
    table_list = []

# --- 4. Create Widgets ---
table_dropdown = widgets.Dropdown(options=table_list, description="Table:")
plot_type = widgets.Dropdown(options=["Scatter", "Bar"], description="Plot Type:")
x_dropdown = widgets.Dropdown(description="X-Axis:")
y_dropdown = widgets.Dropdown(description="Y-Axis:")
color_dropdown = widgets.Dropdown(description="Color by:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style='success')
output_area = widgets.Output()

# --- 5. Update columns when user picks a table or plot type ---
def update_columns(change):
    with output_area:
        pass # Keep UI stable
    
    table = table_dropdown.value
    ptype = plot_type.value

    if not table:
        return

    # Load sample data to find column types
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f'SELECT * FROM "{table}" LIMIT 50', conn)
    conn.close()

    # Get different types of columns
    all_cols = df.columns.tolist()
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    color_options = ['None'] + categorical_cols # Add 'None' option

    if ptype == "Scatter":
        x_dropdown.options = numeric_cols
        y_dropdown.options = numeric_cols
        color_dropdown.options = color_options
        color_dropdown.layout.display = 'flex' 
        best_fit.layout.display = 'flex'
    else:
        x_dropdown.options = all_cols
        y_dropdown.options = all_cols
        color_dropdown.layout.display = 'none'
        best_fit.layout.display = 'none'

    # Set default values to prevent errors
    if x_dropdown.options:
        x_dropdown.value = x_dropdown.options[0]
    if y_dropdown.options and len(y_dropdown.options) > 1:
        y_dropdown.value = y_dropdown.options[1]
    elif y_dropdown.options:
        y_dropdown.value = y_dropdown.options[0]
    if color_dropdown.options:
        color_dropdown.value = 'None'


# --- 6. Plotting Helper Functions (Cleaner) ---

def plot_scatter_on_axis(ax, df, x_col, y_col, color_col, add_best_fit):
    """Creates a scatter plot, now with optional coloring and integer axis ticks."""
    
    df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
    df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
    df = df.dropna(subset=[x_col, y_col])

    if df.empty:
        ax.text(0.5, 0.5, "No numeric data to plot.", ha='center', va='center')
        return

    # --- Color Logic ---
    if color_col and color_col != 'None' and color_col in df.columns:
        df = df.dropna(subset=[color_col]) 
        categories = df[color_col].unique()
        
        for category in categories:
            df_group = df[df[color_col] == category]
            ax.scatter(df_group[x_col], df_group[y_col], alpha=0.7, label=str(category))
        if len(categories) > 0:
            ax.legend(title=color_col)
    else:
        ax.scatter(df[x_col], df[y_col], color='blue', alpha=0.7)

    ax.set_ylabel(y_col)

    # --- Best-Fit Line Logic ---
    if add_best_fit and len(df) > 1:
        m, b = np.polyfit(df[x_col], df[y_col], 1)
        ax.plot(df[x_col], m * df[x_col] + b, color='red', linewidth=2)
        
    # --- Set Integer Ticks for 'age' ---
    if x_col == 'age':
        ax.xaxis.set_major_locator(mticker.MultipleLocator(1))
    if y_col == 'age':
        ax.yaxis.set_major_locator(mticker.MultipleLocator(1))


def plot_bar_on_axis(ax, df, x_col, y_col):
    """Creates a bar plot. 'age' data should be rounded before calling this."""
    df = df.dropna(subset=[x_col, y_col])
    
    if df.empty:
        ax.text(0.5, 0.5, "No data to plot.", ha='center', va='center')
        return

    is_x_categorical = df[x_col].dtype == 'object'
    is_y_categorical = df[y_col].dtype == 'object'
    is_x_numeric_like = pd.api.types.is_numeric_dtype(df[x_col])
    is_y_numeric_like = pd.api.types.is_numeric_dtype(df[y_col])

    if (is_x_categorical or is_y_categorical) or (x_col == 'age' and is_y_categorical):
        # This is the "color for gender vs age" plot
        grouped = pd.crosstab(df[x_col], df[y_col])
        grouped.plot(kind='bar', ax=ax, width=0.8, alpha=0.8)
        ax.set_ylabel("Count")
        ax.legend(title=y_col, loc='upper right')
    else:
        df.groupby(x_col)[y_col].count().plot(kind='bar', ax=ax, color='skyblue')
        ax.set_ylabel("Count")
    
    # Tick rotation is now handled in the main generate_plot function


# --- 7. When user clicks "Generate Plot" (Simple Version) ---
def generate_plot(b):
    with output_area:
        clear_output(wait=True)

        # --- 1. Get User Selections ---
        table = table_dropdown.value
        x_col = x_dropdown.value
        y_col = y_dropdown.value
        ptype = plot_type.value
        color_col = color_dropdown.value
        
        if not table or not x_col or not y_col:
            print("⚠️ Please select a table and valid X/Y columns.")
            return

        # --- 2. Load Full Data ---
        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f'SELECT * FROM "{table}"', conn)
        conn.close()

        # --- 3. Apply Age Filter (if 'age' column exists) ---
        if 'age' in df.columns:
            df['age'] = pd.to_numeric(df['age'], errors='coerce')
            df = df.dropna(subset=['age'])
            df = df[(df['age'] >= 18) & (df['age'] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        if df.empty:
            print("No data available after filtering.")
            return

        # --- 4. Create Plot ---
        fig, ax = plt.subplots(figsize=(7, 4))

        if ptype == "Scatter":
            plot_scatter_on_axis(ax, df, x_col, y_col, color_col, best_fit.value)
        else:
            if 'age' in df.columns:
                df['age'] = df['age'].round().astype(int)
            plot_bar_on_axis(ax, df, x_col, y_col)

        # --- 5. Final Plot Formatting ---
        
        # --- RESET: X-axis on bottom, Y-axis on left (default) ---
        ax.xaxis.tick_bottom()
        ax.xaxis.set_label_position('bottom')
        ax.yaxis.tick_left()
        ax.yaxis.set_label_position('left')
        
        # Set labels
        ax.set_xlabel(x_col)
        ax.set_title(f"{y_col} vs. {x_col} ({ptype})")
        ax.grid(True, linestyle='--', alpha=0.5)

        # --- RESET: Adjust tick rotation for bottom axis ---
        if ptype == "Bar":
            # ha='right' aligns the end of the label with the tick
            plt.xticks(rotation=45, ha='right') 
        
        plt.tight_layout()

        # --- 6. Show Image in Output ---
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        display(widgets.Image(value=buf.read(), format='png'))
        plt.close(fig)

# --- 8. Link Widgets to Events ---
table_dropdown.observe(update_columns, names='value')
plot_type.observe(update_columns, names='value')
generate_button.on_click(generate_plot)

# --- 9. Display Interface ---
controls = widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    # Swapped dropdowns as requested
    widgets.HBox([y_dropdown, x_dropdown, color_dropdown]),
    widgets.HBox([generate_button, best_fit])
])

ui = widgets.VBox([controls, output_area])

# Trigger the update once to populate the column dropdowns
update_columns(None)

display(ui)

In [7]:
# --- 1. Import Libraries ---
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker # For integer axis ticks
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

# --- 2. Database Setup ---
DB_NAME = 'Dataset.db'

# --- 3. Get All Tables from Database ---
try:
    conn = sqlite3.connect(DB_NAME)
    tables = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table' AND name != 'sqlite_sequence';",
        conn
    )
    table_list = tables['name'].tolist()
    conn.close()
except Exception as e:
    print("Error loading database:", e)
    table_list = []

# --- 4. Create Widgets ---
table_dropdown = widgets.Dropdown(options=table_list, description="Table:")
plot_type = widgets.Dropdown(options=["Scatter", "Bar"], description="Plot Type:")
x_dropdown = widgets.Dropdown(description="X-Axis:")
y_dropdown = widgets.Dropdown(description="Y-Axis:")
color_dropdown = widgets.Dropdown(description="Color by:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style='success')
output_area = widgets.Output()

# --- 5. Update columns when user picks a table or plot type ---
def update_columns(change):
    with output_area:
        pass # Keep UI stable
    
    table = table_dropdown.value
    ptype = plot_type.value

    if not table:
        return

    # Load sample data to find column types
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f'SELECT * FROM "{table}" LIMIT 50', conn)
    conn.close()

    # Get different types of columns
    all_cols = df.columns.tolist()
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    color_options = ['None'] + categorical_cols # Add 'None' option

    if ptype == "Scatter":
        x_dropdown.options = numeric_cols
        y_dropdown.options = numeric_cols
        color_dropdown.options = color_options
        color_dropdown.layout.display = 'flex' 
        best_fit.layout.display = 'flex'
    else:
        x_dropdown.options = all_cols
        y_dropdown.options = all_cols
        color_dropdown.layout.display = 'none'
        best_fit.layout.display = 'none'

    # Set default values to prevent errors
    if x_dropdown.options:
        x_dropdown.value = x_dropdown.options[0]
    if y_dropdown.options and len(y_dropdown.options) > 1:
        y_dropdown.value = y_dropdown.options[1]
    elif y_dropdown.options:
        y_dropdown.value = y_dropdown.options[0]
    if color_dropdown.options:
        color_dropdown.value = 'None'


# --- 6. Plotting Helper Functions (Cleaner) ---

def plot_scatter_on_axis(ax, df, x_col, y_col, color_col, add_best_fit):
    """Creates a scatter plot, now with optional coloring and integer axis ticks."""
    
    df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
    df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
    df = df.dropna(subset=[x_col, y_col])

    if df.empty:
        ax.text(0.5, 0.5, "No numeric data to plot.", ha='center', va='center')
        return

    # --- Color Logic ---
    if color_col and color_col != 'None' and color_col in df.columns:
        df = df.dropna(subset=[color_col]) 
        categories = df[color_col].unique()
        
        for category in categories:
            df_group = df[df[color_col] == category]
            ax.scatter(df_group[x_col], df_group[y_col], alpha=0.7, label=str(category))
        if len(categories) > 0:
            ax.legend(title=color_col)
    else:
        ax.scatter(df[x_col], df[y_col], color='blue', alpha=0.7)

    ax.set_ylabel(y_col)

    # --- Best-Fit Line Logic ---
    if add_best_fit and len(df) > 1:
        m, b = np.polyfit(df[x_col], df[y_col], 1)
        ax.plot(df[x_col], m * df[x_col] + b, color='red', linewidth=2)
        
    # --- Set Integer Ticks for 'age' ---
    if x_col == 'age':
        ax.xaxis.set_major_locator(mticker.MultipleLocator(1))
    if y_col == 'age':
        ax.yaxis.set_major_locator(mticker.MultipleLocator(1))


def plot_bar_on_axis(ax, df, x_col, y_col):
    """Creates a bar plot. 'age' data should be rounded before calling this."""
    df = df.dropna(subset=[x_col, y_col])
    
    if df.empty:
        ax.text(0.5, 0.5, "No data to plot.", ha='center', va='center')
        return

    is_x_categorical = df[x_col].dtype == 'object'
    is_y_categorical = df[y_col].dtype == 'object'
    is_x_numeric_like = pd.api.types.is_numeric_dtype(df[x_col])
    is_y_numeric_like = pd.api.types.is_numeric_dtype(df[y_col])

    if (is_x_categorical or is_y_categorical) or (x_col == 'age' and is_y_categorical):
        # This is the "color for gender vs age" plot
        grouped = pd.crosstab(df[x_col], df[y_col])
        grouped.plot(kind='bar', ax=ax, width=0.8, alpha=0.8)
        ax.set_ylabel("Count")
        ax.legend(title=y_col, loc='upper right')
    else:
        df.groupby(x_col)[y_col].count().plot(kind='bar', ax=ax, color='skyblue')
        ax.set_ylabel("Count")
    
    # Tick rotation is now handled in the main generate_plot function


# --- 7. When user clicks "Generate Plot" (Simple Version) ---
def generate_plot(b):
    with output_area:
        clear_output(wait=True)
        
        # --- THIS IS THE FIX ---
        # Close all old, stray plots to prevent the black bar
        plt.close('all') 
        # ---------------------

        # --- 1. Get User Selections ---
        table = table_dropdown.value
        x_col = x_dropdown.value
        y_col = y_dropdown.value
        ptype = plot_type.value
        color_col = color_dropdown.value
        
        if not table or not x_col or not y_col:
            print("⚠️ Please select a table and valid X/Y columns.")
            return

        # --- 2. Load Full Data ---
        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f'SELECT * FROM "{table}"', conn)
        conn.close()

        # --- 3. Apply Age Filter (if 'age' column exists) ---
        if 'age' in df.columns:
            df['age'] = pd.to_numeric(df['age'], errors='coerce')
            df = df.dropna(subset=['age'])
            df = df[(df['age'] >= 18) & (df['age'] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        if df.empty:
            print("No data available after filtering.")
            return

        # --- 4. Create Plot ---
        fig, ax = plt.subplots(figsize=(7, 4))

        if ptype == "Scatter":
            plot_scatter_on_axis(ax, df, x_col, y_col, color_col, best_fit.value)
        else:
            if 'age' in df.columns:
                df['age'] = df['age'].round().astype(int)
            plot_bar_on_axis(ax, df, x_col, y_col)

        # --- 5. Final Plot Formatting ---
        ax.xaxis.tick_bottom()
        ax.xaxis.set_label_position('bottom')
        ax.yaxis.tick_left()
        ax.yaxis.set_label_position('left')
        
        ax.set_xlabel(x_col)
        ax.set_title(f"{y_col} vs. {x_col} ({ptype})")
        ax.grid(True, linestyle='--', alpha=0.5)

        if ptype == "Bar":
            plt.xticks(rotation=45, ha='right') 
        
        plt.tight_layout()

        # --- 6. Show Image in Output ---
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        display(widgets.Image(value=buf.read(), format='png'))
        
        # We still close the *current* fig, just to be tidy
        plt.close(fig)

# --- 8. Link Widgets to Events ---
table_dropdown.observe(update_columns, names='value')
plot_type.observe(update_columns, names='value')
generate_button.on_click(generate_plot)

# --- 9. Display Interface ---
controls = widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    # Swapped dropdowns as requested
    widgets.HBox([y_dropdown, x_dropdown, color_dropdown]),
    widgets.HBox([generate_button, best_fit])
])

ui = widgets.VBox([controls, output_area])

# Trigger the update once to populate the column dropdowns
update_columns(None)

display(ui)

In [8]:
# --- 1. Import Libraries ---
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker # For integer axis ticks
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

# --- 2. Database Setup ---
DB_NAME = 'Dataset.db'

# --- 3. Get All Tables from Database ---
try:
    conn = sqlite3.connect(DB_NAME)
    tables = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table' AND name != 'sqlite_sequence';",
        conn
    )
    table_list = tables['name'].tolist()
    conn.close()
except Exception as e:
    print("Error loading database:", e)
    table_list = []

# --- 4. Create Widgets ---
table_dropdown = widgets.Dropdown(options=table_list, description="Table:")
plot_type = widgets.Dropdown(options=["Scatter", "Bar"], description="Plot Type:")
x_dropdown = widgets.Dropdown(description="X-Axis:")
y_dropdown = widgets.Dropdown(description="Y-Axis:")
color_dropdown = widgets.Dropdown(description="Color by:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style='success')
output_area = widgets.Output()

# --- 5. Update columns when user picks a table or plot type ---
def update_columns(change):
    with output_area:
        pass # Keep UI stable
    
    table = table_dropdown.value
    ptype = plot_type.value

    if not table:
        return

    # Load sample data to find column types
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f'SELECT * FROM "{table}" LIMIT 50', conn)
    conn.close()

    # Get different types of columns
    all_cols = df.columns.tolist()
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    color_options = ['None'] + categorical_cols # Add 'None' option

    if ptype == "Scatter":
        x_dropdown.options = numeric_cols
        y_dropdown.options = numeric_cols
        color_dropdown.options = color_options
        color_dropdown.layout.display = 'flex' 
        best_fit.layout.display = 'flex'
    else:
        x_dropdown.options = all_cols
        y_dropdown.options = all_cols
        color_dropdown.layout.display = 'none'
        best_fit.layout.display = 'none'

    # Set default values to prevent errors
    if x_dropdown.options:
        x_dropdown.value = x_dropdown.options[0]
    if y_dropdown.options and len(y_dropdown.options) > 1:
        y_dropdown.value = y_dropdown.options[1]
    elif y_dropdown.options:
        y_dropdown.value = y_dropdown.options[0]
    if color_dropdown.options:
        color_dropdown.value = 'None'


# --- 6. Plotting Helper Functions (Cleaner) ---

def plot_scatter_on_axis(ax, df, x_col, y_col, color_col, add_best_fit):
    """Creates a scatter plot, now with optional coloring and integer axis ticks."""
    
    df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
    df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
    df = df.dropna(subset=[x_col, y_col])

    if df.empty:
        ax.text(0.5, 0.5, "No numeric data to plot.", ha='center', va='center')
        return

    # --- Color Logic ---
    if color_col and color_col != 'None' and color_col in df.columns:
        df = df.dropna(subset=[color_col]) 
        categories = df[color_col].unique()
        
        for category in categories:
            df_group = df[df[color_col] == category]
            ax.scatter(df_group[x_col], df_group[y_col], alpha=0.7, label=str(category))
        if len(categories) > 0:
            ax.legend(title=color_col)
    else:
        ax.scatter(df[x_col], df[y_col], color='blue', alpha=0.7)

    ax.set_ylabel(y_col)

    # --- Best-Fit Line Logic ---
    if add_best_fit and len(df) > 1:
        m, b = np.polyfit(df[x_col], df[y_col], 1)
        ax.plot(df[x_col], m * df[x_col] + b, color='red', linewidth=2)
        
    # --- Set Integer Ticks for 'age' ---
    if x_col == 'age':
        ax.xaxis.set_major_locator(mticker.MultipleLocator(1))
    if y_col == 'age':
        ax.yaxis.set_major_locator(mticker.MultipleLocator(1))


def plot_bar_on_axis(ax, df, x_col, y_col):
    """Creates a bar plot. 'age' data should be rounded before calling this."""
    df = df.dropna(subset=[x_col, y_col])
    
    if df.empty:
        ax.text(0.5, 0.5, "No data to plot.", ha='center', va='center')
        return

    is_x_categorical = df[x_col].dtype == 'object'
    is_y_categorical = df[y_col].dtype == 'object'

    if (is_x_categorical or is_y_categorical) or (x_col == 'age' and is_y_categorical):
        # This is the "color for gender vs age" plot
        grouped = pd.crosstab(df[x_col], df[y_col])
        grouped.plot(kind='bar', ax=ax, width=0.8, alpha=0.8) # Keep this for grouped
        ax.set_ylabel("Count")
        ax.legend(title=y_col, loc='upper right')
        # Apply rotation settings directly to the axis
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    else:
        # --- THIS IS THE FIX ---
        # Replaced df.groupby(...).plot() with ax.bar()
        grouped_data = df.groupby(x_col)[y_col].count()
        # Ensure index is string for categorical plotting
        ax.bar(grouped_data.index.astype(str), grouped_data.values, color='skyblue')
        ax.set_ylabel("Count")
        # Apply rotation settings directly to the axis
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
        # --- END FIX ---


# --- 7. When user clicks "Generate Plot" (Simple Version) ---
def generate_plot(b):
    with output_area:
        clear_output(wait=True)
        
        # --- Close all old, stray plots to prevent the black bar ---
        plt.close('all') 

        # --- 1. Get User Selections ---
        table = table_dropdown.value
        x_col = x_dropdown.value
        y_col = y_dropdown.value
        ptype = plot_type.value
        color_col = color_dropdown.value
        
        if not table or not x_col or not y_col:
            print("⚠️ Please select a table and valid X/Y columns.")
            return

        # --- 2. Load Full Data ---
        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f'SELECT * FROM "{table}"', conn)
        conn.close()

        # --- 3. Apply Age Filter (if 'age' column exists) ---
        if 'age' in df.columns:
            df['age'] = pd.to_numeric(df['age'], errors='coerce')
            df = df.dropna(subset=['age'])
            df = df[(df['age'] >= 18) & (df['age'] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        if df.empty:
            print("No data available after filtering.")
            return

        # --- 4. Create Plot ---
        fig, ax = plt.subplots(figsize=(7, 4))

        if ptype == "Scatter":
            plot_scatter_on_axis(ax, df, x_col, y_col, color_col, best_fit.value)
        else:
            if 'age' in df.columns:
                df['age'] = df['age'].round().astype(int)
            plot_bar_on_axis(ax, df, x_col, y_col)

        # --- 5. Final Plot Formatting ---
        ax.xaxis.tick_bottom()
        ax.xaxis.set_label_position('bottom')
        ax.yaxis.tick_left()
        ax.yaxis.set_label_position('left')
        
        ax.set_xlabel(x_col)
        ax.set_title(f"{y_col} vs. {x_col} ({ptype})")
        ax.grid(True, linestyle='--', alpha=0.5)

        # --- The rotation logic was moved to plot_bar_on_axis ---
        
        plt.tight_layout()

        # --- 6. Show Image in Output ---
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        display(widgets.Image(value=buf.read(), format='png'))
        
        # We still close the *current* fig, just to be tidy
        plt.close(fig)

# --- 8. Link Widgets to Events ---
table_dropdown.observe(update_columns, names='value')
plot_type.observe(update_columns, names='value')
generate_button.on_click(generate_plot)

# --- 9. Display Interface ---
controls = widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    # Swapped dropdowns as requested
    widgets.HBox([y_dropdown, x_dropdown, color_dropdown]),
    widgets.HBox([generate_button, best_fit])
])

ui = widgets.VBox([controls, output_area])

# Trigger the update once to populate the column dropdowns
update_columns(None)

display(ui)

In [9]:
# --- 1. Import Libraries ---
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker # For integer axis ticks
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

# --- 2. Database Setup ---
DB_NAME = 'Dataset.db'

# --- 3. Get All Tables from Database ---
try:
    conn = sqlite3.connect(DB_NAME)
    tables = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table' AND name != 'sqlite_sequence';",
        conn
    )
    table_list = tables['name'].tolist()
    conn.close()
except Exception as e:
    print("Error loading database:", e)
    table_list = []

# --- 4. Create Widgets ---
table_dropdown = widgets.Dropdown(options=table_list, description="Table:")
plot_type = widgets.Dropdown(options=["Scatter", "Bar"], description="Plot Type:")
x_dropdown = widgets.Dropdown(description="X-Axis:")
y_dropdown = widgets.Dropdown(description="Y-Axis:")
color_dropdown = widgets.Dropdown(description="Color by:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style='success')
output_area = widgets.Output()

# --- 5. Update columns when user picks a table or plot type ---
def update_columns(change):
    with output_area:
        pass # Keep UI stable
    
    table = table_dropdown.value
    ptype = plot_type.value

    if not table:
        return

    # Load sample data to find column types
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f'SELECT * FROM "{table}" LIMIT 50', conn)
    conn.close()

    # Get different types of columns
    all_cols = df.columns.tolist()
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    color_options = ['None'] + categorical_cols # Add 'None' option

    if ptype == "Scatter":
        x_dropdown.options = numeric_cols
        y_dropdown.options = numeric_cols
        color_dropdown.options = color_options
        color_dropdown.layout.display = 'flex' 
        best_fit.layout.display = 'flex'
    else:
        x_dropdown.options = all_cols
        y_dropdown.options = all_cols
        color_dropdown.layout.display = 'none'
        best_fit.layout.display = 'none'

    # Set default values to prevent errors
    if x_dropdown.options:
        x_dropdown.value = x_dropdown.options[0]
    if y_dropdown.options and len(y_dropdown.options) > 1:
        y_dropdown.value = y_dropdown.options[1]
    elif y_dropdown.options:
        y_dropdown.value = y_dropdown.options[0]
    if color_dropdown.options:
        color_dropdown.value = 'None'


# --- 6. Plotting Helper Functions (Cleaner) ---

def plot_scatter_on_axis(ax, df, x_col, y_col, color_col, add_best_fit):
    """Creates a scatter plot, now with optional coloring and integer axis ticks."""
    
    df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
    df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
    df = df.dropna(subset=[x_col, y_col])

    if df.empty:
        ax.text(0.5, 0.5, "No numeric data to plot.", ha='center', va='center')
        return

    # --- Color Logic ---
    if color_col and color_col != 'None' and color_col in df.columns:
        df = df.dropna(subset=[color_col]) 
        categories = df[color_col].unique()
        
        for category in categories:
            df_group = df[df[color_col] == category]
            ax.scatter(df_group[x_col], df_group[y_col], alpha=0.7, label=str(category))
        if len(categories) > 0:
            ax.legend(title=color_col)
    else:
        ax.scatter(df[x_col], df[y_col], color='blue', alpha=0.7)

    ax.set_ylabel(y_col)

    # --- Best-Fit Line Logic ---
    if add_best_fit and len(df) > 1:
        m, b = np.polyfit(df[x_col], df[y_col], 1)
        ax.plot(df[x_col], m * df[x_col] + b, color='red', linewidth=2)
        
    # --- Set Integer Ticks for 'age' ---
    if x_col == 'age':
        ax.xaxis.set_major_locator(mticker.MultipleLocator(1))
    if y_col == 'age':
        ax.yaxis.set_major_locator(mticker.MultipleLocator(1))


# --- THIS ENTIRE FUNCTION IS REWRITTEN TO FIX THE BLACK BAR ---
def plot_bar_on_axis(ax, df, x_col, y_col):
    """Creates a bar plot. 'age' data should be rounded before calling this."""
    df = df.dropna(subset=[x_col, y_col])
    
    if df.empty:
        ax.text(0.5, 0.5, "No data to plot.", ha='center', va='center')
        return

    is_x_categorical = df[x_col].dtype == 'object'
    is_y_categorical = df[y_col].dtype == 'object'

    if (is_x_categorical or is_y_categorical) or (x_col == 'age' and is_y_categorical):
        # --- NEW: Re-write of "color for gender vs age" plot ---
        # This avoids using pandas.plot() which causes the black bar
        grouped = pd.crosstab(df[x_col], df[y_col])
        
        n_groups = len(grouped)
        n_categories = len(grouped.columns)
        
        # Calculate bar width and positions
        total_width_per_group = 0.8
        bar_width = total_width_per_group / n_categories
        x_indices = np.arange(n_groups) # The center position for each group
        
        # Loop for each category (e.g., 'Male', 'Female') and plot its bars
        for i, category in enumerate(grouped.columns):
            # Calculate the position for this specific bar
            offset = (i - (n_categories - 1) / 2) * bar_width
            bar_positions = x_indices + offset
            
            counts = grouped[category]
            ax.bar(bar_positions, counts, width=bar_width, label=category, alpha=0.8)

        ax.set_ylabel("Count")
        ax.legend(title=y_col, loc='upper right')
        
        # Set the x-ticks to be in the center of the groups
        ax.set_xticks(x_indices)
        ax.set_xticklabels(grouped.index, rotation=45, ha='right')
        # --- END NEW ---

    else:
        # This part is already fixed (uses ax.bar)
        grouped_data = df.groupby(x_col)[y_col].count()
        # Ensure index is string for categorical plotting
        ax.bar(grouped_data.index.astype(str), grouped_data.values, color='skyblue')
        ax.set_ylabel("Count")
        # Apply rotation settings directly to the axis
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')


# --- 7. When user clicks "Generate Plot" (Simple Version) ---
def generate_plot(b):
    with output_area:
        clear_output(wait=True)
        
        # --- Close all old, stray plots to prevent the black bar ---
        plt.close('all') 

        # --- 1. Get User Selections ---
        table = table_dropdown.value
        x_col = x_dropdown.value
        y_col = y_dropdown.value
        ptype = plot_type.value
        color_col = color_dropdown.value
        
        if not table or not x_col or not y_col:
            print("⚠️ Please select a table and valid X/Y columns.")
            return

        # --- 2. Load Full Data ---
        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f'SELECT * FROM "{table}"', conn)
        conn.close()

        # --- 3. Apply Age Filter (if 'age' column exists) ---
        if 'age' in df.columns:
            df['age'] = pd.to_numeric(df['age'], errors='coerce')
            df = df.dropna(subset=['age'])
            df = df[(df['age'] >= 18) & (df['age'] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        if df.empty:
            print("No data available after filtering.")
            return

        # --- 4. Create Plot ---
        fig, ax = plt.subplots(figsize=(7, 4))

        if ptype == "Scatter":
            plot_scatter_on_axis(ax, df, x_col, y_col, color_col, best_fit.value)
        else:
            if 'age' in df.columns:
                df['age'] = df['age'].round().astype(int)
            plot_bar_on_axis(ax, df, x_col, y_col)

        # --- 5. Final Plot Formatting ---
        ax.xaxis.tick_bottom()
        ax.xaxis.set_label_position('bottom')
        ax.yaxis.tick_left()
        ax.yaxis.set_label_position('left')
        
        ax.set_xlabel(x_col)
        ax.set_title(f"{y_col} vs. {x_col} ({ptype})")
        ax.grid(True, linestyle='--', alpha=0.5)

        # --- The rotation logic was moved to plot_bar_on_axis ---
        
        plt.tight_layout()

        # --- 6. Show Image in Output ---
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        display(widgets.Image(value=buf.read(), format='png'))
        
        # We still close the *current* fig, just to be tidy
        plt.close(fig)

# --- 8. Link Widgets to Events ---
table_dropdown.observe(update_columns, names='value')
plot_type.observe(update_columns, names='value')
generate_button.on_click(generate_plot)

# --- 9. Display Interface ---
controls = widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    # Swapped dropdowns as requested
    widgets.HBox([y_dropdown, x_dropdown, color_dropdown]),
    widgets.HBox([generate_button, best_fit])
])

ui = widgets.VBox([controls, output_area])

# Trigger the update once to populate the column dropdowns
update_columns(None)

display(ui)

In [10]:
# --- 1. Import Libraries ---
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker # For integer axis ticks
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

# --- 2. Database Setup ---
DB_NAME = 'Dataset.db'

# --- 3. Get All Tables from Database ---
try:
    conn = sqlite3.connect(DB_NAME)
    tables = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table' AND name != 'sqlite_sequence';",
        conn
    )
    table_list = tables['name'].tolist()
    conn.close()
except Exception as e:
    print("Error loading database:", e)
    table_list = []

# --- 4. Create Widgets ---
table_dropdown = widgets.Dropdown(options=table_list, description="Table:")
plot_type = widgets.Dropdown(options=["Scatter", "Bar"], description="Plot Type:")
x_dropdown = widgets.Dropdown(description="X-Axis:")
y_dropdown = widgets.Dropdown(description="Y-Axis:")
color_dropdown = widgets.Dropdown(description="Color by:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style='success')
output_area = widgets.Output()

# --- 5. Update columns when user picks a table or plot type ---
def update_columns(change):
    with output_area:
        pass # Keep UI stable
    
    table = table_dropdown.value
    ptype = plot_type.value

    if not table:
        return

    # Load sample data to find column types
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f'SELECT * FROM "{table}" LIMIT 50', conn)
    conn.close()

    # Get different types of columns
    all_cols = df.columns.tolist()
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    color_options = ['None'] + categorical_cols # Add 'None' option

    if ptype == "Scatter":
        x_dropdown.options = numeric_cols
        y_dropdown.options = numeric_cols
        color_dropdown.options = color_options
        color_dropdown.layout.display = 'flex' 
        best_fit.layout.display = 'flex'
    else:
        x_dropdown.options = all_cols
        y_dropdown.options = all_cols
        color_dropdown.layout.display = 'none'
        best_fit.layout.display = 'none'

    # Set default values to prevent errors
    if x_dropdown.options:
        x_dropdown.value = x_dropdown.options[0]
    if y_dropdown.options and len(y_dropdown.options) > 1:
        y_dropdown.value = y_dropdown.options[1]
    elif y_dropdown.options:
        y_dropdown.value = y_dropdown.options[0]
    if color_dropdown.options:
        color_dropdown.value = 'None'


# --- 6. Plotting Helper Functions (Cleaner) ---

def plot_scatter_on_axis(ax, df, x_col, y_col, color_col, add_best_fit):
    """Creates a scatter plot, now with optional coloring and integer axis ticks."""
    
    df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
    df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
    df = df.dropna(subset=[x_col, y_col])

    if df.empty:
        ax.text(0.5, 0.5, "No numeric data to plot.", ha='center', va='center')
        return

    # --- Color Logic ---
    if color_col and color_col != 'None' and color_col in df.columns:
        df = df.dropna(subset=[color_col]) 
        categories = df[color_col].unique()
        
        for category in categories:
            df_group = df[df[color_col] == category]
            ax.scatter(df_group[x_col], df_group[y_col], alpha=0.7, label=str(category))
        if len(categories) > 0:
            ax.legend(title=color_col)
    else:
        ax.scatter(df[x_col], df[y_col], color='blue', alpha=0.7)

    ax.set_ylabel(y_col)

    # --- Best-Fit Line Logic ---
    if add_best_fit and len(df) > 1:
        m, b = np.polyfit(df[x_col], df[y_col], 1)
        ax.plot(df[x_col], m * df[x_col] + b, color='red', linewidth=2)
        
    # --- Set Integer Ticks for 'age' ---
    if x_col == 'age':
        ax.xaxis.set_major_locator(mticker.MultipleLocator(1))
    if y_col == 'age':
        ax.yaxis.set_major_locator(mticker.MultipleLocator(1))


def plot_bar_on_axis(ax, df, x_col, y_col):
    """Creates a bar plot. 'age' data should be rounded before calling this."""
    df = df.dropna(subset=[x_col, y_col])
    
    if df.empty:
        ax.text(0.5, 0.5, "No data to plot.", ha='center', va='center')
        return

    is_x_categorical = df[x_col].dtype == 'object'
    is_y_categorical = df[y_col].dtype == 'object'

    # --- THIS IS THE FIX ---
    # Only create a grouped chart if BOTH are categorical (e.g., gender vs. major)
    if is_x_categorical and is_y_categorical:
    # --- END FIX ---
        # --- "color for gender vs major" plot ---
        grouped = pd.crosstab(df[x_col], df[y_col])
        
        n_groups = len(grouped)
        n_categories = len(grouped.columns)
        
        # Calculate bar width and positions
        total_width_per_group = 0.8
        bar_width = total_width_per_group / n_categories
        x_indices = np.arange(n_groups) # The center position for each group
        
        # Loop for each category (e.g., 'Male', 'Female') and plot its bars
        for i, category in enumerate(grouped.columns):
            # Calculate the position for this specific bar
            offset = (i - (n_categories - 1) / 2) * bar_width
            bar_positions = x_indices + offset
            
            counts = grouped[category]
            ax.bar(bar_positions, counts, width=bar_width, label=category, alpha=0.8)

        ax.set_ylabel("Count")
        ax.legend(title=y_col, loc='upper right')
        
        # Set the x-ticks to be in the center of the groups
        ax.set_xticks(x_indices)
        ax.set_xticklabels(grouped.index, rotation=45, ha='right')
        # --- END NEW ---

    else:
        # --- This is the simple bar chart (e.g., "gender vs. age") ---
        # It will group by the X-column and count the Y-column entries.
        grouped_data = df.groupby(x_col)[y_col].count()
        # Ensure index is string for categorical plotting
        ax.bar(grouped_data.index.astype(str), grouped_data.values, color='skyblue')
        ax.set_ylabel("Count")

In [11]:
# --- 1. Import Libraries ---
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker # For integer axis ticks
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

# --- 2. Database Setup ---
DB_NAME = 'Dataset.db'

# --- 3. Get All Tables from Database ---
try:
    conn = sqlite3.connect(DB_NAME)
    tables = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table' AND name != 'sqlite_sequence';",
        conn
    )
    table_list = tables['name'].tolist()
    conn.close()
except Exception as e:
    print("Error loading database:", e)
    table_list = []

# --- 4. Create Widgets ---
table_dropdown = widgets.Dropdown(options=table_list, description="Table:")
plot_type = widgets.Dropdown(options=["Scatter", "Bar"], description="Plot Type:")
x_dropdown = widgets.Dropdown(description="X-Axis:")
y_dropdown = widgets.Dropdown(description="Y-Axis:")
color_dropdown = widgets.Dropdown(description="Color by:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style='success')
output_area = widgets.Output()

# --- 5. Update columns when user picks a table or plot type ---
def update_columns(change):
    with output_area:
        pass # Keep UI stable
    
    table = table_dropdown.value
    ptype = plot_type.value

    if not table:
        return

    # Load sample data to find column types
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f'SELECT * FROM "{table}" LIMIT 50', conn)
    conn.close()

    # Get different types of columns
    all_cols = df.columns.tolist()
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    color_options = ['None'] + categorical_cols # Add 'None' option

    if ptype == "Scatter":
        x_dropdown.options = numeric_cols
        y_dropdown.options = numeric_cols
        color_dropdown.options = color_options
        color_dropdown.layout.display = 'flex' 
        best_fit.layout.display = 'flex'
    else:
        x_dropdown.options = all_cols
        y_dropdown.options = all_cols
        color_dropdown.layout.display = 'none'
        best_fit.layout.display = 'none'

    # Set default values to prevent errors
    if x_dropdown.options:
        x_dropdown.value = x_dropdown.options[0]
    if y_dropdown.options and len(y_dropdown.options) > 1:
        y_dropdown.value = y_dropdown.options[1]
    elif y_dropdown.options:
        y_dropdown.value = y_dropdown.options[0]
    if color_dropdown.options:
        color_dropdown.value = 'None'


# --- 6. Plotting Helper Functions (Cleaner) ---

def plot_scatter_on_axis(ax, df, x_col, y_col, color_col, add_best_fit):
    """Creates a scatter plot, now with optional coloring and integer axis ticks."""
    
    df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
    df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
    df = df.dropna(subset=[x_col, y_col])

    if df.empty:
        ax.text(0.5, 0.5, "No numeric data to plot.", ha='center', va='center')
        return

    # --- Color Logic ---
    if color_col and color_col != 'None' and color_col in df.columns:
        df = df.dropna(subset=[color_col]) 
        categories = df[color_col].unique()
        
        for category in categories:
            df_group = df[df[color_col] == category]
            ax.scatter(df_group[x_col], df_group[y_col], alpha=0.7, label=str(category))
        if len(categories) > 0:
            ax.legend(title=color_col)
    else:
        ax.scatter(df[x_col], df[y_col], color='blue', alpha=0.7)

    ax.set_ylabel(y_col)

    # --- Best-Fit Line Logic ---
    if add_best_fit and len(df) > 1:
        m, b = np.polyfit(df[x_col], df[y_col], 1)
        ax.plot(df[x_col], m * df[x_col] + b, color='red', linewidth=2)
        
    # --- Set Integer Ticks for 'age' ---
    if x_col == 'age':
        ax.xaxis.set_major_locator(mticker.MultipleLocator(1))
    if y_col == 'age':
        ax.yaxis.set_major_locator(mticker.MultipleLocator(1))


def plot_bar_on_axis(ax, df, x_col, y_col):
    """Creates a bar plot. 'age' data should be rounded before calling this."""
    df = df.dropna(subset=[x_col, y_col])
    
    if df.empty:
        ax.text(0.5, 0.5, "No data to plot.", ha='center', va='center')
        return

    is_x_categorical = df[x_col].dtype == 'object'
    is_y_categorical = df[y_col].dtype == 'object'

    # --- THIS IS THE FIX ---
    # Only create a grouped chart if BOTH are categorical (e.g., gender vs. major)
    if is_x_categorical and is_y_categorical:
    # --- END FIX ---
        # --- "color for gender vs major" plot ---
        grouped = pd.crosstab(df[x_col], df[y_col])
        
        n_groups = len(grouped)
        n_categories = len(grouped.columns)
        
        # Calculate bar width and positions
        total_width_per_group = 0.8
        bar_width = total_width_per_group / n_categories
        x_indices = np.arange(n_groups) # The center position for each group
        
        # Loop for each category (e.g., 'Male', 'Female') and plot its bars
        for i, category in enumerate(grouped.columns):
            # Calculate the position for this specific bar
            offset = (i - (n_categories - 1) / 2) * bar_width
            bar_positions = x_indices + offset
            
            counts = grouped[category]
            ax.bar(bar_positions, counts, width=bar_width, label=category, alpha=0.8)

        ax.set_ylabel("Count")
        ax.legend(title=y_col, loc='upper right')
        
        # Set the x-ticks to be in the center of the groups
        ax.set_xticks(x_indices)
        ax.set_xticklabels(grouped.index, rotation=45, ha='right')
        # --- END NEW ---

    else:
        # --- This is the simple bar chart (e.g., "gender vs. age") ---
        # It will group by the X-column and count the Y-column entries.
        grouped_data = df.groupby(x_col)[y_col].count()
        # Ensure index is string for categorical plotting
        ax.bar(grouped_data.index.astype(str), grouped_data.values, color='skyblue')
        ax.set_ylabel("Count")
        # Apply rotation settings directly to the axis
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')


# --- 7. When user clicks "Generate Plot" (Simple Version) ---
def generate_plot(b):
    with output_area:
        clear_output(wait=True)
        
        # --- Close all old, stray plots to prevent the black bar ---
        plt.close('all') 

        # --- 1. Get User Selections ---
        table = table_dropdown.value
        x_col = x_dropdown.value
        y_col = y_dropdown.value
        ptype = plot_type.value
        color_col = color_dropdown.value
        
        if not table or not x_col or not y_col:
            print("⚠️ Please select a table and valid X/Y columns.")
            return

        # --- 2. Load Full Data ---
        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f'SELECT * FROM "{table}"', conn)
        conn.close()

        # --- 3. Apply Age Filter (if 'age' column exists) ---
        if 'age' in df.columns:
            df['age'] = pd.to_numeric(df['age'], errors='coerce')
            df = df.dropna(subset=['age'])
            df = df[(df['age'] >= 18) & (df['age'] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        if df.empty:
            print("No data available after filtering.")
            return

        # --- 4. Create Plot ---
        fig, ax = plt.subplots(figsize=(7, 4))

        if ptype == "Scatter":
            plot_scatter_on_axis(ax, df, x_col, y_col, color_col, best_fit.value)
        else:
            if 'age' in df.columns:
                df['age'] = df['age'].round().astype(int)
            plot_bar_on_axis(ax, df, x_col, y_col)

        # --- 5. Final Plot Formatting ---
        ax.xaxis.tick_bottom()
        ax.xaxis.set_label_position('bottom')
        ax.yaxis.tick_left()
        ax.yaxis.set_label_position('left')
        
        ax.set_xlabel(x_col)
        ax.set_title(f"{y_col} vs. {x_col} ({ptype})")
        ax.grid(True, linestyle='--', alpha=0.5)
        
        plt.tight_layout()

        # --- 6. Show Image in Output ---
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        display(widgets.Image(value=buf.read(), format='png'))
        
        # We still close the *current* fig, just to be tidy
        plt.close(fig)

# --- 8. Link Widgets to Events ---
table_dropdown.observe(update_columns, names='value')
plot_type.observe(update_columns, names='value')
generate_button.on_click(generate_plot)

# --- 9. Display Interface ---
controls = widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    # Swapped dropdowns as requested
    widgets.HBox([y_dropdown, x_dropdown, color_dropdown]),
    widgets.HBox([generate_button, best_fit])
])

ui = widgets.VBox([controls, output_area])

# Trigger the update once to populate the column dropdowns
update_columns(None)

display(ui)

In [12]:
# --- 1. Import Libraries ---
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker # For integer axis ticks
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

# --- 2. Database Setup ---
DB_NAME = 'Dataset.db'

# --- 3. Get All Tables from Database ---
try:
    conn = sqlite3.connect(DB_NAME)
    tables = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table' AND name != 'sqlite_sequence';",
        conn
    )
    table_list = tables['name'].tolist()
    conn.close()
except Exception as e:
    print("Error loading database:", e)
    table_list = []

# --- 4. Create Widgets ---
table_dropdown = widgets.Dropdown(options=table_list, description="Table:")
plot_type = widgets.Dropdown(options=["Scatter", "Bar"], description="Plot Type:")
x_dropdown = widgets.Dropdown(description="X-Axis:")
y_dropdown = widgets.Dropdown(description="Y-Axis:")
color_dropdown = widgets.Dropdown(description="Color by:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style='success')
output_area = widgets.Output()

# --- 5. Update columns when user picks a table or plot type ---
def update_columns(change):
    with output_area:
        pass # Keep UI stable
    
    table = table_dropdown.value
    ptype = plot_type.value

    if not table:
        return

    # Load sample data to find column types
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f'SELECT * FROM "{table}" LIMIT 50', conn)
    conn.close()

    # Get different types of columns
    all_cols = df.columns.tolist()
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    color_options = ['None'] + categorical_cols # Add 'None' option

    if ptype == "Scatter":
        x_dropdown.options = numeric_cols
        y_dropdown.options = numeric_cols
        color_dropdown.options = color_options
        color_dropdown.layout.display = 'flex' 
        best_fit.layout.display = 'flex'
    else:
        x_dropdown.options = all_cols
        y_dropdown.options = all_cols
        color_dropdown.layout.display = 'none'
        best_fit.layout.display = 'none'

    # Set default values to prevent errors
    if x_dropdown.options:
        x_dropdown.value = x_dropdown.options[0]
    if y_dropdown.options and len(y_dropdown.options) > 1:
        y_dropdown.value = y_dropdown.options[1]
    elif y_dropdown.options:
        y_dropdown.value = y_dropdown.options[0]
    if color_dropdown.options:
        color_dropdown.value = 'None'


# --- 6. Plotting Helper Functions (Cleaner) ---

def plot_scatter_on_axis(ax, df, x_col, y_col, color_col, add_best_fit):
    """Creates a scatter plot, now with optional coloring and integer axis ticks."""
    
    df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
    df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
    df = df.dropna(subset=[x_col, y_col])

    if df.empty:
        ax.text(0.5, 0.5, "No numeric data to plot.", ha='center', va='center')
        return

    # --- Color Logic ---
    if color_col and color_col != 'None' and color_col in df.columns:
        df = df.dropna(subset=[color_col]) 
        categories = df[color_col].unique()
        
        for category in categories:
            df_group = df[df[color_col] == category]
            ax.scatter(df_group[x_col], df_group[y_col], alpha=0.7, label=str(category))
        if len(categories) > 0:
            ax.legend(title=color_col)
    else:
        ax.scatter(df[x_col], df[y_col], color='blue', alpha=0.7)

    ax.set_ylabel(y_col)

    # --- Best-Fit Line Logic ---
    if add_best_fit and len(df) > 1:
        m, b = np.polyfit(df[x_col], df[y_col], 1)
        ax.plot(df[x_col], m * df[x_col] + b, color='red', linewidth=2)
        
    # --- Set Integer Ticks for 'age' ---
    if x_col == 'age':
        ax.xaxis.set_major_locator(mticker.MultipleLocator(1))
    if y_col == 'age':
        ax.yaxis.set_major_locator(mticker.MultipleLocator(1))


def plot_bar_on_axis(ax, df, x_col, y_col):
    """Creates a bar plot. 'age' data should be rounded before calling this."""
    df = df.dropna(subset=[x_col, y_col])
    
    if df.empty:
        ax.text(0.5, 0.5, "No data to plot.", ha='center', va='center')
        return

    is_x_categorical = df[x_col].dtype == 'object'
    is_y_categorical = df[y_col].dtype == 'object'

    # Only create a grouped chart if BOTH are categorical (e.g., gender vs. major)
    if is_x_categorical and is_y_categorical:
        # --- "color for gender vs major" plot ---
        grouped = pd.crosstab(df[x_col], df[y_col])
        
        n_groups = len(grouped)
        n_categories = len(grouped.columns)
        
        # Calculate bar width and positions
        total_width_per_group = 0.8
        bar_width = total_width_per_group / n_categories
        x_indices = np.arange(n_groups) # The center position for each group
        
        # Loop for each category (e.g., 'Male', 'Female') and plot its bars
        for i, category in enumerate(grouped.columns):
            # Calculate the position for this specific bar
            offset = (i - (n_categories - 1) / 2) * bar_width
            bar_positions = x_indices + offset
            
            counts = grouped[category]
            ax.bar(bar_positions, counts, width=bar_width, label=category, alpha=0.8)

        ax.set_ylabel("Count")
        ax.legend(title=y_col, loc='upper right')
        
        # Set the x-ticks to be in the center of the groups
        ax.set_xticks(x_indices)
        ax.set_xticklabels(grouped.index, rotation=45, ha='right')

    else:
        # --- This is the simple bar chart (e.g., "gender vs. age") ---
        grouped_data = df.groupby(x_col)[y_col].count()
        
        # --- THIS IS THE FIX ---
        # Get labels and positions
        x_labels = grouped_data.index.astype(str)
        x_indices = np.arange(len(x_labels))

        # Plot the bar
        ax.bar(x_indices, grouped_data.values, color='skyblue')
        ax.set_ylabel("Count")
        
        # NOW we set the ticks and labels explicitly
        ax.set_xticks(x_indices) # Tell matplotlib where the ticks are
        ax.set_xticklabels(x_labels, rotation=45, ha='right') # NOW set the labels
        # --- END FIX ---


# --- 7. When user clicks "Generate Plot" (Simple Version) ---
def generate_plot(b):
    with output_area:
        clear_output(wait=True)
        
        # --- Close all old, stray plots to prevent the black bar ---
        plt.close('all') 

        # --- 1. Get User Selections ---
        table = table_dropdown.value
        x_col = x_dropdown.value
        y_col = y_dropdown.value
        ptype = plot_type.value
        color_col = color_dropdown.value
        
        if not table or not x_col or not y_col:
            print("⚠️ Please select a table and valid X/Y columns.")
            return

        # --- 2. Load Full Data ---
        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f'SELECT * FROM "{table}"', conn)
        conn.close()

        # --- 3. Apply Age Filter (if 'age' column exists) ---
        if 'age' in df.columns:
            df['age'] = pd.to_numeric(df['age'], errors='coerce')
            df = df.dropna(subset=['age'])
            df = df[(df['age'] >= 18) & (df['age'] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        if df.empty:
            print("No data available after filtering.")
            return

        # --- 4. Create Plot ---
        fig, ax = plt.subplots(figsize=(7, 4))

        if ptype == "Scatter":
            plot_scatter_on_axis(ax, df, x_col, y_col, color_col, best_fit.value)
        else:
            if 'age' in df.columns:
                df['age'] = df['age'].round().astype(int)
            plot_bar_on_axis(ax, df, x_col, y_col)

        # --- 5. Final Plot Formatting ---
        ax.xaxis.tick_bottom()
        ax.xaxis.set_label_position('bottom')
        ax.yaxis.tick_left()
        ax.yaxis.set_label_position('left')
        
        ax.set_xlabel(x_col)
        ax.set_title(f"{y_col} vs. {x_col} ({ptype})")
        ax.grid(True, linestyle='--', alpha=0.5)
        
        plt.tight_layout()

        # --- 6. Show Image in Output ---
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        display(widgets.Image(value=buf.read(), format='png'))
        
        # We still close the *current* fig, just to be tidy
        plt.close(fig)

# --- 8. Link Widgets to Events ---
table_dropdown.observe(update_columns, names='Svalue')
plot_type.observe(update_columns, names='value')
generate_button.on_click(generate_plot)

# --- 9. Display Interface ---
controls = widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    # Swapped dropdowns as requested
    widgets.HBox([y_dropdown, x_dropdown, color_dropdown]),
    widgets.HBox([generate_button, best_fit])
])

ui = widgets.VBox([controls, output_area])

# Trigger the update once to populate the column dropdowns
update_columns(None)

display(ui)

In [13]:
# --- 1. Import Libraries ---
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker # For integer axis ticks
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

# --- 2. Database Setup ---
DB_NAME = 'Dataset.db'

# --- 3. Get All Tables from Database ---
try:
    conn = sqlite3.connect(DB_NAME)
    tables = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table' AND name != 'sqlite_sequence';",
        conn
    )
    table_list = tables['name'].tolist()
    conn.close()
except Exception as e:
    print("Error loading database:", e)
    table_list = []

# --- 4. Create Widgets ---
table_dropdown = widgets.Dropdown(options=table_list, description="Table:")
plot_type = widgets.Dropdown(options=["Scatter", "Bar"], description="Plot Type:")
x_dropdown = widgets.Dropdown(description="X-Axis:")
y_dropdown = widgets.Dropdown(description="Y-Axis:")
color_dropdown = widgets.Dropdown(description="Color by:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style='success')
output_area = widgets.Output()

# --- 5. Update columns when user picks a table or plot type ---
def update_columns(change):
    with output_area:
        pass # Keep UI stable
    
    table = table_dropdown.value
    ptype = plot_type.value

    if not table:
        return

    # Load sample data to find column types
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f'SELECT * FROM "{table}" LIMIT 50', conn)
    conn.close()

    # Get different types of columns
    all_cols = df.columns.tolist()
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    color_options = ['None'] + categorical_cols # Add 'None' option

    if ptype == "Scatter":
        x_dropdown.options = numeric_cols
        y_dropdown.options = numeric_cols
        color_dropdown.options = color_options
        color_dropdown.layout.display = 'flex' 
        best_fit.layout.display = 'flex'
    else:
        x_dropdown.options = all_cols
        y_dropdown.options = all_cols
        color_dropdown.layout.display = 'none'
        best_fit.layout.display = 'none'

    # Set default values to prevent errors
    if x_dropdown.options:
        x_dropdown.value = x_dropdown.options[0]
    if y_dropdown.options and len(y_dropdown.options) > 1:
        y_dropdown.value = y_dropdown.options[1]
    elif y_dropdown.options:
        y_dropdown.value = y_dropdown.options[0]
    if color_dropdown.options:
        color_dropdown.value = 'None'


# --- 6. Plotting Helper Functions (Cleaner) ---

def plot_scatter_on_axis(ax, df, x_col, y_col, color_col, add_best_fit):
    """Creates a scatter plot, now with optional coloring and integer axis ticks."""
    
    df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
    df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
    df = df.dropna(subset=[x_col, y_col])

    if df.empty:
        ax.text(0.5, 0.5, "No numeric data to plot.", ha='center', va='center')
        return

    # --- Color Logic ---
    if color_col and color_col != 'None' and color_col in df.columns:
        df = df.dropna(subset=[color_col]) 
        categories = df[color_col].unique()
        
        for category in categories:
            df_group = df[df[color_col] == category]
            ax.scatter(df_group[x_col], df_group[y_col], alpha=0.7, label=str(category))
        if len(categories) > 0:
            ax.legend(title=color_col)
    else:
        ax.scatter(df[x_col], df[y_col], color='blue', alpha=0.7)

    ax.set_ylabel(y_col)

    # --- Best-Fit Line Logic ---
    if add_best_fit and len(df) > 1:
        m, b = np.polyfit(df[x_col], df[y_col], 1)
        ax.plot(df[x_col], m * df[x_col] + b, color='red', linewidth=2)
        
    # --- Set Integer Ticks for 'age' ---
    if x_col == 'age':
        ax.xaxis.set_major_locator(mticker.MultipleLocator(1))
    if y_col == 'age':
        ax.yaxis.set_major_locator(mticker.MultipleLocator(1))


def plot_bar_on_axis(ax, df, x_col, y_col):
    """Creates a bar plot. 'age' data should be rounded before calling this."""
    df = df.dropna(subset=[x_col, y_col])
    
    if df.empty:
        ax.text(0.5, 0.5, "No data to plot.", ha='center', va='center')
        return

    is_x_categorical = df[x_col].dtype == 'object'
    is_y_categorical = df[y_col].dtype == 'object'

    # Only create a grouped chart if BOTH are categorical (e.g., gender vs. major)
    if is_x_categorical and is_y_categorical:
        # --- "color for gender vs major" plot ---
        grouped = pd.crosstab(df[x_col], df[y_col])
        
        n_groups = len(grouped)
        n_categories = len(grouped.columns)
        
        # Calculate bar width and positions
        total_width_per_group = 0.8
        bar_width = total_width_per_group / n_categories
        x_indices = np.arange(n_groups) # The center position for each group
        
        # Loop for each category (e.g., 'Male', 'Female') and plot its bars
        for i, category in enumerate(grouped.columns):
            # Calculate the position for this specific bar
            offset = (i - (n_categories - 1) / 2) * bar_width
            bar_positions = x_indices + offset
            
            counts = grouped[category]
            ax.bar(bar_positions, counts, width=bar_width, label=category, alpha=0.8)

        ax.set_ylabel("Count")
        ax.legend(title=y_col, loc='upper right')
        
        # Set the x-ticks to be in the center of the groups
        ax.set_xticks(x_indices)
        ax.set_xticklabels(grouped.index, rotation=45, ha='right')

    else:
        # --- This is the simple bar chart (e.g., "gender vs. age") ---
        grouped_data = df.groupby(x_col)[y_col].count()
        
        # --- THIS IS THE FIX ---
        # Get labels and positions
        x_labels = grouped_data.index.astype(str)
        x_indices = np.arange(len(x_labels))

        # Plot the bar
        ax.bar(x_indices, grouped_data.values, color='skyblue')
        ax.set_ylabel("Count")
        
        # NOW we set the ticks and labels explicitly
        ax.set_xticks(x_indices) # Tell matplotlib where the ticks are
        ax.set_xticklabels(x_labels, rotation=45, ha='right') # NOW set the labels
        # --- END FIX ---


# --- 7. When user clicks "Generate Plot" (Simple Version) ---
def generate_plot(b):
    with output_area:
        clear_output(wait=True)
        
        # --- Close all old, stray plots to prevent the black bar ---
        plt.close('all') 

        # --- 1. Get User Selections ---
        table = table_dropdown.value
        x_col = x_dropdown.value
        y_col = y_dropdown.value
        ptype = plot_type.value
        color_col = color_dropdown.value
        
        if not table or not x_col or not y_col:
            print("⚠️ Please select a table and valid X/Y columns.")
            return

        # --- 2. Load Full Data ---
        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f'SELECT * FROM "{table}"', conn)
        conn.close()

        # --- 3. Apply Age Filter (if 'age' column exists) ---
        if 'age' in df.columns:
            df['age'] = pd.to_numeric(df['age'], errors='coerce')
            df = df.dropna(subset=['age'])
            df = df[(df['age'] >= 18) & (df['age'] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        if df.empty:
            print("No data available after filtering.")
            return

        # --- 4. Create Plot ---
        fig, ax = plt.subplots(figsize=(7, 4))

        if ptype == "Scatter":
            plot_scatter_on_axis(ax, df, x_col, y_col, color_col, best_fit.value)
        else:
            if 'age' in df.columns:
                df['age'] = df['age'].round().astype(int)
            plot_bar_on_axis(ax, df, x_col, y_col)

        # --- 5. Final Plot Formatting ---
        ax.xaxis.tick_bottom()
        ax.xaxis.set_label_position('bottom')
        ax.yaxis.tick_left()
        ax.yaxis.set_label_position('left')
        
        ax.set_xlabel(x_col)
        ax.set_title(f"{y_col} vs. {x_col} ({ptype})")
        ax.grid(True, linestyle='--', alpha=0.5)
        
        plt.tight_layout()

        # --- 6. Show Image in Output ---
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        display(widgets.Image(value=buf.read(), format='png'))
        
        # We still close the *current* fig, just to be tidy
        plt.close(fig)

# --- 8. Link Widgets to Events ---
table_dropdown.observe(update_columns, names='Svalue')
plot_type.observe(update_columns, names='value')
generate_button.on_click(generate_plot)

# --- 9. Display Interface ---
controls = widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    # Swapped dropdowns as requested
    widgets.HBox([y_dropdown, x_dropdown, color_dropdown]),
    widgets.HBox([generate_button, best_fit])
])

ui = widgets.VBox([controls, output_area])

# Trigger the update once to populate the column dropdowns
update_columns(None)

display(ui)

In [14]:
def plot_bar(ax, df, x_col, y_col):
    """Improved bar plot — auto-spaces labels if there are many X values."""
    df = df.dropna(subset=[x_col, y_col])

    if df.empty:
        ax.text(0.5, 0.5, "No data to plot.", ha='center', va='center')
        return

    is_x_cat = df[x_col].dtype == 'object'
    is_y_cat = df[y_col].dtype == 'object'

    if is_x_cat and is_y_cat:
        # --- Grouped Bar Chart ---
        grouped = pd.crosstab(df[x_col], df[y_col])
        grouped.plot(kind='bar', ax=ax, width=0.8, alpha=0.8)
        ax.legend(title=y_col)
        plt.xticks(rotation=45, ha='right')
        ax.set_ylabel("Count")

    else:
        # --- Single Bar Chart (e.g., age vs student_id or similar) ---
        grouped = df.groupby(x_col)[y_col].count().sort_index()
        x_labels = grouped.index.astype(str)
        x_indices = np.arange(len(x_labels))

        # Thin bars if there are many
        bar_width = max(0.8 / np.log1p(len(x_labels)), 0.2)
        ax.bar(x_indices, grouped.values, color='skyblue', width=bar_width)
        ax.set_ylabel("Count")

        # --- Smarter tick spacing ---
        if len(x_labels) > 20:
            # Show every Nth label
            step = max(len(x_labels) // 20, 1)
            ax.set_xticks(x_indices[::step])
            ax.set_xticklabels(x_labels[::step], rotation=45, ha='right')
        else:
            ax.set_xticks(x_indices)
            ax.set_xticklabels(x_labels, rotation=45, ha='right')

        # Avoid clipping labels
        ax.margins(x=0.01)

        # If the X column looks like an ID, make note
        if "id" in x_col.lower():
            ax.set_xlabel(f"{x_col} (sampled labels for readability)")


In [15]:
# --- 1. Import Libraries ---
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

# --- 2. Database Setup ---
DB_NAME = 'Dataset.db'

# --- 3. Get All Tables from Database ---
try:
    conn = sqlite3.connect(DB_NAME)
    tables = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table' AND name != 'sqlite_sequence';",
        conn
    )
    table_list = tables['name'].tolist()
    conn.close()
except Exception as e:
    print("Error loading database:", e)
    table_list = []

# --- 4. Create Widgets ---
table_dropdown = widgets.Dropdown(options=table_list, description="Table:")
plot_type = widgets.Dropdown(options=["Scatter", "Bar"], description="Plot Type:")
x_dropdown = widgets.Dropdown(description="X-Axis:")
y_dropdown = widgets.Dropdown(description="Y-Axis:")
color_dropdown = widgets.Dropdown(description="Color by:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style='success')
output_area = widgets.Output()

# --- 5. Update Columns When Table or Plot Type Changes ---
def update_columns(change):
    with output_area:
        pass  # Keep UI stable

    table = table_dropdown.value
    ptype = plot_type.value
    if not table:
        return

    # Load sample data
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f'SELECT * FROM "{table}" LIMIT 50', conn)
    conn.close()

    all_cols = df.columns.tolist()
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    color_options = ['None'] + categorical_cols

    if ptype == "Scatter":
        x_dropdown.options = numeric_cols
        y_dropdown.options = numeric_cols
        color_dropdown.options = color_options
        color_dropdown.layout.display = 'flex'
        best_fit.layout.display = 'flex'
    else:
        x_dropdown.options = all_cols
        y_dropdown.options = all_cols
        color_dropdown.layout.display = 'none'
        best_fit.layout.display = 'none'

    if x_dropdown.options:
        x_dropdown.value = x_dropdown.options[0]
    if y_dropdown.options and len(y_dropdown.options) > 1:
        y_dropdown.value = y_dropdown.options[1]
    elif y_dropdown.options:
        y_dropdown.value = y_dropdown.options[0]
    if color_dropdown.options:
        color_dropdown.value = 'None'


# --- 6. Plotting Helper Functions ---
def plot_scatter(ax, df, x_col, y_col, color_col, add_best_fit):
    df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
    df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
    df = df.dropna(subset=[x_col, y_col])

    if df.empty:
        ax.text(0.5, 0.5, "No numeric data to plot.", ha='center', va='center')
        return

    # Color handling
    if color_col and color_col != 'None' and color_col in df.columns:
        df = df.dropna(subset=[color_col])
        categories = df[color_col].unique()
        for cat in categories:
            ax.scatter(df[df[color_col] == cat][x_col],
                       df[df[color_col] == cat][y_col],
                       alpha=0.7, label=str(cat))
        ax.legend(title=color_col)
    else:
        ax.scatter(df[x_col], df[y_col], color='blue', alpha=0.7)

    # Best-fit line
    if add_best_fit and len(df) > 1:
        m, b = np.polyfit(df[x_col], df[y_col], 1)
        ax.plot(df[x_col], m * df[x_col] + b, color='red', linewidth=2)

    if x_col == 'age':
        ax.xaxis.set_major_locator(mticker.MultipleLocator(1))
    if y_col == 'age':
        ax.yaxis.set_major_locator(mticker.MultipleLocator(1))

    ax.set_ylabel(y_col)


def plot_bar(ax, df, x_col, y_col):
    df = df.dropna(subset=[x_col, y_col])
    if df.empty:
        ax.text(0.5, 0.5, "No data to plot.", ha='center', va='center')
        return

    is_x_cat = df[x_col].dtype == 'object'
    is_y_cat = df[y_col].dtype == 'object'

    if is_x_cat and is_y_cat:
        # Grouped bar (e.g., gender vs major)
        grouped = pd.crosstab(df[x_col], df[y_col])
        grouped.plot(kind='bar', ax=ax, width=0.8, alpha=0.8)
        ax.legend(title=y_col)
        plt.xticks(rotation=45, ha='right')
        ax.set_ylabel("Count")
    else:
        # Standard bar chart (e.g., age vs student_id)
        grouped = df.groupby(x_col)[y_col].count().sort_index()
        x_labels = grouped.index.astype(str)
        x_indices = np.arange(len(x_labels))

        # Thin bars if many labels
        bar_width = max(0.8 / np.log1p(len(x_labels)), 0.2)
        ax.bar(x_indices, grouped.values, color='skyblue', width=bar_width)
        ax.set_ylabel("Count")

        # Smarter tick spacing
        if len(x_labels) > 20:
            step = max(len(x_labels) // 20, 1)
            ax.set_xticks(x_indices[::step])
            ax.set_xticklabels(x_labels[::step], rotation=45, ha='right')
        else:
            ax.set_xticks(x_indices)
            ax.set_xticklabels(x_labels, rotation=45, ha='right')

        ax.margins(x=0.01)
        if "id" in x_col.lower():
            ax.set_xlabel(f"{x_col} (sampled labels for readability)")


# --- 7. Generate Plot Function ---
def generate_plot(b):
    with output_area:
        clear_output(wait=True)
        plt.close('all')

        table = table_dropdown.value
        x_col = x_dropdown.value
        y_col = y_dropdown.value
        ptype = plot_type.value
        color_col = color_dropdown.value

        if not table or not x_col or not y_col:
            print("⚠️ Please select a table and valid X/Y columns.")
            return

        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f'SELECT * FROM "{table}"', conn)
        conn.close()

        # Age filter
        if 'age' in df.columns:
            df['age'] = pd.to_numeric(df['age'], errors='coerce')
            df = df.dropna(subset=['age'])
            df = df[(df['age'] >= 18) & (df['age'] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        if df.empty:
            print("No data available after filtering.")
            return

        fig, ax = plt.subplots(figsize=(7, 4))

        if ptype == "Scatter":
            plot_scatter(ax, df, x_col, y_col, color_col, best_fit.value)
        else:
            if 'age' in df.columns:
                df['age'] = df['age'].round().astype(int)
            plot_bar(ax, df, x_col, y_col)

        # Final formatting
        ax.set_xlabel(x_col)
        ax.set_title(f"{y_col} vs. {x_col} ({ptype})")
        ax.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()

        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        display(widgets.Image(value=buf.read(), format='png'))
        plt.close(fig)

# --- 8. Link Widgets ---
table_dropdown.observe(update_columns, names='value')
plot_type.observe(update_columns, names='value')
generate_button.on_click(generate_plot)

# --- 9. Display UI ---
controls = widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([y_dropdown, x_dropdown, color_dropdown]),
    widgets.HBox([generate_button, best_fit])
])

ui = widgets.VBox([controls, output_area])
update_columns(None)
display(ui)


In [16]:
# Simple Jupyter Plot App
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Connect to database
DB_NAME = "Dataset.db"
conn = sqlite3.connect(DB_NAME)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
table_names = tables["name"].tolist()
conn.close()

# Widgets
table_dropdown = widgets.Dropdown(options=table_names, description="Table:")
plot_type = widgets.Dropdown(options=["Bar", "Scatter"], description="Plot Type:")
x_axis = widgets.Dropdown(description="X-Axis:")
y_axis = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style="success")
output = widgets.Output()

# Update dropdowns when table changes
def update_columns(change):
    table = table_dropdown.value
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f"SELECT * FROM '{table}' LIMIT 50", conn)
    conn.close()

    cols = df.columns.tolist()
    x_axis.options = cols
    y_axis.options = cols
    if len(cols) > 1:
        x_axis.value = cols[0]
        y_axis.value = cols[1]

table_dropdown.observe(update_columns, names="value")

# Plot when button is clicked
def generate_plot(b):
    with output:
        clear_output(wait=True)
        
        table = table_dropdown.value
        x_col = x_axis.value
        y_col = y_axis.value
        kind = plot_type.value

        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f"SELECT * FROM '{table}'", conn)
        conn.close()

        # Filter by age if column exists
        if "age" in df.columns:
            df["age"] = pd.to_numeric(df["age"], errors="coerce")
            df = df[(df["age"] >= 18) & (df["age"] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        # Drop missing data
        df = df.dropna(subset=[x_col, y_col])
        if df.empty:
            print("No data to plot.")
            return

        plt.figure(figsize=(7,4))
        
        if kind == "Scatter":
            plt.scatter(df[x_col], df[y_col], color="skyblue", alpha=0.7)
            if best_fit.value:
                # Add simple best-fit line
                if df[x_col].dtype != 'object' and df[y_col].dtype != 'object':
                    m, b = np.polyfit(df[x_col], df[y_col], 1)
                    plt.plot(df[x_col], m*df[x_col]+b, color="red")
        else:
            # Group by X and count Y
            counts = df.groupby(x_col)[y_col].count()
            plt.bar(counts.index.astype(str), counts.values, color="skyblue")
            
            # Make X labels readable if too many
            if len(counts) > 20:
                step = max(1, len(counts)//20)
                plt.xticks(counts.index[::step], rotation=45)
            else:
                plt.xticks(rotation=45)
            
        plt.title(f"{y_col} vs {x_col} ({kind})")
        plt.xlabel(x_col)
        plt.ylabel(y_col if kind=="Scatter" else "Count")
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

generate_button.on_click(generate_plot)

# Display all controls
display(widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_axis, y_axis]),
    widgets.HBox([generate_button, best_fit]),
    output
]))


In [17]:
# Simple Jupyter Plot App
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Connect to database
DB_NAME = "Dataset.db"
conn = sqlite3.connect(DB_NAME)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
table_names = [t for t in tables["name"].tolist() if t != "sqlite_sequence"]  # remove sqlite_sequence
conn.close()

# Widgets
table_dropdown = widgets.Dropdown(options=table_names, description="Table:")
plot_type = widgets.Dropdown(options=["Bar", "Scatter"], description="Plot Type:")
x_axis = widgets.Dropdown(description="X-Axis:")
y_axis = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style="success")
output = widgets.Output()

# Update dropdowns when table changes
def update_columns(change):
    table = table_dropdown.value
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f"SELECT * FROM '{table}' LIMIT 50", conn)
    conn.close()

    cols = df.columns.tolist()
    x_axis.options = cols
    y_axis.options = cols
    if len(cols) > 1:
        x_axis.value = cols[0]
        y_axis.value = cols[1]

table_dropdown.observe(update_columns, names="value")

# Plot when button is clicked
def generate_plot(b):
    with output:
        clear_output(wait=True)
        
        table = table_dropdown.value
        x_col = x_axis.value
        y_col = y_axis.value
        kind = plot_type.value

        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f"SELECT * FROM '{table}'", conn)
        conn.close()

        # Filter by age if column exists
        if "age" in df.columns:
            df["age"] = pd.to_numeric(df["age"], errors="coerce")
            df = df[(df["age"] >= 18) & (df["age"] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        # Drop missing data
        df = df.dropna(subset=[x_col, y_col])
        if df.empty:
            print("No data to plot.")
            return

        plt.figure(figsize=(7,4))
        
        if kind == "Scatter":
            plt.scatter(df[x_col], df[y_col], color="skyblue", alpha=0.7)
            if best_fit.value:
                # Add simple best-fit line
                if df[x_col].dtype != 'object' and df[y_col].dtype != 'object':
                    m, b = np.polyfit(df[x_col], df[y_col], 1)
                    plt.plot(df[x_col], m*df[x_col]+b, color="red")
        else:
            # Group by X and count Y
            counts = df.groupby(x_col)[y_col].count()
            plt.bar(counts.index.astype(str), counts.values, color="skyblue")
            
            # Make X labels readable if too many
            if len(counts) > 20:
                step = max(1, len(counts)//20)
                plt.xticks(counts.index[::step], rotation=45)
            else:
                plt.xticks(rotation=45)
            
        plt.title(f"{y_col} vs {x_col} ({kind})")
        plt.xlabel(x_col)
        plt.ylabel(y_col if kind=="Scatter" else "Count")
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

generate_button.on_click(generate_plot)

# Display all controls
display(widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_axis, y_axis]),
    widgets.HBox([generate_button, best_fit]),
    output
]))


In [18]:
# Simple Data Plot App for Beginners
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Connect to database
DB_NAME = "Dataset.db"
conn = sqlite3.connect(DB_NAME)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
table_names = [t for t in tables["name"].tolist() if t != "sqlite_sequence"]
conn.close()

# Widgets
table_dropdown = widgets.Dropdown(options=table_names, description="Table:")
plot_type = widgets.Dropdown(options=["Bar", "Scatter"], description="Plot Type:")
x_axis = widgets.Dropdown(description="X-Axis:")
y_axis = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style="success")
output = widgets.Output()

# Update dropdowns when table changes
def update_columns(change):
    table = table_dropdown.value
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f"SELECT * FROM '{table}' LIMIT 50", conn)
    conn.close()

    cols = df.columns.tolist()
    x_axis.options = cols
    y_axis.options = cols
    if len(cols) > 1:
        x_axis.value = cols[0]
        y_axis.value = cols[1]

table_dropdown.observe(update_columns, names="value")

# Plot when button is clicked
def generate_plot(b):
    with output:
        clear_output(wait=True)
        
        table = table_dropdown.value
        x_col = x_axis.value
        y_col = y_axis.value
        kind = plot_type.value

        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f"SELECT * FROM '{table}'", conn)
        conn.close()

        # Filter by age if it exists
        if "age" in df.columns:
            df["age"] = pd.to_numeric(df["age"], errors="coerce")
            df = df[(df["age"] >= 18) & (df["age"] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        df = df.dropna(subset=[x_col, y_col])
        if df.empty:
            print("No data to plot.")
            return

        plt.figure(figsize=(7,4))
        
        # --- Scatter plot ---
        if kind == "Scatter":
            plt.scatter(df[x_col], df[y_col], color="skyblue", alpha=0.7)
            if best_fit.value:
                if df[x_col].dtype != 'object' and df[y_col].dtype != 'object':
                    m, b = np.polyfit(df[x_col], df[y_col], 1)
                    plt.plot(df[x_col], m*df[x_col]+b, color="red")

        # --- Bar plot ---
        else:
            # Case 1: Both are categorical (grouped bar)
            if df[x_col].dtype == "object" and df[y_col].dtype == "object":
                grouped = pd.crosstab(df[x_col], df[y_col])  # counts per category
                ages = np.arange(len(grouped))
                bar_width = 0.8 / len(grouped.columns)
                
                for i, col in enumerate(grouped.columns):
                    plt.bar(
                        ages + i*bar_width, 
                        grouped[col].values, 
                        width=bar_width, 
                        label=str(col), 
                        alpha=0.8
                    )
                
                plt.xticks(ages + bar_width*(len(grouped.columns)/2 - 0.5), grouped.index, rotation=45)
                plt.legend(title=y_col)
                plt.ylabel("Count")

            # Case 2: One numeric, one categorical (simple count)
            else:
                counts = df.groupby(x_col)[y_col].count()
                plt.bar(counts.index.astype(str), counts.values, color="skyblue")
                if len(counts) > 20:
                    step = max(1, len(counts)//20)
                    plt.xticks(counts.index[::step], rotation=45)
                else:
                    plt.xticks(rotation=45)
                plt.ylabel("Count")
            
        plt.title(f"{y_col} vs {x_col} ({kind})")
        plt.xlabel(x_col)
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

generate_button.on_click(generate_plot)

# Display
display(widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_axis, y_axis]),
    widgets.HBox([generate_button, best_fit]),
    output
]))


In [19]:
# Simple Data Plot App for Beginners
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Connect to database
DB_NAME = "Dataset.db"
conn = sqlite3.connect(DB_NAME)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
table_names = [t for t in tables["name"].tolist() if t != "sqlite_sequence"]
conn.close()

# Widgets
table_dropdown = widgets.Dropdown(options=table_names, description="Table:")
plot_type = widgets.Dropdown(options=["Bar", "Scatter"], description="Plot Type:")
x_axis = widgets.Dropdown(description="X-Axis:")
y_axis = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style="success")
output = widgets.Output()

# Update dropdowns when table changes
def update_columns(change):
    table = table_dropdown.value
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f"SELECT * FROM '{table}' LIMIT 50", conn)
    conn.close()

    cols = df.columns.tolist()
    x_axis.options = cols
    y_axis.options = cols
    if len(cols) > 1:
        x_axis.value = cols[0]
        y_axis.value = cols[1]

table_dropdown.observe(update_columns, names="value")

# Plot when button is clicked
def generate_plot(b):
    with output:
        clear_output(wait=True)
        
        table = table_dropdown.value
        x_col = x_axis.value
        y_col = y_axis.value
        kind = plot_type.value

        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f"SELECT * FROM '{table}'", conn)
        conn.close()

        # Filter by age if it exists
        if "age" in df.columns:
            df["age"] = pd.to_numeric(df["age"], errors="coerce")
            df = df[(df["age"] >= 18) & (df["age"] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        df = df.dropna(subset=[x_col, y_col])
        if df.empty:
            print("No data to plot.")
            return

        plt.figure(figsize=(7,4))
        
        # --- Scatter plot ---
        if kind == "Scatter":
            plt.scatter(df[x_col], df[y_col], color="skyblue", alpha=0.7)
            if best_fit.value:
                if df[x_col].dtype != 'object' and df[y_col].dtype != 'object':
                    m, b = np.polyfit(df[x_col], df[y_col], 1)
                    plt.plot(df[x_col], m*df[x_col]+b, color="red")

        # --- Bar plot ---
        else:
            # Case 1: Both are categorical (grouped bar)
            if df[x_col].dtype == "object" and df[y_col].dtype == "object":
                grouped = pd.crosstab(df[x_col], df[y_col])  # counts per category
                ages = np.arange(len(grouped))
                bar_width = 0.8 / len(grouped.columns)
                
                for i, col in enumerate(grouped.columns):
                    plt.bar(
                        ages + i*bar_width, 
                        grouped[col].values, 
                        width=bar_width, 
                        label=str(col), 
                        alpha=0.8
                    )
                
                plt.xticks(ages + bar_width*(len(grouped.columns)/2 - 0.5), grouped.index, rotation=45)
                plt.legend(title=y_col)
                plt.ylabel("Count")

            # Case 2: One numeric, one categorical (simple count)
            else:
                counts = df.groupby(x_col)[y_col].count()
                plt.bar(counts.index.astype(str), counts.values, color="skyblue")
                if len(counts) > 20:
                    step = max(1, len(counts)//20)
                    plt.xticks(counts.index[::step], rotation=45)
                else:
                    plt.xticks(rotation=45)
                plt.ylabel("Count")
            
        plt.title(f"{y_col} vs {x_col} ({kind})")
        plt.xlabel(x_col)
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

generate_button.on_click(generate_plot)

# Display
display(widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_axis, y_axis]),
    widgets.HBox([generate_button, best_fit]),
    output
]))


In [20]:
# Simple Data Plot App for Beginners
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Connect to database
DB_NAME = "Dataset.db"
conn = sqlite3.connect(DB_NAME)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
table_names = [t for t in tables["name"].tolist() if t != "sqlite_sequence"]
conn.close()

# Widgets
table_dropdown = widgets.Dropdown(options=table_names, description="Table:")
plot_type = widgets.Dropdown(options=["Bar", "Scatter"], description="Plot Type:")
x_axis = widgets.Dropdown(description="X-Axis:")
y_axis = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style="success")
output = widgets.Output()

# Update dropdowns when table changes
def update_columns(change):
    table = table_dropdown.value
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f"SELECT * FROM '{table}' LIMIT 50", conn)
    conn.close()

    cols = df.columns.tolist()
    x_axis.options = cols
    y_axis.options = cols
    if len(cols) > 1:
        x_axis.value = cols[0]
        y_axis.value = cols[1]

table_dropdown.observe(update_columns, names="value")

# Plot when button is clicked
def generate_plot(b):
    with output:
        clear_output(wait=True)
        
        table = table_dropdown.value
        x_col = x_axis.value
        y_col = y_axis.value
        kind = plot_type.value

        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f"SELECT * FROM '{table}'", conn)
        conn.close()

        # Filter by age if it exists
        if "age" in df.columns:
            df["age"] = pd.to_numeric(df["age"], errors="coerce")
            df = df[(df["age"] >= 18) & (df["age"] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        df = df.dropna(subset=[x_col, y_col])
        if df.empty:
            print("No data to plot.")
            return

        plt.figure(figsize=(7,4))
        
        # --- Scatter plot ---
        if kind == "Scatter":
            plt.scatter(df[x_col], df[y_col], color="skyblue", alpha=0.7)
            if best_fit.value:
                if df[x_col].dtype != 'object' and df[y_col].dtype != 'object':
                    m, b = np.polyfit(df[x_col], df[y_col], 1)
                    plt.plot(df[x_col], m*df[x_col]+b, color="red")

        # --- Bar plot ---
        else:
            # Convert age to string to allow grouped bars (treat numeric as category)
            if x_col == "age":
                df["age"] = df["age"].astype(int).astype(str)
            
            # Grouped bar: count of Y per X
            grouped = pd.crosstab(df[x_col], df[y_col])
            groups = np.arange(len(grouped))
            bar_width = 0.8 / len(grouped.columns)

            for i, cat in enumerate(grouped.columns):
                plt.bar(
                    groups + i * bar_width,
                    grouped[cat].values,
                    width=bar_width,
                    label=str(cat),
                    alpha=0.8
                )

            plt.xticks(groups + bar_width*(len(grouped.columns)/2 - 0.5), grouped.index, rotation=45)
            plt.ylabel("Count")
            plt.legend(title=y_col)

        plt.title(f"{y_col} vs {x_col} ({kind})")
        plt.xlabel(x_col)
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

generate_button.on_click(generate_plot)

# Display
display(widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_axis, y_axis]),
    widgets.HBox([generate_button, best_fit]),
    output
]))


In [21]:
# Simple Jupyter Plot App with explicit age vs gender bars
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Database file name
DB_NAME = "Dataset.db"

# Read table names (skip sqlite_sequence)
conn = sqlite3.connect(DB_NAME)
tables_df = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
table_names = [t for t in tables_df["name"].tolist() if t != "sqlite_sequence"]
conn.close()

# Widgets
table_dropdown = widgets.Dropdown(options=table_names, description="Table:")
plot_type = widgets.Dropdown(options=["Bar", "Scatter"], description="Plot Type:")
x_axis = widgets.Dropdown(description="X-Axis:")
y_axis = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style="success")
output = widgets.Output()

# Populate column dropdowns when table changes
def update_columns(change):
    table = table_dropdown.value
    if not table:
        return
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f"SELECT * FROM '{table}' LIMIT 50", conn)
    conn.close()
    cols = df.columns.tolist()
    x_axis.options = cols
    y_axis.options = cols
    if len(cols) > 1:
        x_axis.value = cols[0]
        y_axis.value = cols[1]

table_dropdown.observe(update_columns, names="value")

# Plotting
def generate_plot(b):
    with output:
        clear_output(wait=True)
        table = table_dropdown.value
        x_col = x_axis.value
        y_col = y_axis.value
        kind = plot_type.value

        if not table or not x_col or not y_col:
            print("Select a table and both X and Y columns.")
            return

        # Load full data
        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f"SELECT * FROM '{table}'", conn)
        conn.close()

        # Age filter if present
        if "age" in df.columns:
            df["age"] = pd.to_numeric(df["age"], errors="coerce")
            df = df[(df["age"] >= 18) & (df["age"] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        # Drop rows missing selected columns
        df = df.dropna(subset=[x_col, y_col])
        if df.empty:
            print("No data to plot after filtering.")
            return

        plt.figure(figsize=(8,4))

        if kind == "Scatter":
            # Simple scatter
            plt.scatter(df[x_col], df[y_col], color="skyblue", alpha=0.7)
            # Best-fit line only when both numeric
            if best_fit.value:
                if pd.api.types.is_numeric_dtype(df[x_col]) and pd.api.types.is_numeric_dtype(df[y_col]):
                    m, b = np.polyfit(df[x_col], df[y_col], 1)
                    plt.plot(df[x_col], m*df[x_col] + b, color="red")
            plt.xlabel(x_col)
            plt.ylabel(y_col)

        else:
            # ---------- Special explicit case: age (X) vs gender (Y) ----------
            if x_col == "age" and y_col.lower() == "gender":
                # Ensure age is integer and string for ticks
                df["age"] = df["age"].round().astype(int)
                # Keep only the ages 18-22 in sorted order (if present)
                ages = sorted(df["age"].unique())
                ages = [a for a in ages if 18 <= a <= 22]

                # Get counts per age for Male and Female (case-insensitive)
                df["gender_norm"] = df["gender"].astype(str).str.strip().str.lower()
                male_counts = []
                female_counts = []
                for a in ages:
                    sub = df[df["age"] == a]
                    male_counts.append((sub["gender_norm"] == "male").sum())
                    female_counts.append((sub["gender_norm"] == "female").sum())

                x = np.arange(len(ages))
                width = 0.35
                plt.bar(x - width/2, male_counts, width=width, label="Male")
                plt.bar(x + width/2, female_counts, width=width, label="Female")
                plt.xticks(x, [str(a) for a in ages], rotation=45)
                plt.ylabel("Count")
                plt.xlabel("age")
                plt.legend()
                plt.title("gender counts by age (Male vs Female)")
            else:
                # ---------- Generic bar: use crosstab to make grouped bars ----------
                # Treat numeric X as categorical for bar grouping
                if pd.api.types.is_numeric_dtype(df[x_col]):
                    df[x_col] = df[x_col].round().astype(int).astype(str)
                else:
                    df[x_col] = df[x_col].astype(str)

                df[y_col] = df[y_col].astype(str)
                grouped = pd.crosstab(df[x_col], df[y_col])
                if grouped.shape[1] == 1:
                    # Single series: just plot counts
                    counts = grouped.iloc[:,0].values
                    labels = grouped.index.tolist()
                    plt.bar(labels, counts, color="skyblue")
                    plt.xticks(rotation=45)
                    plt.ylabel("Count")
                    plt.xlabel(x_col)
                else:
                    # Multiple categories: grouped bars
                    n_groups = len(grouped)
                    n_cats = grouped.shape[1]
                    bar_width = 0.8 / n_cats
                    indices = np.arange(n_groups)
                    for i, col in enumerate(grouped.columns):
                        plt.bar(indices + i*bar_width, grouped[col].values, width=bar_width, label=str(col))
                    plt.xticks(indices + (n_cats-1)*bar_width/2, grouped.index, rotation=45)
                    plt.ylabel("Count")
                    plt.xlabel(x_col)
                    plt.legend(title=y_col)

        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

generate_button.on_click(generate_plot)

# Initialize columns for the first table if present
if table_names:
    table_dropdown.value = table_names[0]
    update_columns(None)

# Display the UI
display(widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_axis, y_axis]),
    widgets.HBox([generate_button, best_fit]),
    output
]))


In [22]:
# Simple Data Plot App for Beginners
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Connect to database
DB_NAME = "Dataset.db"
conn = sqlite3.connect(DB_NAME)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
table_names = [t for t in tables["name"].tolist() if t != "sqlite_sequence"]
conn.close()

# Widgets
table_dropdown = widgets.Dropdown(options=table_names, description="Table:")
plot_type = widgets.Dropdown(options=["Bar", "Scatter"], description="Plot Type:")
x_axis = widgets.Dropdown(description="X-Axis:")
y_axis = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style="success")
output = widgets.Output()

# Update dropdowns when table changes
def update_columns(change):
    table = table_dropdown.value
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f"SELECT * FROM '{table}' LIMIT 50", conn)
    conn.close()

    cols = df.columns.tolist()
    x_axis.options = cols
    y_axis.options = cols
    if len(cols) > 1:
        x_axis.value = cols[0]
        y_axis.value = cols[1]

table_dropdown.observe(update_columns, names="value")

# Plot when button is clicked
def generate_plot(b):
    with output:
        clear_output(wait=True)
        
        table = table_dropdown.value
        x_col = x_axis.value
        y_col = y_axis.value
        kind = plot_type.value

        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f"SELECT * FROM '{table}'", conn)
        conn.close()

        # Filter by age if it exists
        if "age" in df.columns:
            df["age"] = pd.to_numeric(df["age"], errors="coerce")
            df = df[(df["age"] >= 18) & (df["age"] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        df = df.dropna(subset=[x_col, y_col])
        if df.empty:
            print("No data to plot.")
            return

        plt.figure(figsize=(7,4))
        
        # --- Scatter plot ---
        if kind == "Scatter":
            plt.scatter(df[x_col], df[y_col], color="skyblue", alpha=0.7)
            if best_fit.value:
                if df[x_col].dtype != 'object' and df[y_col].dtype != 'object':
                    m, b = np.polyfit(df[x_col], df[y_col], 1)
                    plt.plot(df[x_col], m*df[x_col]+b, color="red")

        # --- Bar plot ---
        else:
            # Case 1: Both are categorical (grouped bar)
            if df[x_col].dtype == "object" and df[y_col].dtype == "object":
                grouped = pd.crosstab(df[x_col], df[y_col])  # counts per category
                ages = np.arange(len(grouped))
                bar_width = 0.8 / len(grouped.columns)
                
                for i, col in enumerate(grouped.columns):
                    plt.bar(
                        ages + i*bar_width, 
                        grouped[col].values, 
                        width=bar_width, 
                        label=str(col), 
                        alpha=0.8
                    )
                
                plt.xticks(ages + bar_width*(len(grouped.columns)/2 - 0.5), grouped.index, rotation=45)
                plt.legend(title=y_col)
                plt.ylabel("Count")

            # Case 2: One numeric, one categorical (simple count)
            else:
                counts = df.groupby(x_col)[y_col].count()
                plt.bar(counts.index.astype(str), counts.values, color="skyblue")
                if len(counts) > 20:
                    step = max(1, len(counts)//20)
                    plt.xticks(counts.index[::step], rotation=45)
                else:
                    plt.xticks(rotation=45)
                plt.ylabel("Count")
            
        plt.title(f"{y_col} vs {x_col} ({kind})")
        plt.xlabel(x_col)
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

generate_button.on_click(generate_plot)

# Display
display(widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_axis, y_axis]),
    widgets.HBox([generate_button, best_fit]),
    output
]))


In [23]:
# Simple Data Plot App for Beginners (with Male/Female per Age)
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Connect to database
DB_NAME = "Dataset.db"
conn = sqlite3.connect(DB_NAME)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
table_names = [t for t in tables["name"].tolist() if t != "sqlite_sequence"]
conn.close()

# Widgets
table_dropdown = widgets.Dropdown(options=table_names, description="Table:")
plot_type = widgets.Dropdown(options=["Bar", "Scatter"], description="Plot Type:")
x_axis = widgets.Dropdown(description="X-Axis:")
y_axis = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style="success")
output = widgets.Output()

# Update dropdowns when table changes
def update_columns(change):
    table = table_dropdown.value
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f"SELECT * FROM '{table}' LIMIT 50", conn)
    conn.close()

    cols = df.columns.tolist()
    x_axis.options = cols
    y_axis.options = cols
    if len(cols) > 1:
        x_axis.value = cols[0]
        y_axis.value = cols[1]

table_dropdown.observe(update_columns, names="value")

# Plot when button is clicked
def generate_plot(b):
    with output:
        clear_output(wait=True)
        
        table = table_dropdown.value
        x_col = x_axis.value
        y_col = y_axis.value
        kind = plot_type.value

        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f"SELECT * FROM '{table}'", conn)
        conn.close()

        # Filter by age if it exists
        if "age" in df.columns:
            df["age"] = pd.to_numeric(df["age"], errors="coerce")
            df = df[(df["age"] >= 18) & (df["age"] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        df = df.dropna(subset=[x_col, y_col])
        if df.empty:
            print("No data to plot.")
            return

        plt.figure(figsize=(7,4))
        
        # --- Scatter plot ---
        if kind == "Scatter":
            plt.scatter(df[x_col], df[y_col], color="skyblue", alpha=0.7)
            if best_fit.value:
                if df[x_col].dtype != 'object' and df[y_col].dtype != 'object':
                    m, b = np.polyfit(df[x_col], df[y_col], 1)
                    plt.plot(df[x_col], m*df[x_col]+b, color="red")

        # --- Bar plot ---
        else:
            # Special case: gender vs age (show male/female bars per age)
            if (x_col == "age" and y_col == "gender") or (x_col == "gender" and y_col == "age"):
                # Make sure "age" is numeric
                df["age"] = pd.to_numeric(df["age"], errors="coerce")
                df = df.dropna(subset=["age", "gender"])
                
                grouped = df.groupby(["age", "gender"]).size().unstack(fill_value=0)
                ages = np.arange(len(grouped))
                bar_width = 0.35

                # Male and Female bars
                if "Male" in grouped.columns:
                    plt.bar(ages - bar_width/2, grouped["Male"], width=bar_width, label="Male", color="skyblue")
                if "Female" in grouped.columns:
                    plt.bar(ages + bar_width/2, grouped["Female"], width=bar_width, label="Female", color="pink")

                plt.xticks(ages, grouped.index.astype(int))
                plt.ylabel("Count")
                plt.xlabel("Age")
                plt.legend(title="Gender")

            # Case 1: Both categorical (grouped bar)
            elif df[x_col].dtype == "object" and df[y_col].dtype == "object":
                grouped = pd.crosstab(df[x_col], df[y_col])
                labels = np.arange(len(grouped))
                bar_width = 0.8 / len(grouped.columns)
                
                for i, col in enumerate(grouped.columns):
                    plt.bar(
                        labels + i * bar_width,
                        grouped[col].values,
                        width=bar_width,
                        label=str(col),
                        alpha=0.8
                    )
                
                plt.xticks(labels + bar_width*(len(grouped.columns)/2 - 0.5), grouped.index, rotation=45)
                plt.legend(title=y_col)
                plt.ylabel("Count")

            # Case 2: One numeric, one categorical (simple count)
            else:
                counts = df.groupby(x_col)[y_col].count()
                plt.bar(counts.index.astype(str), counts.values, color="skyblue")
                if len(counts) > 20:
                    step = max(1, len(counts)//20)
                    plt.xticks(counts.index[::step], rotation=45)
                else:
                    plt.xticks(rotation=45)
                plt.ylabel("Count")
            
        plt.title(f"{y_col} vs {x_col} ({kind})")
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

generate_button.on_click(generate_plot)

# Display
display(widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_axis, y_axis]),
    widgets.HBox([generate_button, best_fit]),
    output
]))


In [24]:
# Simple Data Plot App for Beginners (fixed gender legend issue)
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Connect to database
DB_NAME = "Dataset.db"
conn = sqlite3.connect(DB_NAME)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
table_names = [t for t in tables["name"].tolist() if t != "sqlite_sequence"]
conn.close()

# Widgets
table_dropdown = widgets.Dropdown(options=table_names, description="Table:")
plot_type = widgets.Dropdown(options=["Bar", "Scatter"], description="Plot Type:")
x_axis = widgets.Dropdown(description="X-Axis:")
y_axis = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style="success")
output = widgets.Output()

# Update dropdowns when table changes
def update_columns(change):
    table = table_dropdown.value
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f"SELECT * FROM '{table}' LIMIT 50", conn)
    conn.close()

    cols = df.columns.tolist()
    x_axis.options = cols
    y_axis.options = cols
    if len(cols) > 1:
        x_axis.value = cols[0]
        y_axis.value = cols[1]

table_dropdown.observe(update_columns, names="value")

# Plot when button is clicked
def generate_plot(b):
    with output:
        clear_output(wait=True)
        
        table = table_dropdown.value
        x_col = x_axis.value
        y_col = y_axis.value
        kind = plot_type.value

        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f"SELECT * FROM '{table}'", conn)
        conn.close()

        # Filter by age if available
        if "age" in df.columns:
            df["age"] = pd.to_numeric(df["age"], errors="coerce")
            df = df[(df["age"] >= 18) & (df["age"] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        df = df.dropna(subset=[x_col, y_col])
        if df.empty:
            print("No data to plot.")
            return

        plt.figure(figsize=(7,4))
        
        # --- Scatter plot ---
        if kind == "Scatter":
            plt.scatter(df[x_col], df[y_col], color="skyblue", alpha=0.7)
            if best_fit.value:
                if df[x_col].dtype != 'object' and df[y_col].dtype != 'object':
                    m, b = np.polyfit(df[x_col], df[y_col], 1)
                    plt.plot(df[x_col], m*df[x_col]+b, color="red")

        # --- Bar plot ---
        else:
            # Special case: gender vs age
            if (x_col == "age" and y_col == "gender") or (x_col == "gender" and y_col == "age"):
                df["age"] = pd.to_numeric(df["age"], errors="coerce")
                df = df.dropna(subset=["age", "gender"])
                
                grouped = df.groupby(["age", "gender"]).size().unstack(fill_value=0)
                ages = np.arange(len(grouped))
                bar_width = 0.35

                bars_drawn = []

                if "Male" in grouped.columns:
                    bars_drawn.append(
                        plt.bar(ages - bar_width/2, grouped["Male"], width=bar_width, label="Male", color="skyblue")
                    )
                if "Female" in grouped.columns:
                    bars_drawn.append(
                        plt.bar(ages + bar_width/2, grouped["Female"], width=bar_width, label="Female", color="pink")
                    )

                plt.xticks(ages, grouped.index.astype(int))
                plt.ylabel("Count")
                plt.xlabel("Age")

                if bars_drawn:  # only show legend if bars exist
                    plt.legend(title="Gender")

            # Case 1: Both categorical (grouped bar)
            elif df[x_col].dtype == "object" and df[y_col].dtype == "object":
                grouped = pd.crosstab(df[x_col], df[y_col])
                labels = np.arange(len(grouped))
                bar_width = 0.8 / len(grouped.columns)
                
                for i, col in enumerate(grouped.columns):
                    plt.bar(
                        labels + i * bar_width,
                        grouped[col].values,
                        width=bar_width,
                        label=str(col),
                        alpha=0.8
                    )
                
                plt.xticks(labels + bar_width*(len(grouped.columns)/2 - 0.5), grouped.index, rotation=45)
                plt.legend(title=y_col)
                plt.ylabel("Count")

            # Case 2: One numeric, one categorical (simple count)
            else:
                counts = df.groupby(x_col)[y_col].count()
                plt.bar(counts.index.astype(str), counts.values, color="skyblue")
                if len(counts) > 20:
                    step = max(1, len(counts)//20)
                    plt.xticks(counts.index[::step], rotation=45)
                else:
                    plt.xticks(rotation=45)
                plt.ylabel("Count")
            
        plt.title(f"{y_col} vs {x_col} ({kind})")
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

generate_button.on_click(generate_plot)

# Display
display(widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_axis, y_axis]),
    widgets.HBox([generate_button, best_fit]),
    output
]))


In [25]:
# Simple Data Plot App for Beginners
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Connect to database
DB_NAME = "Dataset.db"
conn = sqlite3.connect(DB_NAME)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
table_names = [t for t in tables["name"].tolist() if t != "sqlite_sequence"]
conn.close()

# Widgets
table_dropdown = widgets.Dropdown(options=table_names, description="Table:")
plot_type = widgets.Dropdown(options=["Bar", "Scatter"], description="Plot Type:")
x_axis = widgets.Dropdown(description="X-Axis:")
y_axis = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style="success")
output = widgets.Output()

# Update dropdowns when table changes
def update_columns(change):
    table = table_dropdown.value
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f"SELECT * FROM '{table}' LIMIT 50", conn)
    conn.close()

    cols = df.columns.tolist()
    x_axis.options = cols
    y_axis.options = cols
    if len(cols) > 1:
        x_axis.value = cols[0]
        y_axis.value = cols[1]

table_dropdown.observe(update_columns, names="value")

# Plot when button is clicked
def generate_plot(b):
    with output:
        clear_output(wait=True)
        
        table = table_dropdown.value
        x_col = x_axis.value
        y_col = y_axis.value
        kind = plot_type.value

        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f"SELECT * FROM '{table}'", conn)
        conn.close()

        # Filter by age if it exists
        if "age" in df.columns:
            df["age"] = pd.to_numeric(df["age"], errors="coerce")
            df = df[(df["age"] >= 18) & (df["age"] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        df = df.dropna(subset=[x_col, y_col])
        if df.empty:
            print("No data to plot.")
            return

        plt.figure(figsize=(7,4))
        
        # --- Scatter plot ---
        if kind == "Scatter":
            plt.scatter(df[x_col], df[y_col], color="skyblue", alpha=0.7)
            if best_fit.value:
                if df[x_col].dtype != 'object' and df[y_col].dtype != 'object':
                    m, b = np.polyfit(df[x_col], df[y_col], 1)
                    plt.plot(df[x_col], m*df[x_col]+b, color="red")

        # --- Bar plot ---
        else:
            # Case 1: Both are categorical (grouped bar)
            if df[x_col].dtype == "object" and df[y_col].dtype == "object":
                grouped = pd.crosstab(df[x_col], df[y_col])  # counts per category
                ages = np.arange(len(grouped))
                bar_width = 0.8 / len(grouped.columns)
                
                for i, col in enumerate(grouped.columns):
                    plt.bar(
                        ages + i*bar_width, 
                        grouped[col].values, 
                        width=bar_width, 
                        label=str(col), 
                        alpha=0.8
                    )
                
                plt.xticks(ages + bar_width*(len(grouped.columns)/2 - 0.5), grouped.index, rotation=45)
                plt.legend(title=y_col)
                plt.ylabel("Count")

            # Case 2: One numeric, one categorical (simple count)
            else:
                counts = df.groupby(x_col)[y_col].count()
                plt.bar(counts.index.astype(str), counts.values, color="skyblue")
                if len(counts) > 20:
                    step = max(1, len(counts)//20)
                    plt.xticks(counts.index[::step], rotation=45)
                else:
                    plt.xticks(rotation=45)
                plt.ylabel("Count")
            
        plt.title(f"{y_col} vs {x_col} ({kind})")
        plt.xlabel(x_col)
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

generate_button.on_click(generate_plot)

# Display
display(widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_axis, y_axis]),
    widgets.HBox([generate_button, best_fit]),
    output
]))


In [26]:
# Simple Data Plot App for Beginners
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Connect to database
DB_NAME = "Dataset.db"
conn = sqlite3.connect(DB_NAME)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
table_names = [t for t in tables["name"].tolist() if t != "sqlite_sequence"]
conn.close()

# Widgets
table_dropdown = widgets.Dropdown(options=table_names, description="Table:")
plot_type = widgets.Dropdown(options=["Bar", "Scatter"], description="Plot Type:")
x_axis = widgets.Dropdown(description="X-Axis:")
y_axis = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style="success")
output = widgets.Output()

# Update dropdowns when table changes
def update_columns(change):
    table = table_dropdown.value
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f"SELECT * FROM '{table}' LIMIT 50", conn)
    conn.close()

    cols = df.columns.tolist()
    x_axis.options = cols
    y_axis.options = cols
    if len(cols) > 1:
        x_axis.value = cols[0]
        y_axis.value = cols[1]

table_dropdown.observe(update_columns, names="value")

# Plot when button is clicked
def generate_plot(b):
    with output:
        clear_output(wait=True)
        
        table = table_dropdown.value
        x_col = x_axis.value
        y_col = y_axis.value
        kind = plot_type.value

        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f"SELECT * FROM '{table}'", conn)
        conn.close()

        # Filter by age if it exists
        if "age" in df.columns:
            # --- Added rounding for bar charts ---
            df["age"] = pd.to_numeric(df["age"], errors="coerce")
            df = df.dropna(subset=['age'])
            df = df[(df["age"] >= 18) & (df["age"] <= 22)]
            if kind == "Bar":
                 df['age'] = df['age'].round().astype(int)
            print(f"Filter applied: {len(df)} students aged 18–22.")

        df = df.dropna(subset=[x_col, y_col])
        if df.empty:
            print("No data to plot.")
            return

        plt.figure(figsize=(7,4))
        
        # --- Scatter plot ---
        if kind == "Scatter":
            # Convert to numeric for scatter
            df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
            df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
            df = df.dropna(subset=[x_col, y_col])
            
            plt.scatter(df[x_col], df[y_col], color="skyblue", alpha=0.7)
            
            if best_fit.value:
                # Check dtypes again after conversion
                if df[x_col].dtype != 'object' and df[y_col].dtype != 'object' and len(df) > 1:
                    m, b = np.polyfit(df[x_col], df[y_col], 1)
                    plt.plot(df[x_col], m*df[x_col]+b, color="red")

        # --- Bar plot ---
        else:
            # --- THIS IS THE FIX ---
            # If Y is categorical (like 'gender'), make a grouped bar chart
            if df[y_col].dtype == "object":
                # Case 1: Grouped bar chart (e.g., X=age, Y=gender)
                grouped = pd.crosstab(df[x_col], df[y_col]) # counts per category
                ages = np.arange(len(grouped))
                # Use a narrower bar width for grouped charts
                bar_width = 0.8 / len(grouped.columns) 
                
                for i, col in enumerate(grouped.columns):
                    plt.bar(
                        ages + i*bar_width, 
                        grouped[col].values, 
                        width=bar_width, 
                        label=str(col), 
                        alpha=0.8
                    )
                
                # Center the x-ticks between the groups
                plt.xticks(ages + bar_width*(len(grouped.columns)/2 - 0.5), grouped.index, rotation=45, ha="right")
                plt.legend(title=y_col)
                plt.ylabel("Count")

            # --- THIS LOGIC IS ALSO IMPROVED ---
            # Case 2: Simple count (e.g., X=age, Y=student_id)
            else:
                counts = df.groupby(x_col)[y_col].count()
                
                # Get labels and integer positions for plotting
                labels = counts.index.astype(str)
                positions = np.arange(len(labels))
                
                plt.bar(positions, counts.values, color="skyblue")
                plt.ylabel("Count")

                # Set ticks
                if len(positions) > 50:
                    # If too many, hide labels to prevent black bar
                    plt.xticks([])
                elif len(positions) > 20:
                    # If a lot, show every Nth label
                    step = max(1, len(positions) // 20)
                    plt.xticks(positions[::step], labels[::step], rotation=45, ha="right")
                else:
                    # Otherwise, show all
                    plt.xticks(positions, labels, rotation=45, ha="right")
            
        plt.title(f"{y_col} vs {x_col} ({kind})")
        plt.xlabel(x_col)
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

generate_button.on_click(generate_plot)

# Display
display(widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_axis, y_axis]),
    widgets.HBox([generate_button, best_fit]),
    output
]))

# Trigger the update once to load the first table's columns
if table_names:
    update_columns(None)

In [27]:
# Simple Data Plot App for Beginners
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Connect to database
DB_NAME = "Dataset.db"
conn = sqlite3.connect(DB_NAME)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
table_names = [t for t in tables["name"].tolist() if t != "sqlite_sequence"]
conn.close()

# Widgets
table_dropdown = widgets.Dropdown(options=table_names, description="Table:")
plot_type = widgets.Dropdown(options=["Bar", "Scatter"], description="Plot Type:")
x_axis = widgets.Dropdown(description="X-Axis:")
y_axis = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style="success")
output = widgets.Output()

# Update dropdowns when table changes
def update_columns(change):
    table = table_dropdown.value
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f"SELECT * FROM '{table}' LIMIT 50", conn)
    conn.close()

    cols = df.columns.tolist()
    x_axis.options = cols
    y_axis.options = cols
    if len(cols) > 1:
        x_axis.value = cols[0]
        y_axis.value = cols[1]

table_dropdown.observe(update_columns, names="value")

# Plot when button is clicked
def generate_plot(b):
    with output:
        clear_output(wait=True)
        
        table = table_dropdown.value
        x_col = x_axis.value
        y_col = y_axis.value
        kind = plot_type.value

        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f"SELECT * FROM '{table}'", conn)
        conn.close()

        # Filter by age if it exists
        if "age" in df.columns:
            # Added rounding for bar charts
            df["age"] = pd.to_numeric(df["age"], errors="coerce")
            df = df.dropna(subset=['age'])
            df = df[(df["age"] >= 18) & (df["age"] <= 22)]
            if kind == "Bar":
                 df['age'] = df['age'].round().astype(int)
            print(f"Filter applied: {len(df)} students aged 18–22.")

        df = df.dropna(subset=[x_col, y_col])
        if df.empty:
            print("No data to plot.")
            return

        plt.figure(figsize=(7,4))
        
        # --- Scatter plot ---
        if kind == "Scatter":
            # Convert to numeric for scatter
            df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
            df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
            df = df.dropna(subset=[x_col, y_col])
            
            plt.scatter(df[x_col], df[y_col], color="skyblue", alpha=0.7)
            
            if best_fit.value:
                # Check dtypes again after conversion
                if df[x_col].dtype != 'object' and df[y_col].dtype != 'object' and len(df) > 1:
                    m, b = np.polyfit(df[x_col], df[y_col], 1)
                    plt.plot(df[x_col], m*df[x_col]+b, color="red")

        # --- Bar plot ---
        else:
            # If Y is categorical (like 'gender'), make a grouped bar chart
            if df[y_col].dtype == "object":
                # Case 1: Grouped bar chart (e.g., X=age, Y=gender)
                grouped = pd.crosstab(df[x_col], df[y_col]) # counts per category
                ages = np.arange(len(grouped))
                bar_width = 0.8 / len(grouped.columns) 
                
                for i, col in enumerate(grouped.columns):
                    plt.bar(
                        ages + i*bar_width, 
                        grouped[col].values, 
                        width=bar_width, 
                        label=str(col), 
                        alpha=0.8
                    )
                
                plt.legend(title=y_col)
                plt.ylabel("Count")

                # --- THIS IS THE FIX ---
                # Get labels and positions for ticks
                labels = grouped.index.astype(str)
                positions = ages + bar_width*(len(grouped.columns)/2 - 0.5)

                if len(positions) > 50:
                    # If too many, hide labels to prevent black bar
                    plt.xticks([])
                elif len(positions) > 20:
                    # If a lot, show every Nth label
                    step = max(1, len(positions) // 20)
                    plt.xticks(positions[::step], labels[::step], rotation=45, ha="right")
                else:
                    # Otherwise, show all
                    plt.xticks(positions, labels, rotation=45, ha="right")
                # --- END FIX ---

            # Case 2: Simple count (e.g., X=age, Y=student_id)
            else:
                counts = df.groupby(x_col)[y_col].count()
                
                # Get labels and integer positions for plotting
                labels = counts.index.astype(str)
                positions = np.arange(len(labels))
                
                plt.bar(positions, counts.values, color="skyblue")
                plt.ylabel("Count")

                # Set ticks (This logic was already correct)
                if len(positions) > 50:
                    plt.xticks([])
                elif len(positions) > 20:
                    step = max(1, len(positions) // 20)
                    plt.xticks(positions[::step], labels[::step], rotation=45, ha="right")
                else:
                    plt.xticks(positions, labels, rotation=45, ha="right")
            
        plt.title(f"{y_col} vs {x_col} ({kind})")
        plt.xlabel(x_col)
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

generate_button.on_click(generate_plot)

# Display
display(widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_axis, y_axis]),
    widgets.HBox([generate_button, best_fit]),
    output
]))

# Trigger the update once to load the first table's columns
if table_names:
    update_columns(None)

In [28]:
# Simple Data Plot App for Beginners
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Connect to database
DB_NAME = "Dataset.db"
conn = sqlite3.connect(DB_NAME)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
table_names = [t for t in tables["name"].tolist() if t != "sqlite_sequence"]
conn.close()

# Widgets
table_dropdown = widgets.Dropdown(options=table_names, description="Table:")
plot_type = widgets.Dropdown(options=["Bar", "Scatter"], description="Plot Type:")
x_axis = widgets.Dropdown(description="X-Axis:")
y_axis = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style="success")
output = widgets.Output()

# Update dropdowns when table changes
def update_columns(change):
    table = table_dropdown.value
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f"SELECT * FROM '{table}' LIMIT 50", conn)
    conn.close()

    cols = df.columns.tolist()
    x_axis.options = cols
    y_axis.options = cols
    if len(cols) > 1:
        x_axis.value = cols[0]
        y_axis.value = cols[1]

table_dropdown.observe(update_columns, names="value")

# Plot when button is clicked
def generate_plot(b):
    with output:
        clear_output(wait=True)
        
        table = table_dropdown.value
        x_col = x_axis.value
        y_col = y_axis.value
        kind = plot_type.value

        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f"SELECT * FROM '{table}'", conn)
        conn.close()

        # Filter by age if it exists
        if "age" in df.columns:
            df["age"] = pd.to_numeric(df["age"], errors="coerce")
            df = df[(df["age"] >= 18) & (df["age"] <= 22)]
            print(f"Filter applied: {len(df)} students aged 18–22.")

        df = df.dropna(subset=[x_col, y_col])
        if df.empty:
            print("No data to plot.")
            return

        plt.figure(figsize=(7,4))
        
        # --- Scatter plot ---
        if kind == "Scatter":
            plt.scatter(df[x_col], df[y_col], color="skyblue", alpha=0.7)
            if best_fit.value:
                if df[x_col].dtype != 'object' and df[y_col].dtype != 'object':
                    m, b = np.polyfit(df[x_col], df[y_col], 1)
                    plt.plot(df[x_col], m*df[x_col]+b, color="red")

        # --- Bar plot ---
        else:
            # Case 1: Both are categorical (grouped bar)
            if df[x_col].dtype == "object" and df[y_col].dtype == "object":
                grouped = pd.crosstab(df[x_col], df[y_col])  # counts per category
                ages = np.arange(len(grouped))
                bar_width = 0.8 / len(grouped.columns)
                
                for i, col in enumerate(grouped.columns):
                    plt.bar(
                        ages + i*bar_width, 
                        grouped[col].values, 
                        width=bar_width, 
                        label=str(col), 
                        alpha=0.8
                    )
                
                plt.xticks(ages + bar_width*(len(grouped.columns)/2 - 0.5), grouped.index, rotation=45)
                plt.legend(title=y_col)
                plt.ylabel("Count")

            # Case 2: One numeric, one categorical (simple count)
            else:
                counts = df.groupby(x_col)[y_col].count()
                plt.bar(counts.index.astype(str), counts.values, color="skyblue")
                if len(counts) > 20:
                    step = max(1, len(counts)//20)
                    plt.xticks(counts.index[::step], rotation=45)
                else:
                    plt.xticks(rotation=45)
                plt.ylabel("Count")
            
        plt.title(f"{y_col} vs {x_col} ({kind})")
        plt.xlabel(x_col)
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

generate_button.on_click(generate_plot)

# Display
display(widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_axis, y_axis]),
    widgets.HBox([generate_button, best_fit]),
    output
]))


In [29]:
# Simple Data Plot App for Beginners
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Connect to database
DB_NAME = "Dataset.db"
conn = sqlite3.connect(DB_NAME)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
table_names = [t for t in tables["name"].tolist() if t != "sqlite_sequence"]
conn.close()

# Widgets
table_dropdown = widgets.Dropdown(options=table_names, description="Table:")
plot_type = widgets.Dropdown(options=["Bar", "Scatter"], description="Plot Type:")
x_axis = widgets.Dropdown(description="X-Axis:")
y_axis = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style="success")
output = widgets.Output()

# --- THIS FUNCTION IS NOW FIXED ---
# Update dropdowns when table or plot type changes
def update_columns(change):
    table = table_dropdown.value
    ptype = plot_type.value # Get the plot type
    
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f"SELECT * FROM '{table}' LIMIT 50", conn)
    conn.close()

    if ptype == "Scatter":
        # SCATTER: Only show numeric columns
        numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
        x_axis.options = numeric_cols
        y_axis.options = numeric_cols
        best_fit.layout.display = 'flex' # Show checkbox
    else:
        # BAR: Show all columns
        all_cols = df.columns.tolist()
        x_axis.options = all_cols
        y_axis.options = all_cols
        best_fit.layout.display = 'none' # Hide checkbox

    # Set default values
    if x_axis.options:
        x_axis.value = x_axis.options[0]
    if y_axis.options and len(y_axis.options) > 1:
        y_axis.value = y_axis.options[1]
    elif y_axis.options:
        y_axis.value = y_axis.options[0]

# --- OBSERVERS ARE NOW FIXED ---
table_dropdown.observe(update_columns, names="value")
plot_type.observe(update_columns, names="value") # Added this observer

# Plot when button is clicked
def generate_plot(b):
    with output:
        clear_output(wait=True)
        
        table = table_dropdown.value
        x_col = x_axis.value
        y_col = y_axis.value
        kind = plot_type.value
        
        if not x_col or not y_col:
            print("⚠️ Please select valid columns.")
            return

        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f"SELECT * FROM '{table}'", conn)
        conn.close()

        # Filter by age if it exists
        if "age" in df.columns:
            df["age"] = pd.to_numeric(df["age"], errors="coerce")
            df = df.dropna(subset=['age'])
            df = df[(df["age"] >= 18) & (df["age"] <= 22)]
            if kind == "Bar":
                 df['age'] = df['age'].round().astype(int)
            print(f"Filter applied: {len(df)} students aged 18–22.")

        # --- THIS IS THE OTHER FIX ---
        # The .dropna() is now moved inside each plot's logic

        plt.figure(figsize=(7,4))
        
        # --- Scatter plot ---
        if kind == "Scatter":
            # Convert to numeric and drop NAs
            df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
            df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
            df = df.dropna(subset=[x_col, y_col]) # Drop NAs here

            if df.empty:
                print("No numeric data to plot for scatter.")
                plt.close() # Close the empty figure
                return
                
            plt.scatter(df[x_col], df[y_col], color="skyblue", alpha=0.7)
            
            if best_fit.value:
                if len(df) > 1:
                    m, b = np.polyfit(df[x_col], df[y_col], 1)
                    plt.plot(df[x_col], m*df[x_col]+b, color="red")

        # --- Bar plot ---
        else:
            df = df.dropna(subset=[x_col, y_col]) # Drop NAs here
            if df.empty:
                print("No data to plot for bar chart.")
                plt.close() # Close the empty figure
                return

            # If Y is categorical (like 'gender'), make a grouped bar chart
            if df[y_col].dtype == "object":
                # Case 1: Grouped bar chart (e.g., X=age, Y=gender)
                grouped = pd.crosstab(df[x_col], df[y_col]) 
                ages = np.arange(len(grouped))
                bar_width = 0.8 / len(grouped.columns) 
                
                for i, col in enumerate(grouped.columns):
                    plt.bar(
                        ages + i*bar_width, 
                        grouped[col].values, 
                        width=bar_width, 
                        label=str(col), 
                        alpha=0.8
                    )
                
                plt.legend(title=y_col)
                plt.ylabel("Count")

                # Fix for clumped labels
                labels = grouped.index.astype(str)
                positions = ages + bar_width*(len(grouped.columns)/2 - 0.5)
                if len(positions) > 50:
                    plt.xticks([])
                else:
                    plt.xticks(positions, labels, rotation=45, ha="right")

            # Case 2: Simple count (e.g., X=age, Y=student_id)
            else:
                counts = df.groupby(x_col)[y_col].count()
                labels = counts.index.astype(str)
                positions = np.arange(len(labels))
                
                plt.bar(positions, counts.values, color="skyblue")
                plt.ylabel("Count")

                # Fix for clumped labels
                if len(positions) > 50:
                    plt.xticks([])
                else:
                    plt.xticks(positions, labels, rotation=45, ha="right")
            
        plt.title(f"{y_col} vs {x_col} ({kind})")
        plt.xlabel(x_col)
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

generate_button.on_click(generate_plot)

# Display
display(widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_axis, y_axis]), # Kept your original swapped layout
    widgets.HBox([generate_button, best_fit]),
    output
]))

# Trigger the update once to load the first table's columns
if table_names:
    update_columns(None)

In [30]:
# Simple Data Plot App for Beginners
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Connect to database
DB_NAME = "Dataset.db"
conn = sqlite3.connect(DB_NAME)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
table_names = [t for t in tables["name"].tolist() if t != "sqlite_sequence"]
conn.close()

# Widgets
table_dropdown = widgets.Dropdown(options=table_names, description="Table:")
plot_type = widgets.Dropdown(options=["Bar", "Scatter"], description="Plot Type:")
x_axis = widgets.Dropdown(description="X-Axis:")
y_axis = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style="success")
output = widgets.Output()

# Update dropdowns when table or plot type changes
def update_columns(change):
    table = table_dropdown.value
    ptype = plot_type.value # Get the plot type
    
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f"SELECT * FROM '{table}' LIMIT 50", conn)
    conn.close()

    if ptype == "Scatter":
        # SCATTER: Only show numeric columns
        numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
        x_axis.options = numeric_cols
        y_axis.options = numeric_cols
        best_fit.layout.display = 'flex' # Show checkbox
    else:
        # BAR: Show all columns
        all_cols = df.columns.tolist()
        x_axis.options = all_cols
        y_axis.options = all_cols
        best_fit.layout.display = 'none' # Hide checkbox

    # Set default values
    if x_axis.options:
        x_axis.value = x_axis.options[0]
    if y_axis.options and len(y_axis.options) > 1:
        y_axis.value = y_axis.options[1]
    elif y_axis.options:
        y_axis.value = y_axis.options[0]

# Observers
table_dropdown.observe(update_columns, names="value")
plot_type.observe(update_columns, names="value")

# Plot when button is clicked
def generate_plot(b):
    with output:
        clear_output(wait=True)
        
        table = table_dropdown.value
        x_col = x_axis.value
        y_col = y_axis.value
        kind = plot_type.value
        
        if not x_col or not y_col:
            print("⚠️ Please select valid columns.")
            return

        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f"SELECT * FROM '{table}'", conn)
        conn.close()

        # Filter by age if it exists
        if "age" in df.columns:
            df["age"] = pd.to_numeric(df["age"], errors="coerce")
            df = df.dropna(subset=['age'])
            df = df[(df["age"] >= 18) & (df["age"] <= 22)]
            # We round 'age' *before* plotting
            if 'age' in df.columns:
                df['age'] = df['age'].round()
            print(f"Filter applied: {len(df)} students aged 18–22.")

        plt.figure(figsize=(7,4))
        
        # --- Scatter plot ---
        if kind == "Scatter":
            # Convert to numeric and drop NAs
            df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
            df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
            df = df.dropna(subset=[x_col, y_col]) # Drop NAs here

            if df.empty:
                print("No numeric data to plot for scatter.")
                plt.close() # Close the empty figure
                return
                
            plt.scatter(df[x_col], df[y_col], color="skyblue", alpha=0.7)
            
            if best_fit.value:
                if len(df) > 1:
                    m, b = np.polyfit(df[x_col], df[y_col], 1)
                    plt.plot(df[x_col], m*df[x_col]+b, color="red")

        # --- Bar plot ---
        else:
            df = df.dropna(subset=[x_col, y_col]) # Drop NAs here
            if df.empty:
                print("No data to plot for bar chart.")
                plt.close() # Close the empty figure
                return

            # If Y is categorical (like 'gender'), make a grouped bar chart
            if df[y_col].dtype == "object":
                # Case 1: Grouped bar chart (e.g., X=age, Y=gender)
                grouped = pd.crosstab(df[x_col], df[y_col]) 
                ages = np.arange(len(grouped))
                bar_width = 0.8 / len(grouped.columns) 
                
                for i, col in enumerate(grouped.columns):
                    plt.bar(
                        ages + i*bar_width, 
                        grouped[col].values, 
                        width=bar_width, 
                        label=str(col), 
                        alpha=0.8
                    )
                
                plt.legend(title=y_col)
                plt.ylabel("Count") # This is still a count

                # Fix for clumped labels
                labels = grouped.index.astype(str)
                positions = ages + bar_width*(len(grouped.columns)/2 - 0.5)
                if len(positions) > 50:
                    plt.xticks([])
                else:
                    plt.xticks(positions, labels, rotation=45, ha="right")

            # --- THIS IS THE FIX ---
            # Case 2: Y-column is numeric, so show AVERAGE
            else:
                # Calculate the mean (average)
                agg_data = df.groupby(x_col)[y_col].mean()
                labels = agg_data.index.astype(str)
                positions = np.arange(len(labels))
                
                plt.bar(positions, agg_data.values, color="skyblue")
                # Set the label to show it's an average
                plt.ylabel(f"Average {y_col}") 

                # Fix for clumped labels (handles X-axis)
                if len(positions) > 50:
                    plt.xticks([])
                else:
                    plt.xticks(positions, labels, rotation=45, ha="right")
            # --- END FIX ---
            
        plt.title(f"{y_col} vs {x_col} ({kind})")
        plt.xlabel(x_col)
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

generate_button.on_click(generate_plot)

# Display
display(widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_axis, y_axis]), # Kept your original swapped layout
    widgets.HBox([generate_button, best_fit]),
    output
]))

# Trigger the update once to load the first table's columns
if table_names:
    update_columns(None)

In [31]:
# Simple Data Plot App for Beginners
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Connect to database
DB_NAME = "Dataset.db"
conn = sqlite3.connect(DB_NAME)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
table_names = [t for t in tables["name"].tolist() if t != "sqlite_sequence"]
conn.close()

# Widgets
table_dropdown = widgets.Dropdown(options=table_names, description="Table:")
plot_type = widgets.Dropdown(options=["Bar", "Scatter"], description="Plot Type:")
x_axis = widgets.Dropdown(description="X-Axis:")
y_axis = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style="success")
output = widgets.Output()

# Update dropdowns when table or plot type changes
def update_columns(change):
    table = table_dropdown.value
    ptype = plot_type.value # Get the plot type
    
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f"SELECT * FROM '{table}' LIMIT 50", conn)
    conn.close()

    if ptype == "Scatter":
        # SCATTER: Only show numeric columns
        numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
        x_axis.options = numeric_cols
        y_axis.options = numeric_cols
        best_fit.layout.display = 'flex' # Show checkbox
    else:
        # BAR: Show all columns
        all_cols = df.columns.tolist()
        x_axis.options = all_cols
        y_axis.options = all_cols
        best_fit.layout.display = 'none' # Hide checkbox

    # Set default values
    if x_axis.options:
        x_axis.value = x_axis.options[0]
    if y_axis.options and len(y_axis.options) > 1:
        y_axis.value = y_axis.options[1]
    elif y_axis.options:
        y_axis.value = y_axis.options[0]

# Observers
table_dropdown.observe(update_columns, names="value")
plot_type.observe(update_columns, names="value")

# Plot when button is clicked
def generate_plot(b):
    with output:
        clear_output(wait=True)
        
        table = table_dropdown.value
        x_col = x_axis.value
        y_col = y_axis.value
        kind = plot_type.value
        
        if not x_col or not y_col:
            print("⚠️ Please select valid columns.")
            return

        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f"SELECT * FROM '{table}'", conn)
        conn.close()

        # Filter by age if it exists
        if "age" in df.columns:
            df["age"] = pd.to_numeric(df["age"], errors="coerce")
            df = df.dropna(subset=['age'])
            df = df[(df["age"] >= 18) & (df["age"] <= 22)]
            # We round 'age' *before* plotting
            if 'age' in df.columns:
                df['age'] = df['age'].round()
            print(f"Filter applied: {len(df)} students aged 18–22.")

        plt.figure(figsize=(7,4))
        
        # --- Scatter plot ---
        if kind == "Scatter":
            # Convert to numeric and drop NAs
            df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
            df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
            df = df.dropna(subset=[x_col, y_col]) # Drop NAs here

            if df.empty:
                print("No numeric data to plot for scatter.")
                plt.close() # Close the empty figure
                return
                
            plt.scatter(df[x_col], df[y_col], color="skyblue", alpha=0.7)
            
            if best_fit.value:
                if len(df) > 1:
                    m, b = np.polyfit(df[x_col], df[y_col], 1)
                    plt.plot(df[x_col], m*df[x_col]+b, color="red")

        # --- Bar plot ---
        else:
            df = df.dropna(subset=[x_col, y_col]) # Drop NAs here
            if df.empty:
                print("No data to plot for bar chart.")
                plt.close() # Close the empty figure
                return
            
            # --- THIS IS THE NEW, SMARTER LOGIC ---
            is_x_categorical = df[x_col].dtype == 'object'
            is_y_categorical = df[y_col].dtype == 'object'

            # Case 1: Y is categorical (e.g., X=age, Y=gender)
            # This is the grouped "Count" chart
            if is_y_categorical:
                grouped = pd.crosstab(df[x_col], df[y_col]) 
                ages = np.arange(len(grouped))
                bar_width = 0.8 / len(grouped.columns) 
                
                for i, col in enumerate(grouped.columns):
                    plt.bar(
                        ages + i*bar_width, 
                        grouped[col].values, 
                        width=bar_width, 
                        label=str(col), 
                        alpha=0.8
                    )
                
                plt.legend(title=y_col)
                plt.ylabel("Count") # This is a count

                labels = grouped.index.astype(str)
                positions = ages + bar_width*(len(grouped.columns)/2 - 0.5)
                if len(positions) > 50:
                    plt.xticks([])
                else:
                    plt.xticks(positions, labels, rotation=45, ha="right")

            # Case 2: Y is numeric and X is categorical (e.g., X=gender, Y=age)
            # This is the "Average" chart
            elif is_x_categorical and not is_y_categorical:
                agg_data = df.groupby(x_col)[y_col].mean()
                labels = agg_data.index.astype(str)
                positions = np.arange(len(labels))
                
                plt.bar(positions, agg_data.values, color="skyblue")
                plt.ylabel(f"Average {y_col}") # This is an average

                if len(positions) > 50:
                    plt.xticks([])
                else:
                    plt.xticks(positions, labels, rotation=45, ha="right")

            # Case 3: Both X and Y are numeric (e.g., X=age, Y=student_id)
            # This is the simple "Count" chart
            else:
                agg_data = df.groupby(x_col)[y_col].count()
                labels = agg_data.index.astype(str)
                positions = np.arange(len(labels))
                
                plt.bar(positions, agg_data.values, color="skyblue")
                plt.ylabel(f"Count of {y_col}") # This is a count

                if len(positions) > 50:
                    plt.xticks([])
                else:
                    plt.xticks(positions, labels, rotation=45, ha="right")
            # --- END NEW LOGIC ---
            
        plt.title(f"{y_col} vs {x_col} ({kind})")
        plt.xlabel(x_col)
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

generate_button.on_click(generate_plot)

# Display
display(widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_axis, y_axis]), # Kept your original swapped layout
    widgets.HBox([generate_button, best_fit]),
    output
]))

# Trigger the update once to load the first table's columns
if table_names:
    update_columns(None)

In [32]:
# Simple Data Plot App for Beginners
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Connect to database
DB_NAME = "Dataset.db"
conn = sqlite3.connect(DB_NAME)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
table_names = [t for t in tables["name"].tolist() if t != "sqlite_sequence"]
conn.close()

# Widgets
table_dropdown = widgets.Dropdown(options=table_names, description="Table:")
plot_type = widgets.Dropdown(options=["Bar", "Scatter"], description="Plot Type:")
x_axis = widgets.Dropdown(description="X-Axis:")
y_axis = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style="success")
output = widgets.Output()

# Update dropdowns when table or plot type changes
def update_columns(change):
    table = table_dropdown.value
    ptype = plot_type.value # Get the plot type
    
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f"SELECT * FROM '{table}' LIMIT 50", conn)
    conn.close()

    if ptype == "Scatter":
        # SCATTER: Only show numeric columns
        numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
        x_axis.options = numeric_cols
        y_axis.options = numeric_cols
        best_fit.layout.display = 'flex' # Show checkbox
    else:
        # BAR: Show all columns
        all_cols = df.columns.tolist()
        x_axis.options = all_cols
        y_axis.options = all_cols
        best_fit.layout.display = 'none' # Hide checkbox

    # Set default values
    if x_axis.options:
        x_axis.value = x_axis.options[0]
    if y_axis.options and len(y_axis.options) > 1:
        y_axis.value = y_axis.options[1]
    elif y_axis.options:
        y_axis.value = y_axis.options[0]

# Observers
table_dropdown.observe(update_columns, names="value")
plot_type.observe(update_columns, names="value")

# Plot when button is clicked
def generate_plot(b):
    with output:
        clear_output(wait=True)
        
        table = table_dropdown.value
        x_col = x_axis.value
        y_col = y_axis.value
        kind = plot_type.value
        
        if not x_col or not y_col:
            print("⚠️ Please select valid columns.")
            return

        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f"SELECT * FROM '{table}'", conn)
        conn.close()

        # Filter by age if it exists
        if "age" in df.columns:
            df["age"] = pd.to_numeric(df["age"], errors="coerce")
            original_count = len(df) # Get count before filtering
            df = df.dropna(subset=['age'])
            df = df[(df["age"] >= 18) & (df["age"] <= 22)]
            # We round 'age' *before* plotting
            if 'age' in df.columns:
                df['age'] = df['age'].round()
            print(f"Filter applied: Kept {len(df)} of {original_count} students aged 18–22.")

        plt.figure(figsize=(7,4))
        
        # --- Scatter plot ---
        if kind == "Scatter":
            # Convert to numeric and drop NAs
            df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
            df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
            df = df.dropna(subset=[x_col, y_col]) # Drop NAs here

            if df.empty:
                print("No numeric data to plot for scatter.")
                plt.close() # Close the empty figure
                return
                
            plt.scatter(df[x_col], df[y_col], color="skyblue", alpha=0.7)
            
            if best_fit.value:
                if len(df) > 1:
                    m, b = np.polyfit(df[x_col], df[y_col], 1)
                    plt.plot(df[x_col], m*df[x_col]+b, color="red")

        # --- Bar plot ---
        else:
            # Try to convert columns to numeric where possible
            # This helps our "is_categorical" check
            x_data = pd.to_numeric(df[x_col], errors='coerce')
            if x_data.isnull().all(): # Failed to convert (it's text)
                x_data = df[x_col]
                
            y_data = pd.to_numeric(df[y_col], errors='coerce')
            if y_data.isnull().all(): # Failed to convert (it's text)
                y_data = df[y_col]
                
            df[x_col] = x_data
            df[y_col] = y_data
            
            df = df.dropna(subset=[x_col, y_col]) # Drop NAs here
            if df.empty:
                print("No data to plot for bar chart.")
                plt.close() # Close the empty figure
                return
            
            # --- THIS IS THE NEW, SMARTER LOGIC ---
            
            # A column is "categorical" if it's an 'object' (text)
            # OR if it's numeric but has a small number of unique values.
            def is_col_categorical(col_name, unique_thresh=25):
                # Check if dtype is object
                if df[col_name].dtype == 'object':
                    return True
                # Check if it's numeric and has few unique values
                if pd.api.types.is_numeric_dtype(df[col_name]):
                    if df[col_name].nunique() < unique_thresh:
                        return True
                return False

            is_x_categorical = is_col_categorical(x_col)
            is_y_categorical = is_col_categorical(y_col)
            # --- END NEW LOGIC ---
            
            # Case 1: Y is categorical (e.g., X=age, Y=gender)
            # This is the grouped "Count" chart
            if is_y_categorical:
                grouped = pd.crosstab(df[x_col], df[y_col]) 
                ages = np.arange(len(grouped))
                bar_width = 0.8 / len(grouped.columns) 
                
                for i, col in enumerate(grouped.columns):
                    plt.bar(
                        ages + i*bar_width, 
                        grouped[col].values, 
                        width=bar_width, 
                        label=str(col), 
                        alpha=0.8
                    )
                
                plt.legend(title=y_col)
                plt.ylabel("Count") # This is a count

                labels = grouped.index.astype(str)
                positions = ages + bar_width*(len(grouped.columns)/2 - 0.5)
                if len(positions) > 50:
                    plt.xticks([])
                else:
                    plt.xticks(positions, labels, rotation=45, ha="right")

            # Case 2: Y is numeric (not categorical) and X is categorical
            # (e.g., X=gender, Y=age)
            # This is the "Average" chart
            elif is_x_categorical and not is_y_categorical:
                agg_data = df.groupby(x_col)[y_col].mean()
                labels = agg_data.index.astype(str)
                positions = np.arange(len(labels))
                
                plt.bar(positions, agg_data.values, color="skyblue")
                plt.ylabel(f"Average {y_col}") # This is an average

                if len(positions) > 50:
                    plt.xticks([])
                else:
                    plt.xticks(positions, labels, rotation=45, ha="right")

            # Case 3: Both X and Y are numeric (and not categorical)
            # (e.g., X=age, Y=student_id)
            # This is the simple "Count" chart
            else:
                agg_data = df.groupby(x_col)[y_col].count()
                labels = agg_data.index.astype(str)
                positions = np.arange(len(labels))
                
                plt.bar(positions, agg_data.values, color="skyblue")
                plt.ylabel(f"Count of {y_col}") # This is a count

                if len(positions) > 50:
                    plt.xticks([])
                else:
                    plt.xticks(positions, labels, rotation=45, ha="right")
            # --- END LOGIC ---
            
        plt.title(f"{y_col} vs {x_col} ({kind})")
        plt.xlabel(x_col)
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

generate_button.on_click(generate_plot)

# Display
display(widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_axis, y_axis]), # Kept your original swapped layout
    widgets.HBox([generate_button, best_fit]),
    output
]))

# Trigger the update once to load the first table's columns
if table_names:
    update_columns(None)

In [33]:
# Simple Data Plot App for Beginners
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Connect to database
DB_NAME = "Dataset.db"
conn = sqlite3.connect(DB_NAME)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
table_names = [t for t in tables["name"].tolist() if t != "sqlite_sequence"]
conn.close()

# Widgets
table_dropdown = widgets.Dropdown(options=table_names, description="Table:")
plot_type = widgets.Dropdown(options=["Bar", "Scatter"], description="Plot Type:")
x_axis = widgets.Dropdown(description="X-Axis:")
y_axis = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style="success")
output = widgets.Output()

# Update dropdowns when table or plot type changes
def update_columns(change):
    table = table_dropdown.value
    ptype = plot_type.value # Get the plot type
    
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query(f"SELECT * FROM '{table}' LIMIT 50", conn)
    conn.close()

    if ptype == "Scatter":
        # SCATTER: Only show numeric columns
        numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
        x_axis.options = numeric_cols
        y_axis.options = numeric_cols
        best_fit.layout.display = 'flex' # Show checkbox
    else:
        # BAR: Show all columns
        all_cols = df.columns.tolist()
        x_axis.options = all_cols
        y_axis.options = all_cols
        best_fit.layout.display = 'none' # Hide checkbox

    # Set default values
    if x_axis.options:
        x_axis.value = x_axis.options[0]
    if y_axis.options and len(y_axis.options) > 1:
        y_axis.value = y_axis.options[1]
    elif y_axis.options:
        y_axis.value = y_axis.options[0]

# Observers
table_dropdown.observe(update_columns, names="value")
plot_type.observe(update_columns, names="value")

# Plot when button is clicked
def generate_plot(b):
    with output:
        clear_output(wait=True)
        
        table = table_dropdown.value
        x_col = x_axis.value
        y_col = y_axis.value
        kind = plot_type.value
        
        if not x_col or not y_col:
            print("⚠️ Please select valid columns.")
            return

        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f"SELECT * FROM '{table}'", conn)
        conn.close()

        # Filter by age if it exists
        if "age" in df.columns:
            df["age"] = pd.to_numeric(df["age"], errors="coerce")
            df = df.dropna(subset=['age'])
            df = df[(df["age"] >= 18) & (df["age"] <= 22)]
            # We round 'age' *before* plotting
            if 'age' in df.columns:
                df['age'] = df['age'].round()
            print(f"Filter applied: {len(df)} students aged 18–22.")

        plt.figure(figsize=(7,4))
        
        # --- Scatter plot ---
        if kind == "Scatter":
            # Convert to numeric and drop NAs
            df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
            df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
            df = df.dropna(subset=[x_col, y_col]) # Drop NAs here

            if df.empty:
                print("No numeric data to plot for scatter.")
                plt.close() # Close the empty figure
                return
                
            plt.scatter(df[x_col], df[y_col], color="skyblue", alpha=0.7)
            
            if best_fit.value:
                if len(df) > 1:
                    m, b = np.polyfit(df[x_col], df[y_col], 1)
                    plt.plot(df[x_col], m*df[x_col]+b, color="red")

        # --- Bar plot ---
        else:
            df = df.dropna(subset=[x_col, y_col]) # Drop NAs here
            if df.empty:
                print("No data to plot for bar chart.")
                plt.close() # Close the empty figure
                return
            
            # --- THIS IS THE NEW, SMARTER LOGIC ---
            is_x_categorical = df[x_col].dtype == 'object'
            is_y_categorical = df[y_col].dtype == 'object'

            # Case 1: Y is categorical (e.g., X=age, Y=gender)
            # This is the grouped "Count" chart
            if is_y_categorical:
                grouped = pd.crosstab(df[x_col], df[y_col]) 
                ages = np.arange(len(grouped))
                bar_width = 0.8 / len(grouped.columns) 
                
                for i, col in enumerate(grouped.columns):
                    plt.bar(
                        ages + i*bar_width, 
                        grouped[col].values, 
                        width=bar_width, 
                        label=str(col), 
                        alpha=0.8
                    )
                
                plt.legend(title=y_col)
                plt.ylabel("Count") # This is a count

                labels = grouped.index.astype(str)
                positions = ages + bar_width*(len(grouped.columns)/2 - 0.5)
                if len(positions) > 50:
                    plt.xticks([])
                else:
                    plt.xticks(positions, labels, rotation=45, ha="right")

            # Case 2: Y is numeric and X is categorical (e.g., X=gender, Y=age)
            # This is the "Average" chart
            elif is_x_categorical and not is_y_categorical:
                agg_data = df.groupby(x_col)[y_col].mean()
                labels = agg_data.index.astype(str)
                positions = np.arange(len(labels))
                
                plt.bar(positions, agg_data.values, color="skyblue")
                plt.ylabel(f"Average {y_col}") # This is an average

                if len(positions) > 50:
                    plt.xticks([])
                else:
                    plt.xticks(positions, labels, rotation=45, ha="right")

            # Case 3: Both X and Y are numeric (e.g., X=age, Y=student_id)
            # This is the simple "Count" chart
            else:
                agg_data = df.groupby(x_col)[y_col].count()
                labels = agg_data.index.astype(str)
                positions = np.arange(len(labels))
                
                plt.bar(positions, agg_data.values, color="skyblue")
                plt.ylabel(f"Count of {y_col}") # This is a count

                if len(positions) > 50:
                    plt.xticks([])
                else:
                    plt.xticks(positions, labels, rotation=45, ha="right")
            # --- END NEW LOGIC ---
            
        plt.title(f"{y_col} vs {x_col} ({kind})")
        plt.xlabel(x_col)
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

generate_button.on_click(generate_plot)

# Display
display(widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_axis, y_axis]), # Kept your original swapped layout
    widgets.HBox([generate_button, best_fit]),
    output
]))

# Trigger the update once to load the first table's columns
if table_names:
    update_columns(None)

In [34]:
# Simple Data Plot App for Beginners
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Connect to database
DB_NAME = "Dataset.db"
conn = sqlite3.connect(DB_NAME)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
table_names = [t for t in tables["name"].tolist() if t != "sqlite_sequence"]
conn.close()

# Widgets
table_dropdown = widgets.Dropdown(options=table_names, description="Table:")
plot_type = widgets.Dropdown(options=["Bar", "Scatter"], description="Plot Type:")
x_axis = widgets.Dropdown(description="X-Axis:")
y_axis = widgets.Dropdown(description="Y-Axis:")
best_fit = widgets.Checkbox(value=True, description="Add Best-Fit Line")
generate_button = widgets.Button(description="Generate Plot", button_style="success")
output = widgets.Output()

# Update dropdowns when table or plot type changes
def update_columns(change):
    table = table_dropdown.value
    ptype = plot_type.value # Get the plot type
    
    conn = sqlite3.connect(DB_NAME)
    # Read only a few rows to quickly get column names and types
    df = pd.read_sql_query(f"SELECT * FROM '{table}' LIMIT 50", conn)
    conn.close()

    if ptype == "Scatter":
        # SCATTER: Only show numeric columns
        numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
        x_axis.options = numeric_cols
        y_axis.options = numeric_cols
        best_fit.layout.display = 'flex' # Show checkbox
    else:
        # BAR: Show all columns
        all_cols = df.columns.tolist()
        x_axis.options = all_cols
        y_axis.options = all_cols
        best_fit.layout.display = 'none' # Hide checkbox

    # Set default values
    if x_axis.options:
        x_axis.value = x_axis.options[0]
    if y_axis.options and len(y_axis.options) > 1:
        # Try to set a different default for Y if possible
        y_axis.value = y_axis.options[1]
    elif y_axis.options:
        y_axis.value = y_axis.options[0]

# Observers
table_dropdown.observe(update_columns, names="value")
plot_type.observe(update_columns, names="value")

# Plot when button is clicked
def generate_plot(b):
    with output:
        clear_output(wait=True)
        
        table = table_dropdown.value
        x_col = x_axis.value
        y_col = y_axis.value
        kind = plot_type.value
        
        if not x_col or not y_col:
            print("⚠️ Please select valid columns.")
            return

        conn = sqlite3.connect(DB_NAME)
        df = pd.read_sql_query(f"SELECT * FROM '{table}'", conn)
        conn.close()

        # --- Age Filter ---
        if "age" in df.columns:
            df["age"] = pd.to_numeric(df["age"], errors="coerce")
            original_count = len(df) # Get count before filtering
            df = df.dropna(subset=['age'])
            df = df[(df["age"] >= 18) & (df["age"] <= 22)]
            # We round 'age' *before* plotting
            if 'age' in df.columns:
                df['age'] = df['age'].round()
            print(f"Filter applied: Kept {len(df)} of {original_count} students aged 18–22.")

        plt.figure(figsize=(7,4))
        
        # --- Scatter plot ---
        if kind == "Scatter":
            # Convert to numeric and drop NAs
            df[x_col] = pd.to_numeric(df[x_col], errors='coerce')
            df[y_col] = pd.to_numeric(df[y_col], errors='coerce')
            df = df.dropna(subset=[x_col, y_col]) # Drop NAs here

            if df.empty:
                print("No numeric data to plot for scatter.")
                plt.close() # Close the empty figure
                return
                
            plt.scatter(df[x_col], df[y_col], color="skyblue", alpha=0.7)
            
            if best_fit.value:
                if len(df) > 1:
                    m, b = np.polyfit(df[x_col], df[y_col], 1)
                    plt.plot(df[x_col], m*df[x_col]+b, color="red")

        # --- Bar plot ---
        else:
            # Try to convert columns to numeric where possible
            # This helps our "is_categorical" check
            x_data = pd.to_numeric(df[x_col], errors='coerce')
            if x_data.isnull().all(): # Failed to convert (it's text)
                x_data = df[x_col]
                
            y_data = pd.to_numeric(df[y_col], errors='coerce')
            if y_data.isnull().all(): # Failed to convert (it't text)
                y_data = df[y_col]
                
            df[x_col] = x_data
            df[y_col] = y_data
            
            df = df.dropna(subset=[x_col, y_col]) # Drop NAs here
            if df.empty:
                print("No data to plot for bar chart.")
                plt.close() # Close the empty figure
                return
            
            # --- THIS IS THE NEW, SMARTER LOGIC ---
            
            # A column is "categorical" if it's an 'object' (text)
            # OR if it's numeric but has a small number of unique values.
            def is_col_categorical(col_name, unique_thresh=25):
                """Helper function to check if a column should be treated as categorical."""
                # Check if dtype is object (text)
                if pd.api.types.is_object_dtype(df[col_name]):
                    return True
                # Check if it's numeric and has few unique values
                if pd.api.types.is_numeric_dtype(df[col_name]):
                    if df[col_name].nunique() < unique_thresh:
                        # Don't treat a "count" column (like ID) as categorical
                        if 'id' not in str(col_name).lower():
                            return True
                return False

            is_x_categorical = is_col_categorical(x_col)
            is_y_categorical = is_col_categorical(y_col)
            # --- END NEW LOGIC ---
            
            # Case 1: Y is categorical (e.g., X=age, Y=gender)
            # This is the grouped "Count" chart
            if is_y_categorical:
                grouped = pd.crosstab(df[x_col], df[y_col]) 
                # Sort by index (X-axis) to ensure it's in order
                grouped = grouped.sort_index()
                
                indices = np.arange(len(grouped))
                bar_width = 0.8 / len(grouped.columns) 
                
                for i, col in enumerate(grouped.columns):
                    plt.bar(
                        indices + i*bar_width, 
                        grouped[col].values, 
                        width=bar_width, 
                        label=str(col), 
                        alpha=0.8
                    )
                
                plt.legend(title=y_col)
                plt.ylabel("Count") # This is a count

                labels = grouped.index.astype(str)
                positions = indices + bar_width*(len(grouped.columns)/2 - 0.5)
                if len(positions) > 50:
                    plt.xticks([])
                else:
                    plt.xticks(positions, labels, rotation=45, ha="right")

            # Case 2: Y is numeric (not categorical) and X is categorical
            # (e.g., X=future_career_concerns, Y=study_load)
            # This is the "Average" (Mean) chart
            elif is_x_categorical and not is_y_categorical:
                agg_data = df.groupby(x_col)[y_col].mean()
                # Sort by index (X-axis) to ensure it's in order (e.g., 0, 1, 2, 3...)
                agg_data = agg_data.sort_index()
                
                labels = agg_data.index.astype(str)
                positions = np.arange(len(labels))
                
                plt.bar(positions, agg_data.values, color="skyblue")
                plt.ylabel(f"Average {y_col}") # This is an average

                if len(positions) > 50:
                    plt.xticks([])
                else:
                    plt.xticks(positions, labels, rotation=45, ha="right")

            # Case 3: Both X and Y are numeric (and not categorical)
            # (e.g., X=age, Y=student_id)
            # This is the simple "Count" chart
            else:
                agg_data = df.groupby(x_col)[y_col].count()
                # Sort by index (X-axis) to ensure it's in order
                agg_data = agg_data.sort_index()
                
                labels = agg_data.index.astype(str)
                positions = np.arange(len(labels))
                
                plt.bar(positions, agg_data.values, color="skyblue")
                plt.ylabel(f"Count of {y_col}") # This is a count

                if len(positions) > 50:
                    plt.xticks([])
                else:
                    plt.xticks(positions, labels, rotation=45, ha="right")
            # --- END LOGIC ---
            
        plt.title(f"{y_col} vs {x_col} ({kind})")
        plt.xlabel(x_col)
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

generate_button.on_click(generate_plot)

# Display
display(widgets.VBox([
    widgets.HBox([table_dropdown, plot_type]),
    widgets.HBox([x_axis, y_axis]), # Kept your original swapped layout
    widgets.HBox([generate_button, best_fit]),
    output
]))

# Trigger the update once to load the first table's columns
if table_names:
    update_columns(None)